# Multi-modal Gesture Recognition: Plan

Goal: Win a medal by building a robust pipeline with strong validation and fast iterations.

High-level plan:
- Environment gate: verify GPU availability and correct CUDA stack.
- Data audit: inspect training.csv, test.csv, randomPredictions.csv format and archives content.
- Validation: design user-independent CV if possible (mirror test), else robust grouped CV by sequence/user if fields exist in training.csv.
- Baseline:
  - Stage 1: Fast baseline predicting from skeleton/joints if readily available; else simple heuristics or provided sample code to establish a working submission.
  - Stage 2: Extract features from .mat files (skeleton, joint orientation, audio/rgb/depth descriptors) to create sequence-level features.
- Modeling:
  - Start with classical models on aggregated temporal features (XGBoost/CatBoost with GPU).
  - Explore sequence models on skeleton (GRU/LSTM/Temporal CNN) if time permits.
- Ensembling: blend diverse models (tree + RNN) using OOF-calibrated weights.
- Error analysis: bucket by class, confidence, and sequence length to guide FE iterations.

Milestones (request expert review at each):
1) Plan (this) → ask for medal-winning strategies and pitfalls.
2) Data loading/EDA & archive structure understanding.
3) Baseline working submission.
4) Feature engineering v1 (skeleton-centric).
5) Model v1 (GPU trees) + solid CV.
6) Improvements/ensembling.

Next: Run environment and data sanity checks.

Note: We'll cache heavy transforms and always log progress/elapsed times.

---

In [2]:
# 0) Environment and data sanity checks
import os, sys, subprocess, time, tarfile, shutil, glob, math, random, warnings
warnings.filterwarnings('ignore')
start_ts = time.time()

def run(cmd):
    print("$", " ".join(cmd), flush=True)
    out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(out.stdout, flush=True)
    return out.stdout

print("[Env] nvidia-smi:")
run(['bash','-lc','nvidia-smi || true'])

print("[Env] Python:", sys.version)
print("[CWD]", os.getcwd())
print("[LS]")
run(['bash','-lc','ls -alh'])

import pandas as pd
print("[Load] training.csv / test.csv / randomPredictions.csv heads")
train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('test.csv')
rand_df = pd.read_csv('randomPredictions.csv')
print("training.csv shape:", train_df.shape); print(train_df.head(3))
print("test.csv shape:", test_df.shape); print(test_df.head(3))
print("randomPredictions.csv shape:", rand_df.shape); print(rand_df.head(3))

print("[Archives] List first few members of training/validation/test archives")
def list_tar(path, n=10):
    try:
        with tarfile.open(path, 'r:gz') as tf:
            names = tf.getnames()
            print(f"{path}: {len(names)} files. First {n}:")
            for x in names[:n]:
                print(" -", x)
    except Exception as e:
        print(f"Failed to open {path}: {e}")

for p in ['training1.tar.gz','training2.tar.gz','training3.tar.gz','validation1.tar.gz','validation2.tar.gz','validation3.tar.gz','test.tar.gz']:
    if os.path.exists(p):
        list_tar(p, n=10)

elapsed = time.time() - start_ts
print(f"[Done sanity checks] Elapsed: {elapsed:.1f}s", flush=True)

[Env] nvidia-smi:
$ bash -lc nvidia-smi || true


Mon Sep 29 02:38:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

[Env] Python: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
[CWD] /var/lib/simon/agent_run_states/multi-modal-gesture-recognition-20250929-022629
[LS]
$ bash -lc ls -alh


total 22G
drwxrwxrwx 3 simon simon 4.0K Sep 29 02:38 .
drwxr-xr-x 3 root  root  4.0K Sep 29 02:29 ..
-rw-r--r-- 1 simon simon  182 Sep 29 02:38 .00_eda_and_planning_kernel_state.json
-rw-r--r-- 1 simon simon 4.8K Sep 29 02:38 00_eda_and_planning.ipynb
drwxr-xr-x 3 simon simon 4.0K Sep 29 02:37 agent_metadata
-rw-rw-rw- 1 simon simon  22K Sep 29 02:35 description.md
-rw-rw-rw- 1 simon simon 2.1G Sep 29 02:35 devel01-40.7z
-rw-rw-r-- 1 simon simon  38K Sep 29 02:38 docker_run.log
-rw-rw-rw- 1 simon simon 5.3K Sep 29 02:34 randomPredictions.csv
-rw-r--r-- 1 simon simon 2.0K Sep 29 02:37 requirements.txt
-rw-rw-rw- 1 simon simon 7.6K Sep 29 02:35 sample_code_mmrgc.zip
-rw-r--r-- 1 simon simon 3.9K Sep 29 02:29 task.txt
-rw-rw-rw- 1 simon simon  478 Sep 29 02:34 test.csv
-rw-rw-rw- 1 simon simon 2.0G Sep 29 02:34 test.tar.gz
-rw-rw-rw- 1 simon simon  17K Sep 29 02:34 training.csv
-rw-rw-rw- 1 simon simon 4.1G Sep 29 02:35 training1.tar.gz
-rw-rw-rw- 1 simon simon 1.7G Sep 29 02:35 training2

[Load] training.csv / test.csv / randomPredictions.csv heads
training.csv shape: (297, 2)
   Id                                           Sequence
0   1  2 14 20 6 7 3 1 13 18 5 12 16 15 4 9 10 8 17 1...
1   3  12 3 18 14 16 20 5 2 4 1 10 6 9 19 15 17 11 13...
2   4  13 1 8 18 7 17 16 9 5 10 11 4 20 3 19 2 14 6 1...
test.csv shape: (95, 1)
    Id
0  300
1  301
2  302
randomPredictions.csv shape: (95, 2)
    Id                                           Sequence
0  300  13 14 2 9 16 7 20 5 8 6 10 4 3 12 18 1 15 17 1...
1  301  4 3 11 16 20 6 7 15 10 18 17 9 8 12 5 19 1 13 ...
2  302  13 1 16 11 8 12 6 15 2 4 10 17 9 7 20 5 18 19 ...
[Archives] List first few members of training/validation/test archives


...

training3.tar.gz: 100 files. First 10:
 - ./Sample00200.zip
 - ./Sample00201.zip
 - ./Sample00202.zip
 - ./Sample00203.zip
 - ./Sample00204.zip
 - ./Sample00205.zip
 - ./Sample00206.zip
 - ./Sample00207.zip
 - ./Sample00208.zip
 - ./Sample00209.zip


validation1.tar.gz: 99 files. First 10:
 - Sample00410.zip
 - Sample00411.zip
 - Sample00412.zip
 - Sample00413.zip
 - Sample00414.zip
 - Sample00415.zip
 - Sample00416.zip
 - Sample00417.zip
 - Sample00418.zip
 - Sample00420.zip


validation2.tar.gz: 104 files. First 10:
 - Sample00510.zip
 - Sample00516.zip
 - Sample00517.zip
 - Sample00518.zip
 - Sample00519.zip
 - Sample00520.zip
 - Sample00521.zip
 - Sample00522.zip
 - Sample00523.zip
 - Sample00524.zip


validation3.tar.gz: 84 files. First 10:
 - Sample00621.zip
 - Sample00622.zip
 - Sample00623.zip
 - Sample00624.zip
 - Sample00625.zip
 - Sample00626.zip
 - Sample00627.zip
 - Sample00628.zip
 - Sample00629.zip
 - Sample00630.zip


test.tar.gz: 95 files. First 10:
 - ./Sample00300.zip
 - ./Sample00301.zip
 - ./Sample00302.zip
 - ./Sample00303.zip
 - ./Sample00304.zip
 - ./Sample00305.zip
 - ./Sample00306.zip
 - ./Sample00307.zip
 - ./Sample00308.zip
 - ./Sample00309.zip
[Done sanity checks] Elapsed: 25.3s


In [3]:
# 1) Surgical tar->zip->mat probe for a single sample
import io, zipfile
import numpy as np

def ensure_scipy():
    try:
        import scipy.io as sio  # noqa
        return
    except Exception:
        print("[Setup] Installing scipy...")
        run(['bash','-lc', f"{sys.executable} -m pip install -q scipy"])
ensure_scipy()
import scipy.io as sio

def probe_one_sample(tar_path):
    print(f"[Probe] {tar_path}")
    with tarfile.open(tar_path, 'r:gz') as tf:
        # pick the first Sample*.zip member
        zmem = next(m for m in tf.getmembers() if m.name.lower().endswith('.zip'))
        print("  zip member:", zmem.name)
        zbytes = tf.extractfile(zmem).read()
        with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
            names = zf.namelist()
            print("  zip contains:", len(names), "files. First 10:")
            for x in names[:10]:
                print("   -", x)
            # find *_data.mat
            mat_name = next(n for n in names if n.lower().endswith('_data.mat'))
            print("  data mat:", mat_name)
            with zf.open(mat_name) as f:
                d = sio.loadmat(f, simplify_cells=True)
    return d

def summarize_dict(d, prefix=""):
    for k, v in d.items():
        if k.startswith('__'):
            continue
        try:
            if isinstance(v, dict):
                print(f"{prefix}{k}: dict({len(v)})")
                summarize_dict(v, prefix + "  ")
            elif isinstance(v, (list, tuple)):
                lens = len(v)
                print(f"{prefix}{k}: {type(v).__name__} len={lens}")
                if lens and hasattr(v[0], 'shape'):
                    print(f"{prefix}  [0] shape={v[0].shape}")
            elif hasattr(v, 'shape'):
                print(f"{prefix}{k}: array shape={v.shape} dtype={getattr(v,'dtype','')} ")
            else:
                t = type(v).__name__
                vs = str(v)
                if len(vs) > 80: vs = vs[:80] + '...'
                print(f"{prefix}{k}: {t} = {vs}")
        except Exception as e:
            print(f"{prefix}{k}: error summarizing -> {e}")

def find_keys_recursive(d, kw_list):
    hits = []
    def rec(x, path):
        if isinstance(x, dict):
            for k, v in x.items():
                low = k.lower()
                if any(kw in low for kw in kw_list):
                    hits.append(("/".join(path+[k]), type(v).__name__))
                rec(v, path+[k])
        elif isinstance(x, (list, tuple)):
            for i, v in enumerate(x):
                rec(v, path+[f"[{i}]"])
    rec(d, [])
    return hits

sample_dict = probe_one_sample('training1.tar.gz')
print("[Summary] Top-level keys:", [k for k in sample_dict.keys() if not k.startswith('__')])
summarize_dict(sample_dict)

label_hits = find_keys_recursive(sample_dict, ["label","annot","segment","boundary","start","end"])
print("[Search] Potential label/segment keys (train/val only):")
for h in label_hits[:20]:
    print(" -", h)

skeleton_hits = find_keys_recursive(sample_dict, ["skeleton","joint","position","world","orientation","user","subject","fps","frame"])
print("[Search] Potential skeleton/meta keys:")
for h in skeleton_hits[:20]:
    print(" -", h)

print("[Note] Confirm exact fields for: skeleton 3D positions (T x J x 3), labels as (cls,start,end), fps, and user/subject.")

[Probe] training1.tar.gz


  zip member: ./Sample00001.zip
  zip contains: 5 files. First 10:
   - Sample00001_color.mp4
   - Sample00001_depth.mp4
   - Sample00001_user.mp4
   - Sample00001_data.mat
   - Sample00001_audio.wav
  data mat: Sample00001_data.mat
[Summary] Top-level keys: ['Video']
Video: dict(5)
  NumFrames: int = 1254
  FrameRate: int = 20
  Frames: list len=1254
  MaxDepth: int = 3293
  Labels: list len=20
[Search] Potential label/segment keys (train/val only):
 - ('Video/Labels', 'list')
 - ('Video/Labels/[0]/End', 'int')
 - ('Video/Labels/[1]/End', 'int')
 - ('Video/Labels/[2]/End', 'int')
 - ('Video/Labels/[3]/End', 'int')
 - ('Video/Labels/[4]/End', 'int')
 - ('Video/Labels/[5]/End', 'int')
 - ('Video/Labels/[6]/End', 'int')
 - ('Video/Labels/[7]/End', 'int')
 - ('Video/Labels/[8]/End', 'int')
 - ('Video/Labels/[9]/End', 'int')
 - ('Video/Labels/[10]/End', 'int')
 - ('Video/Labels/[11]/End', 'int')
 - ('Video/Labels/[12]/End', 'int')
 - ('Video/Labels/[13]/End', 'int')
 - ('Video/Labels/[14]/

In [4]:
# 2) Inspect structures within one sample: labels and skeleton layout
vid = sample_dict.get('Video', {})
print('[Video] keys:', list(vid.keys()))
print('[Video] NumFrames:', vid.get('NumFrames'), 'FrameRate:', vid.get('FrameRate'))

# Inspect labels content
labels = vid.get('Labels', [])
print('[Labels] count:', len(labels))
if labels:
    first = labels[0]
    print('[Labels][0] type:', type(first))
    if isinstance(first, dict):
        print('[Labels][0] keys:', list(first.keys()))
        print('[Labels][0] sample:', {k:first[k] for k in list(first.keys())[:5]})
    else:
        print('[Labels][0]:', first)

# Inspect first few frames skeleton arrays
frames = vid.get('Frames', [])
print('[Frames] count:', len(frames))
def frame_info(i):
    fr = frames[i]
    sk = fr.get('Skeleton') if isinstance(fr, dict) else None
    if not isinstance(sk, dict):
        return f'Frame {i}: no skeleton dict'
    wp = sk.get('WorldPosition', None)
    wr = sk.get('WorldRotation', None)
    jt = sk.get('JointType', None)
    px = sk.get('PixelPosition', None)
    return f'Frame {i}: WP={getattr(wp,"shape",None)} WR={getattr(wr,"shape",None)} JT={getattr(jt,"shape",None)} PX={getattr(px,"shape",None)}'

for i in [0,1,2,10,50]:
    if i < len(frames):
        print(frame_info(i))

# Try to infer joint count and names/ids from a frame
if frames:
    sk0 = frames[0].get('Skeleton', {}) if isinstance(frames[0], dict) else {}
    jt0 = sk0.get('JointType', None)
    if jt0 is not None:
        print('[JointType] dtype:', getattr(jt0,'dtype',None), 'shape:', getattr(jt0,'shape',None))
        try:
            print('[JointType] sample values:', jt0[:10])
        except Exception as e:
            print('[JointType] print error:', e)

# Check if hand state or user/subject metadata exists
meta_hits = find_keys_recursive(sample_dict, ['hand','state','user','subject','performer'])
print('[Search] Potential hand/user fields:')
for h in meta_hits[:20]:
    print(' -', h)

[Video] keys: ['NumFrames', 'FrameRate', 'Frames', 'MaxDepth', 'Labels']
[Video] NumFrames: 1254 FrameRate: 20
[Labels] count: 20
[Labels][0] type: <class 'dict'>
[Labels][0] keys: ['Name', 'Begin', 'End']
[Labels][0] sample: {'Name': 'vieniqui', 'Begin': 1, 'End': 79}
[Frames] count: 1254
Frame 0: WP=(20, 3) WR=(20, 4) JT=(20,) PX=(20, 2)
Frame 1: WP=(20, 3) WR=(20, 4) JT=(20,) PX=(20, 2)
Frame 2: WP=(20, 3) WR=(20, 4) JT=(20,) PX=(20, 2)
Frame 10: WP=(20, 3) WR=(20, 4) JT=(20,) PX=(20, 2)
Frame 50: WP=(20, 3) WR=(20, 4) JT=(20,) PX=(20, 2)
[JointType] dtype: object shape: (20,)
[JointType] sample values: [array([], dtype=float64) array([], dtype=float64)
 array([], dtype=float64) array([], dtype=float64)
 array([], dtype=float64) array([], dtype=float64)
 array([], dtype=float64) array([], dtype=float64)
 array([], dtype=float64) array([], dtype=float64)]
[Search] Potential hand/user fields:


In [6]:
# 3) Build name<->class-id mapping quickly (stop at 20) and sanity-load one sample
import re, io, zipfile, tarfile, numpy as np, pandas as pd, time

train_df = pd.read_csv('training.csv')
train_ids_set = set(train_df['Id'].tolist())

def id_to_tar_and_zipname(sample_id:int):
    if sample_id < 100:
        tar = 'training1.tar.gz'
    elif 100 <= sample_id < 200:
        tar = 'training2.tar.gz'
    else:
        tar = 'training3.tar.gz'
    return tar, f'./Sample{sample_id:05d}.zip'

def list_ids_in_tar(tar_path):
    ids = []
    with tarfile.open(tar_path, 'r:gz') as tf:
        for m in tf.getmembers():
            if m.name.lower().endswith('.zip'):
                m2 = re.findall(r'(\d{5})', m.name)
                if m2:
                    sid = int(m2[0])
                    ids.append(sid)
    return ids

def load_labels_names_for_id(sample_id:int):
    tar, zipname = id_to_tar_and_zipname(sample_id)
    with tarfile.open(tar, 'r:gz') as tf:
        mem = next((m for m in tf.getmembers() if m.name.endswith(zipname)), None)
        if mem is None:
            raise FileNotFoundError(f'Zip {zipname} not found in {tar}')
        zbytes = tf.extractfile(mem).read()
        with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
            mat_name = next(n for n in zf.namelist() if n.lower().endswith('_data.mat'))
            with zf.open(mat_name) as f:
                d = sio.loadmat(f, simplify_cells=True)
    labels = d['Video']['Labels']
    names = [lab['Name'] for lab in labels]
    return names

def parse_sequence_numbers(seq_str:str):
    return [int(x) for x in re.findall(r'\d+', str(seq_str))]

def build_mapping_until_20(candidate_ids):
    name_to_id, id_to_name, conflicts = {}, {}, []
    seen_class_ids = set()
    t0 = time.time()
    for i, sid in enumerate(candidate_ids):
        try:
            print(f'[Map] {i+1}/{len(candidate_ids)}: Id={sid}', flush=True)
            names = load_labels_names_for_id(sid)
            seq_nums = parse_sequence_numbers(train_df.loc[train_df.Id==sid, 'Sequence'].values[0])
            if len(seq_nums) != len(names):
                print(f'[Warn] Mismatch counts for {sid}: nums={len(seq_nums)} names={len(names)}')
            m = min(len(seq_nums), len(names))
            for k in range(m):
                n = names[k]
                cid = seq_nums[k]
                if n in name_to_id and name_to_id[n] != cid:
                    conflicts.append((n, name_to_id[n], cid, sid))
                name_to_id[n] = cid
                id_to_name[cid] = n
                seen_class_ids.add(cid)
            if len(seen_class_ids) >= 20 or len(name_to_id) >= 20:
                print(f'[Map] Reached {len(seen_class_ids)} unique class ids; stopping.', flush=True)
                break
        except Exception as e:
            print(f'[Warn] Failed Id={sid}: {e}')
    print(f'[Map] Done in {time.time()-t0:.1f}s. Unique classes: {len(seen_class_ids)} names: {len(name_to_id)} conflicts: {len(conflicts)}')
    return name_to_id, id_to_name, conflicts

# Build a small deterministic candidate id list from each training tar (first ~12 per tar that exist in training.csv)
cand = []
for tar_path in ['training1.tar.gz','training2.tar.gz','training3.tar.gz']:
    ids = [i for i in list_ids_in_tar(tar_path) if i in train_ids_set]
    ids = sorted(ids)[:12]
    cand.extend(ids)
print('[Cand] ids:', cand[:20], '... total', len(cand))

name_to_id, id_to_name, conflicts = build_mapping_until_20(cand)
print('[Mapping] size:', len(name_to_id), 'unique class ids:', len(set(name_to_id.values())))
print('[Mapping] sample pairs:', sorted([(v,k) for k,v in name_to_id.items()])[:10], '...')

# Sanity: load one sample quickly to verify frame targets expansion
def load_skeleton_and_frame_targets(sample_id:int, name_to_id_map:dict):
    tar, zipname = id_to_tar_and_zipname(sample_id)
    with tarfile.open(tar, 'r:gz') as tf:
        mem = next(m for m in tf.getmembers() if m.name.endswith(zipname))
        zbytes = tf.extractfile(mem).read()
        with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
            mat_name = next(n for n in zf.namelist() if n.lower().endswith('_data.mat'))
            with zf.open(mat_name) as f:
                d = sio.loadmat(f, simplify_cells=True)
    vid = d['Video']
    T = int(vid['NumFrames'])
    frames = vid['Frames']
    J = frames[0]['Skeleton']['WorldPosition'].shape[0]
    skel = np.zeros((T, J, 3), dtype=np.float32)
    for t in range(T):
        wp = frames[t]['Skeleton']['WorldPosition']
        if wp is None:
            if t>0: skel[t] = skel[t-1]
            continue
        skel[t] = np.asarray(wp, dtype=np.float32)
    y = np.zeros(T, dtype=np.int16)
    for lab in vid['Labels']:
        cls = name_to_id_map.get(lab['Name'])
        if cls is None: continue
        b = max(0, int(lab['Begin'])-1)
        e = min(T, int(lab['End']))
        y[b:e] = cls
    return skel, y, vid

test_id = cand[0] if len(cand)>0 else int(train_df.Id.iloc[0])
test_skel, test_y, test_vid = load_skeleton_and_frame_targets(test_id, name_to_id)
print('[Loader] sample id:', test_id, 'shapes:', test_skel.shape, test_y.shape, 'unique y:', np.unique(test_y))

[Cand] ids: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 101, 102, 103, 104, 105, 106, 107, 108] ... total 36
[Map] 1/36: Id=1


[Map] Reached 20 unique class ids; stopping.


[Map] Done in 5.8s. Unique classes: 20 names: 20 conflicts: 0
[Mapping] size: 20 unique class ids: 20
[Mapping] sample pairs: [(1, 'vattene'), (2, 'vieniqui'), (3, 'perfetto'), (4, 'furbo'), (5, 'cheduepalle'), (6, 'chevuoi'), (7, 'daccordo'), (8, 'seipazzo'), (9, 'combinato'), (10, 'freganiente')] ...


[Loader] sample id: 1 shapes: (1254, 20, 3) (1254,) unique y: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


In [7]:
# 4) Feature engineering, decoding, and smoke dataset builder
import numpy as np, pandas as pd, tarfile, zipfile, io, time, os, math, subprocess, sys

def pip_install(pkg):
    print(f"[Setup] Installing {pkg}...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', pkg], check=True)

try:
    import xgboost as xgb  # noqa
except Exception:
    pip_install('xgboost==2.1.1')
    import xgboost as xgb

# --- FE: skeleton -> per-frame features ---
def robust_scale_center(coords):
    # coords: (J,3)
    c = np.nanmean(coords, axis=0)  # center by mean joint
    centered = coords - c
    # scale by robust skeleton size: median pairwise distance or std
    flat = centered.reshape(-1)
    scale = np.nanstd(flat) + 1e-6
    centered /= scale
    return centered, scale

def features_from_skeleton(skel):
    # skel: (T,J,3) float32
    T, J, _ = skel.shape
    feats = np.zeros((T, J*3*2), dtype=np.float32)  # centered coords (J*3) + velocities (J*3)
    prev = None
    for t in range(T):
        coords = skel[t]
        centered, _ = robust_scale_center(coords)
        if prev is None:
            vel = np.zeros_like(centered)
        else:
            vel = centered - prev
        prev = centered
        feats[t, :J*3] = centered.reshape(-1)
        feats[t, J*3:] = vel.reshape(-1)
    return feats  # shape (T, 2*J*3)

# --- Decoding utilities ---
def median_filter(arr, k=7):
    if k <= 1: return arr
    k = int(k) | 1
    pad = k//2
    out = np.copy(arr)
    for i in range(len(arr)):
        s = max(0, i-pad); e = min(len(arr), i+pad+1)
        out[i] = np.bincount(arr[s:e]).argmax()
    return out

def collapse_runs(labels):
    seq = []
    prev = None
    for x in labels:
        if x != prev:
            seq.append(x)
            prev = x
    return seq

def decode_sequence(probs, window=7, merge_gap=4, min_len=6, mean_thr=0.45, max_thr=0.6):
    # probs: (T, C) softmax, C=21 with 0=background
    labels = probs.argmax(1).astype(int)
    labels = median_filter(labels, k=window)
    T = len(labels)
    # build segments
    segs = []  # (cls, b, e, mean_p, max_p)
    b = 0
    for i in range(1, T+1):
        if i==T or labels[i] != labels[b]:
            cls = labels[b]
            p = probs[b:i, cls] if cls < probs.shape[1] else np.zeros(i-b)
            mean_p = float(p.mean()) if (i-b)>0 else 0.0
            max_p = float(p.max()) if (i-b)>0 else 0.0
            segs.append([cls, b, i, mean_p, max_p])
            b = i
    # merge small gaps between same class
    merged = []
    i = 0
    while i < len(segs):
        cur = segs[i]
        j = i + 1
        while j < len(segs):
            if segs[j][0] == cur[0] and segs[j-1][0]==0 and (segs[j][1]-cur[2]) <= merge_gap:
                # extend cur to segs[j]
                cur[2] = segs[j][2]
                cur[3] = float(np.mean(probs[cur[1]:cur[2], cur[0]]))
                cur[4] = float(np.max(probs[cur[1]:cur[2], cur[0]]))
                j += 1
            else:
                break
        merged.append(cur)
        i = j
    # filter by min_len and hysteresis thresholds
    out = []
    for cls, sb, se, mp, xp in merged:
        if cls == 0: continue
        if (se - sb) < min_len: continue
        if not (mp >= mean_thr and xp >= max_thr): continue
        out.append(cls)
    # collapse duplicates
    final_seq = []
    for c in out:
        if not final_seq or final_seq[-1] != c:
            final_seq.append(c)
    return final_seq

# --- Levenshtein distance ---
def levenshtein(a, b):
    n, m = len(a), len(b)
    if n == 0: return m
    if m == 0: return n
    dp = list(range(m+1))
    for i in range(1, n+1):
        prev = dp[0]
        dp[0] = i
        for j in range(1, m+1):
            temp = dp[j]
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[j] = min(dp[j] + 1, dp[j-1] + 1, prev + cost)
            prev = temp
    return dp[m]

# --- Dataset builder (smoke) ---
def build_frame_dataset(sample_ids, name_to_id_map):
    X_list, y_list, gid = [], [], []
    t0 = time.time()
    for idx, sid in enumerate(sample_ids):
        skel, y, vid = load_skeleton_and_frame_targets(sid, name_to_id_map)
        X = features_from_skeleton(skel)
        X_list.append(X)
        y_list.append(y.astype(np.int32))
        gid.extend([sid]*len(y))
        if (idx+1) % 2 == 0:
            print(f"[Build] {idx+1}/{len(sample_ids)} ids, cum frames={sum(x.shape[0] for x in X_list)}")
    X = np.vstack(X_list)
    y = np.concatenate(y_list)
    gid = np.array(gid, dtype=np.int32)
    print(f"[Build] Done in {time.time()-t0:.1f}s. X={X.shape} y={y.shape} groups={len(np.unique(gid))}")
    return X, y, gid

print('[Ready] FE, decoding, and dataset builder prepared.')

[Ready] FE, decoding, and dataset builder prepared.


In [8]:
# 5) Smoke experiment: train per-frame XGBoost on a few samples and evaluate decoding via Levenshtein
import numpy as np, pandas as pd, xgboost as xgb, time

def true_sequence_for_id(sample_id:int) -> list:
    row = train_df.loc[train_df.Id==sample_id]
    if row.empty: return []
    return [int(x) for x in re.findall(r'\d+', str(row['Sequence'].values[0]))]

# pick small set from previously listed candidates
smoke_ids = cand[:12] if len(cand) >= 12 else sorted(list(train_ids_set))[:12]
train_ids_smoke = smoke_ids[:9]
val_ids_smoke = smoke_ids[9:12]
print('[Smoke] train ids:', train_ids_smoke)
print('[Smoke] val ids:', val_ids_smoke)

# Build datasets
X_tr, y_tr, gid_tr = build_frame_dataset(train_ids_smoke, name_to_id)
X_va, y_va, gid_va = build_frame_dataset(val_ids_smoke, name_to_id)

# Optional: downweight background class 0
def make_weights(y):
    w = np.ones_like(y, dtype=np.float32)
    w[y==0] = 0.4
    return w
w_tr = make_weights(y_tr)
w_va = make_weights(y_va)

dtr = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
dva = xgb.DMatrix(X_va, label=y_va, weight=w_va)

params = {
    'objective': 'multi:softprob',
    'num_class': 21,
    'eval_metric': 'mlogloss',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'max_depth': 8,
    'learning_rate': 0.08,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'min_child_weight': 2.0,
    'lambda': 1.0
}
print('[XGB] Training...')
t0 = time.time()
bst = xgb.train(params, dtr, num_boost_round=1200, evals=[(dtr,'train'),(dva,'valid')], early_stopping_rounds=200, verbose_eval=100)
print(f"[XGB] Done in {time.time()-t0:.1f}s. Best iters={bst.best_iteration}")

# Evaluate decoding on val ids
def predict_probs_for_id(sample_id:int):
    skel, y, vid = load_skeleton_and_frame_targets(sample_id, name_to_id)
    X = features_from_skeleton(skel)
    dm = xgb.DMatrix(X)
    P = bst.predict(dm, iteration_range=(0, bst.best_iteration+1))
    return P, y, vid

def eval_on_ids(ids, window=7, merge_gap=4, min_len=6, mean_thr=0.45, max_thr=0.6):
    scores = []
    for sid in ids:
        P, y_frames, vid = predict_probs_for_id(sid)
        pred_seq = decode_sequence(P, window=window, merge_gap=merge_gap, min_len=min_len, mean_thr=mean_thr, max_thr=max_thr)
        true_seq = true_sequence_for_id(sid)
        lev = levenshtein(pred_seq, true_seq)
        scores.append((sid, lev, len(true_seq)))
        print(f"[Eval] Id={sid} pred={pred_seq} true={true_seq} lev={lev}")
    mean_lev = float(np.mean([s[1] for s in scores])) if scores else None
    mean_norm = float(np.mean([s[1]/max(1,s[2]) for s in scores])) if scores else None
    print(f"[Eval] Val mean Levenshtein={mean_lev}, normalized={mean_norm}")
    return scores, mean_lev, mean_norm

scores, mean_lev, mean_norm = eval_on_ids(val_ids_smoke)
print('[Smoke] Completed.')

[Smoke] train ids: [1, 3, 4, 5, 6, 7, 8, 9, 10]
[Smoke] val ids: [11, 12, 13]


[Build] 2/9 ids, cum frames=2371


[Build] 4/9 ids, cum frames=5041


[Build] 6/9 ids, cum frames=7367


[Build] 8/9 ids, cum frames=9778


...

[262]	train-mlogloss:0.01320	valid-mlogloss:2.22562


[XGB] Done in 13.5s. Best iters=63


[Eval] Id=11 pred=[1, 5, 9, 10, 18, 13, 7, 11, 20, 14, 19, 15, 6, 16, 4, 3] true=[12, 1, 5, 9, 10, 18, 13, 7, 2, 20, 14, 19, 15, 6, 16, 4, 3, 11, 17, 8] lev=5


[Eval] Id=12 pred=[8, 13, 10, 15, 12, 15, 17, 2, 16, 9, 20, 2, 4, 7, 5, 1, 19, 8] true=[8, 13, 10, 11, 12, 15, 17, 2, 16, 9, 20, 14, 4, 7, 5, 1, 19, 6, 3, 18] lev=5


[Eval] Id=13 pred=[15, 11, 19, 13, 4, 8, 6, 2, 13, 6, 1, 19, 9, 18, 16, 18, 3] true=[15, 11, 19, 7, 5, 14, 4, 8, 6, 17, 2, 20, 13, 1, 10, 9, 12, 16, 18, 3] lev=8
[Eval] Val mean Levenshtein=6.0, normalized=0.3
[Smoke] Completed.


In [9]:
# 6) Improved FE v2 (hip-center + shoulder-width scaling + distances/speeds/angles) and smoke re-run
import numpy as np, time, xgboost as xgb, re

# Kinect-20 indices
HIP=0; SPINE=1; SHOULDER_CENTER=2; HEAD=3; L_SHOULDER=4; L_ELBOW=5; L_WRIST=6; L_HAND=7; R_SHOULDER=8; R_ELBOW=9; R_WRIST=10; R_HAND=11

def safe_norm(v):
    return float(np.linalg.norm(v)) if v is not None else 0.0

def features_from_skeleton_v2(skel):
    # skel: (T,J,3)
    T, J, _ = skel.shape
    base_dim = J*3*2
    extra_dim = 13  # 7 distances + 4 speeds + 2 elbow angles
    feats = np.zeros((T, base_dim + extra_dim), dtype=np.float32)
    prev_centered = None
    for t in range(T):
        coords = skel[t].astype(np.float32)
        # center by hip
        hip = coords[HIP]
        centered = coords - hip
        # scale by shoulder width
        shw = np.linalg.norm(centered[R_SHOULDER] - centered[L_SHOULDER])
        scale = max(shw, 1e-3)
        centered /= scale
        # velocities on normalized coords
        if prev_centered is None:
            vel = np.zeros_like(centered)
        else:
            vel = centered - prev_centered
        prev_centered = centered
        # base features
        feats[t, :J*3] = centered.reshape(-1)
        feats[t, J*3:J*3*2] = vel.reshape(-1)
        # extra distances
        d_hl_hd = np.linalg.norm(centered[L_HAND] - centered[HEAD])
        d_hr_hd = np.linalg.norm(centered[R_HAND] - centered[HEAD])
        d_hl_hip = np.linalg.norm(centered[L_HAND] - centered[HIP])
        d_hr_hip = np.linalg.norm(centered[R_HAND] - centered[HIP])
        d_hl_hr = np.linalg.norm(centered[L_HAND] - centered[R_HAND])
        d_hl_sc = np.linalg.norm(centered[L_HAND] - centered[SHOULDER_CENTER])
        d_hr_sc = np.linalg.norm(centered[R_HAND] - centered[SHOULDER_CENTER])
        # wrist/hand speeds (use vel)
        s_wl = np.linalg.norm(vel[L_WRIST])
        s_wr = np.linalg.norm(vel[R_WRIST])
        s_hl = np.linalg.norm(vel[L_HAND])
        s_hr = np.linalg.norm(vel[R_HAND])
        # elbow flexion cosines
        def elbow_cos(l_sh, l_el, l_wr):
            u = centered[l_sh] - centered[l_el]
            v = centered[l_wr] - centered[l_el]
            nu = np.linalg.norm(u); nv = np.linalg.norm(v)
            if nu < 1e-6 or nv < 1e-6: return 1.0
            return float(np.dot(u, v) / (nu*nv))
        cos_el = elbow_cos(L_SHOULDER, L_ELBOW, L_WRIST)
        cos_er = elbow_cos(R_SHOULDER, R_ELBOW, R_WRIST)
        extras = np.array([d_hl_hd, d_hr_hd, d_hl_hip, d_hr_hip, d_hl_hr, d_hl_sc, d_hr_sc, s_wl, s_wr, s_hl, s_hr, cos_el, cos_er], dtype=np.float32)
        feats[t, base_dim:] = extras
    return feats

def build_frame_dataset_with_fe(sample_ids, name_to_id_map, fe_fn):
    X_list, y_list, gid = [], [], []
    t0 = time.time()
    for idx, sid in enumerate(sample_ids):
        skel, y, vid = load_skeleton_and_frame_targets(sid, name_to_id_map)
        X = fe_fn(skel)
        X_list.append(X)
        y_list.append(y.astype(np.int32))
        gid.extend([sid]*len(y))
        if (idx+1) % 2 == 0:
            print(f"[BuildV2] {idx+1}/{len(sample_ids)} ids, cum frames={sum(x.shape[0] for x in X_list)}")
    X = np.vstack(X_list)
    y = np.concatenate(y_list)
    gid = np.array(gid, dtype=np.int32)
    print(f"[BuildV2] Done in {time.time()-t0:.1f}s. X={X.shape} y={y.shape} groups={len(np.unique(gid))}")
    return X, y, gid

# Re-run smoke with FE v2
smoke_ids = cand[:12] if len(cand) >= 12 else sorted(list(train_ids_set))[:12]
train_ids_smoke = smoke_ids[:9]
val_ids_smoke = smoke_ids[9:12]
print('[SmokeV2] train ids:', train_ids_smoke)
print('[SmokeV2] val ids:', val_ids_smoke)

X_tr, y_tr, gid_tr = build_frame_dataset_with_fe(train_ids_smoke, name_to_id, features_from_skeleton_v2)
X_va, y_va, gid_va = build_frame_dataset_with_fe(val_ids_smoke, name_to_id, features_from_skeleton_v2)

def make_weights(y):
    w = np.ones_like(y, dtype=np.float32); w[y==0] = 0.4; return w
dtr = xgb.DMatrix(X_tr, label=y_tr, weight=make_weights(y_tr))
dva = xgb.DMatrix(X_va, label=y_va, weight=make_weights(y_va))

params = {
    'objective': 'multi:softprob',
    'num_class': 21,
    'eval_metric': 'mlogloss',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'max_depth': 8,
    'learning_rate': 0.09,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'min_child_weight': 3.0,
    'lambda': 1.0
}
print('[XGB-V2] Training...')
bst2 = xgb.train(params, dtr, num_boost_round=1000, evals=[(dtr,'train'),(dva,'valid')], early_stopping_rounds=150, verbose_eval=100)
print('[XGB-V2] Best iters:', bst2.best_iteration)

def predict_probs_for_id_v2(sample_id:int):
    skel, y, vid = load_skeleton_and_frame_targets(sample_id, name_to_id)
    X = features_from_skeleton_v2(skel)
    dm = xgb.DMatrix(X)
    P = bst2.predict(dm, iteration_range=(0, bst2.best_iteration+1))
    return P, y, vid

def true_sequence_for_id(sample_id:int) -> list:
    row = train_df.loc[train_df.Id==sample_id]
    if row.empty: return []
    return [int(x) for x in re.findall(r'\d+', str(row['Sequence'].values[0]))]

scores2 = []
for sid in val_ids_smoke:
    P, y_frames, vid = predict_probs_for_id_v2(sid)
    pred_seq = decode_sequence(P, window=7, merge_gap=4, min_len=6, mean_thr=0.45, max_thr=0.6)
    true_seq = true_sequence_for_id(sid)
    lev = levenshtein(pred_seq, true_seq)
    scores2.append((sid, lev, len(true_seq)))
    print(f"[EvalV2] Id={sid} pred={pred_seq} true={true_seq} lev={lev}")
mean_lev2 = float(np.mean([s[1] for s in scores2])) if scores2 else None
mean_norm2 = float(np.mean([s[1]/max(1,s[2]) for s in scores2])) if scores2 else None
print(f"[EvalV2] Val mean Levenshtein={mean_lev2}, normalized={mean_norm2}")

[SmokeV2] train ids: [1, 3, 4, 5, 6, 7, 8, 9, 10]
[SmokeV2] val ids: [11, 12, 13]


[BuildV2] 2/9 ids, cum frames=2371


[BuildV2] 4/9 ids, cum frames=5041


[BuildV2] 6/9 ids, cum frames=7367


[BuildV2] 8/9 ids, cum frames=9778


...

[204]	train-mlogloss:0.01357	valid-mlogloss:2.42058


[XGB-V2] Best iters: 54


[EvalV2] Id=11 pred=[11, 5, 9, 18, 13, 7, 2, 11, 20, 15, 20, 4, 3] true=[12, 1, 5, 9, 10, 18, 13, 7, 2, 20, 14, 19, 15, 6, 16, 4, 3, 11, 17, 8] lev=11


[EvalV2] Id=12 pred=[8, 12, 15, 17, 9, 2, 4, 7, 5, 1, 11] true=[8, 13, 10, 11, 12, 15, 17, 2, 16, 9, 20, 14, 4, 7, 5, 1, 19, 6, 3, 18] lev=11


[EvalV2] Id=13 pred=[15, 11, 19, 6, 15, 4, 8, 20, 13, 9, 20, 16, 18, 3] true=[15, 11, 19, 7, 5, 14, 4, 8, 6, 17, 2, 20, 13, 1, 10, 9, 12, 16, 18, 3] lev=9
[EvalV2] Val mean Levenshtein=10.333333333333334, normalized=0.5166666666666667


In [10]:
# 7) Full train on training1-3 with FE v1, evaluate on validation1-3 (defaults), then prepare for grid tuning
import numpy as np, pandas as pd, time, tarfile, zipfile, io, xgboost as xgb, re

# Use FE v1 (performed better in smoke)

def list_all_ids_from_tars(tar_paths):
    ids = []
    for p in tar_paths:
        with tarfile.open(p, 'r:gz') as tf:
            for m in tf.getmembers():
                if m.name.lower().endswith('.zip'):
                    m2 = re.findall(r'(\d{5})', m.name)
                    if m2:
                        ids.append(int(m2[0]))
    return sorted(ids)

def load_vid_for_id(sample_id:int, tar_paths=None):
    # auto-detect tar based on id
    tar, zipname = id_to_tar_and_zipname(sample_id)
    with tarfile.open(tar, 'r:gz') as tf:
        mem = next(m for m in tf.getmembers() if m.name.endswith(zipname))
        zbytes = tf.extractfile(mem).read()
        with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
            mat_name = next(n for n in zf.namelist() if n.lower().endswith('_data.mat'))
            with zf.open(mat_name) as f:
                d = sio.loadmat(f, simplify_cells=True)
    return d['Video']

def true_token_sequence_from_video(vid, name_to_id_map):
    seq = []
    for lab in vid['Labels']:
        cid = name_to_id_map.get(lab['Name'])
        if cid is not None:
            seq.append(cid)
    return seq

print('[FullTrain] Building training frame dataset (this may take a few minutes)...')
full_train_ids = train_df['Id'].tolist()
X_tr, y_tr, gid_tr = build_frame_dataset(full_train_ids, name_to_id)
print('[FullTrain] Dataset:', X_tr.shape, y_tr.shape)

def make_weights(y):
    w = np.ones_like(y, dtype=np.float32); w[y==0] = 0.4; return w
dtr = xgb.DMatrix(X_tr, label=y_tr, weight=make_weights(y_tr))

params = {
    'objective': 'multi:softprob',
    'num_class': 21,
    'eval_metric': 'mlogloss',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'max_depth': 8,
    'learning_rate': 0.09,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'min_child_weight': 3.0,
    'lambda': 1.0,
    'max_bin': 256
}
print('[FullTrain] Training XGBoost (no holdout; rely on validation set for tuning)...')
t0 = time.time()
bst_full = xgb.train(params, dtr, num_boost_round=900)
print(f"[FullTrain] Done in {time.time()-t0:.1f}s. Rounds=900")

print('[ValEval] Listing validation ids...')
val_ids = list_all_ids_from_tars(['validation1.tar.gz','validation2.tar.gz','validation3.tar.gz'])
print('[ValEval] Count:', len(val_ids), 'first 5:', val_ids[:5])

def predict_and_decode_for_id(sample_id:int, model, window=7, merge_gap=4, min_len=6, mean_thr=0.45, max_thr=0.6):
    skel, y_frames, vid = load_skeleton_and_frame_targets(sample_id, name_to_id)
    X = features_from_skeleton(skel)
    P = model.predict(xgb.DMatrix(X))
    pred_seq = decode_sequence(P, window=window, merge_gap=merge_gap, min_len=min_len, mean_thr=mean_thr, max_thr=max_thr)
    return pred_seq, vid

print('[ValEval] Evaluating default decoder on validation set...')
levs = []
cnt = 0
t0 = time.time()
for sid in val_ids:
    pred_seq, vid = predict_and_decode_for_id(sid, bst_full)
    true_seq = true_token_sequence_from_video(vid, name_to_id)
    lev = levenshtein(pred_seq, true_seq)
    norm = lev / max(1, len(true_seq))
    levs.append(norm)
    cnt += 1
    if cnt % 10 == 0:
        print(f"  [ValEval] {cnt}/{len(val_ids)} mean_norm={np.mean(levs):.3f}")
print(f"[ValEval] Done {len(val_ids)} samples in {time.time()-t0:.1f}s. Mean normalized Levenshtein={np.mean(levs):.4f}")

print('[Next] If acceptable, grid search decoder params on validation set; then train on train+val and run test to create submission.csv.')

[FullTrain] Building training frame dataset (this may take a few minutes)...


[Build] 2/297 ids, cum frames=2371


KeyboardInterrupt: 

In [12]:
# 8) Caching pipeline: per-sample feature NPZ to ./cache with downsampling
import os, re, tarfile, zipfile, io, time, numpy as np

CACHE_DIR = './cache'
os.makedirs(CACHE_DIR, exist_ok=True)

def split_and_tar_for_id(sample_id:int, split_hint:str='train'):
    # Determine tar path based on id and split
    if split_hint == 'train':
        if sample_id < 100: return 'train', 'training1.tar.gz'
        elif sample_id < 200: return 'train', 'training2.tar.gz'
        else: return 'train', 'training3.tar.gz'
    elif split_hint == 'val':
        # Observed ranges from listing: 00410.., 00510.., 00621..
        if sample_id < 500: return 'val', 'validation1.tar.gz'
        elif sample_id < 621: return 'val', 'validation2.tar.gz'
        else: return 'val', 'validation3.tar.gz'
    elif split_hint == 'test':
        return 'test', 'test.tar.gz'
    else:
        raise ValueError('Unknown split_hint')

def get_zip_member(tf: tarfile.TarFile, sample_id:int):
    # Try fast path with exact names, then fallback to search
    candidates = [f'./Sample{sample_id:05d}.zip', f'Sample{sample_id:05d}.zip']
    for nm in candidates:
        try:
            return tf.getmember(nm)
        except KeyError:
            pass
    # fallback search once
    for m in tf.getmembers():
        if m.name.endswith(f'Sample{sample_id:05d}.zip'):
            return m
    raise FileNotFoundError(f'Sample zip for id {sample_id} not found in tar')

def load_video_from_split(sample_id:int, split_hint:str):
    _, tar_path = split_and_tar_for_id(sample_id, split_hint)
    with tarfile.open(tar_path, 'r:gz') as tf:
        mem = get_zip_member(tf, sample_id)
        zbytes = tf.extractfile(mem).read()
        with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
            mat_name = next(n for n in zf.namelist() if n.lower().endswith('_data.mat'))
            with zf.open(mat_name) as f:
                d = sio.loadmat(f, simplify_cells=True)
    return d['Video']

def stack_world_positions(vid, stride:int=1):
    T = int(vid['NumFrames']); frames = vid['Frames']
    J = frames[0]['Skeleton']['WorldPosition'].shape[0]
    idxs = list(range(0, T, stride))
    skel = np.zeros((len(idxs), J, 3), dtype=np.float32)
    ti = 0
    last = None
    for t in idxs:
        wp = frames[t]['Skeleton']['WorldPosition']
        arr = np.asarray(wp, dtype=np.float32) if wp is not None else last
        if arr is None:
            arr = np.zeros((J,3), dtype=np.float32)
        skel[ti] = arr; last = arr; ti += 1
    return skel, idxs

def true_token_sequence_from_video(vid, name_to_id_map):
    seq = []
    for lab in vid['Labels']:
        cid = name_to_id_map.get(lab['Name'])
        if cid is not None: seq.append(cid)
    return seq

def cache_sample(sample_id:int, split_hint:str, name_to_id_map:dict, stride:int=2, use_fe_v2:bool=False):
    out_path = os.path.join(CACHE_DIR, f'{split_hint}_{sample_id:05d}.npz')
    if os.path.exists(out_path):
        return out_path
    t0 = time.time()
    vid = load_video_from_split(sample_id, split_hint)
    skel, idxs = stack_world_positions(vid, stride=stride)
    if use_fe_v2:
        X = features_from_skeleton_v2(skel)
    else:
        X = features_from_skeleton(skel)
    # frame labels y at downsampled indices (0 background)
    T_full = int(vid['NumFrames'])
    y_full = np.zeros(T_full, dtype=np.int16)
    if split_hint in ('train','val'):
        for lab in vid['Labels']:
            cid = name_to_id_map.get(lab['Name'])
            if cid is None: continue
            b = max(0, int(lab['Begin'])-1); e = min(T_full, int(lab['End']))
            y_full[b:e] = cid
    y_ds = np.array([y_full[t] for t in idxs], dtype=np.int16)
    true_seq = true_token_sequence_from_video(vid, name_to_id_map) if split_hint in ('train','val') else []
    fps = int(vid.get('FrameRate', 20))
    np.savez_compressed(out_path, X=X, y=y_ds, seq=np.array(true_seq, dtype=np.int16), fps=fps, stride=stride)
    print(f"[Cache] {split_hint} id={sample_id} -> {out_path} X={X.shape} saved in {time.time()-t0:.1f}s")
    return out_path

def list_ids_in_tar_fast(tar_path):
    ids = []
    with tarfile.open(tar_path, 'r:gz') as tf:
        for m in tf.getmembers():
            if m.name.lower().endswith('.zip'):
                m2 = re.findall(r'(\d{5})', m.name)
                if m2: ids.append(int(m2[0]))
    return sorted(ids)

def cache_split_ids(split_hint:str, ids:list, stride:int=2, use_fe_v2:bool=False, limit:int=None):
    t0 = time.time()
    done = 0
    for i, sid in enumerate(ids[:limit] if limit else ids):
        cache_sample(sid, split_hint, name_to_id, stride=stride, use_fe_v2=use_fe_v2)
        done += 1
        if done % 10 == 0:
            print(f"[CacheProgress] {split_hint}: {done}/{len(ids if not limit else ids[:limit])} cached; elapsed {time.time()-t0:.1f}s")
    print(f"[CacheDone] {split_hint}: cached {done} samples in {time.time()-t0:.1f}s")

print('[Cache] Ready. Next: cache training (297 ids) and validation (~287 ids) with stride=2 to speed up training/eval.')

[Cache] Ready. Next: cache training (297 ids) and validation (~287 ids) with stride=2 to speed up training/eval.


In [13]:
# 9) Cache train and validation splits (stride=2) for fast training/eval
import time

print('[CacheRun] Collecting ids...')
train_ids_all = train_df['Id'].tolist()
val_ids_all = list_ids_in_tar_fast('validation1.tar.gz') + list_ids_in_tar_fast('validation2.tar.gz') + list_ids_in_tar_fast('validation3.tar.gz')
print('[CacheRun] Train ids:', len(train_ids_all), 'Val ids:', len(val_ids_all))

t0 = time.time()
print('[CacheRun] Caching TRAIN (stride=2)...')
cache_split_ids('train', train_ids_all, stride=2, use_fe_v2=False)
print('[CacheRun] Caching VAL (stride=2)...')
cache_split_ids('val', val_ids_all, stride=2, use_fe_v2=False)
print(f"[CacheRun] All caching done in {time.time()-t0:.1f}s. Cache dir: {CACHE_DIR}")

[CacheRun] Collecting ids...


[CacheRun] Train ids: 297 Val ids: 287
[CacheRun] Caching TRAIN (stride=2)...


[Cache] train id=1 -> ./cache/train_00001.npz X=(627, 120) saved in 5.8s


[Cache] train id=3 -> ./cache/train_00003.npz X=(559, 120) saved in 5.8s


[Cache] train id=4 -> ./cache/train_00004.npz X=(668, 120) saved in 6.0s


...

[Cache] train id=69 -> ./cache/train_00069.npz X=(590, 120) saved in 9.9s


[Cache] train id=70 -> ./cache/train_00070.npz X=(668, 120) saved in 10.0s


[Cache] train id=71 -> ./cache/train_00071.npz X=(830, 120) saved in 10.1s
[CacheProgress] train: 70/297 cached; elapsed 545.2s


[Cache] train id=72 -> ./cache/train_00072.npz X=(810, 120) saved in 10.2s


KeyboardInterrupt: 

In [14]:
# 10) Parallel caching (resume) for TRAIN and VAL to speed up IO
import os, time, concurrent.futures as cf

def remaining_ids(split_hint, ids):
    rem = []
    for sid in ids:
        out_path = os.path.join(CACHE_DIR, f'{split_hint}_{sid:05d}.npz')
        if not os.path.exists(out_path):
            rem.append(sid)
    return rem

def parallel_cache(split_hint, ids, max_workers=12, stride=2, use_fe_v2=False):
    todo = remaining_ids(split_hint, ids)
    print(f"[ParCache] {split_hint}: {len(todo)}/{len(ids)} remaining to cache.")
    if not todo:
        return
    t0 = time.time()
    done = 0
    def task(sid):
        try:
            return cache_sample(sid, split_hint, name_to_id, stride=stride, use_fe_v2=use_fe_v2)
        except Exception as e:
            print(f"[ParCache][ERR] {split_hint} id={sid}: {e}")
            return None
    with cf.ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(task, sid) for sid in todo]
        for i, fut in enumerate(cf.as_completed(futures), 1):
            _ = fut.result()
            done += 1
            if done % 20 == 0:
                print(f"[ParCache] {split_hint}: {done}/{len(todo)} done; elapsed {time.time()-t0:.1f}s")
    print(f"[ParCache] {split_hint}: completed {done} files in {time.time()-t0:.1f}s")

print('[ParCache] Preparing id lists...')
train_ids_all = train_df['Id'].tolist()
val_ids_all = list_ids_in_tar_fast('validation1.tar.gz') + list_ids_in_tar_fast('validation2.tar.gz') + list_ids_in_tar_fast('validation3.tar.gz')
print('[ParCache] Train ids:', len(train_ids_all), 'Val ids:', len(val_ids_all))

print('[ParCache] Starting parallel TRAIN cache (stride=2)...')
parallel_cache('train', train_ids_all, max_workers=12, stride=2, use_fe_v2=False)
print('[ParCache] Starting parallel VAL cache (stride=2)...')
parallel_cache('val', val_ids_all, max_workers=12, stride=2, use_fe_v2=False)
print('[ParCache] Done.')

[ParCache] Preparing id lists...


[ParCache] Train ids: 297 Val ids: 287
[ParCache] Starting parallel TRAIN cache (stride=2)...
[ParCache] train: 226/297 remaining to cache.


[Cache] train id=73 -> ./cache/train_00073.npz X=(791, 120) saved in 494.5s


[Cache] train id=74 -> ./cache/train_00074.npz X=(805, 120) saved in 499.4s


[Cache] train id=78 -> ./cache/train_00078.npz X=(817, 120) saved in 502.1s


...

[Cache] val id=706 -> ./cache/val_00706.npz X=(951, 120) saved in 342.3s


[Cache] val id=707 -> ./cache/val_00707.npz X=(926, 120) saved in 307.5s


[Cache] val id=708 -> ./cache/val_00708.npz X=(932, 120) saved in 256.6s


[Cache] val id=709 -> ./cache/val_00709.npz X=(937, 120) saved in 227.1s


[Cache] val id=710 -> ./cache/val_00710.npz X=(918, 120) saved in 216.3s
[ParCache] val: completed 287 files in 7155.0s
[ParCache] Done.


In [15]:
# 11) Train XGBoost on cached TRAIN and cache VAL per-frame probabilities; coarse decoder grid
import os, glob, time, numpy as np, pandas as pd
import xgboost as xgb

CACHE_DIR = './cache'

def load_cached_split(split_prefix):
    files = sorted(glob.glob(os.path.join(CACHE_DIR, f"{split_prefix}_*.npz")))
    Xs, ys, ids, idx_spans = [], [], [], []
    n_frames = 0
    t0 = time.time()
    for i, fp in enumerate(files, 1):
        d = np.load(fp, allow_pickle=False)
        X = d['X']; y = d['y']
        sid = int(os.path.basename(fp).split('_')[1].split('.')[0])
        Xs.append(X); ys.append(y.astype(np.int32))
        ids.append(sid)
        idx_spans.append((n_frames, n_frames + len(y), sid))
        n_frames += len(y)
        if i % 20 == 0:
            print(f"[LoadCache] {split_prefix}: {i}/{len(files)} files, cum frames={n_frames}", flush=True)
    X = np.vstack(Xs) if Xs else np.zeros((0,0), dtype=np.float32)
    y = np.concatenate(ys) if ys else np.zeros((0,), dtype=np.int32)
    print(f"[LoadCache] {split_prefix}: X={X.shape} y={y.shape} ids={len(ids)} elapsed={time.time()-t0:.1f}s", flush=True)
    return X, y, ids, idx_spans, files

def make_weights_with_boundary_erosion(y: np.ndarray, w0: float=0.38):
    w = np.ones_like(y, dtype=np.float32)
    w[y==0] = w0
    # boundary erosion: zero weight on boundary frames (label changes) to reduce noise
    if len(y) > 2:
        bmask = (y[1:-1] != y[:-2]) | (y[1:-1] != y[2:])
        w[1:-1][bmask] = w[1:-1][bmask] * 0.0
    return w

def train_xgb_on_cache():
    X_tr, y_tr, ids_tr, spans_tr, _ = load_cached_split('train')
    assert X_tr.shape[0] == y_tr.shape[0] and X_tr.shape[0] > 0, "Empty training cache"
    w_tr = make_weights_with_boundary_erosion(y_tr, w0=0.38)
    dtr = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0
    }
    print('[XGB] Training on cached TRAIN...', flush=True)
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=100)
    print(f"[XGB] Done in {time.time()-t0:.1f}s", flush=True)
    bst.save_model('xgb_train.model')
    return bst

def cache_val_probs(bst):
    # Load VAL cache and predict per-id, saving probs for decoder tuning
    _, _, _, _, val_files = load_cached_split('val')
    os.makedirs('./cache_probs', exist_ok=True)
    t0 = time.time()
    for i, fp in enumerate(val_files, 1):
        d = np.load(fp, allow_pickle=False)
        X = d['X']; seq = d['seq']; fps = int(d['fps']); stride = int(d['stride'])
        sid = int(os.path.basename(fp).split('_')[1].split('.')[0])
        dm = xgb.DMatrix(X)
        P = bst.predict(dm)
        outp = f"./cache_probs/valprobs_{sid:05d}.npz"
        np.savez_compressed(outp, probs=P.astype(np.float32), seq=seq.astype(np.int16), fps=fps, stride=stride, sid=sid)
        if i % 20 == 0:
            print(f"[ValProbs] {i}/{len(val_files)} saved; elapsed {time.time()-t0:.1f}s", flush=True)
    print(f"[ValProbs] Done {len(val_files)} ids in {time.time()-t0:.1f}s", flush=True)

def load_all_val_probs():
    files = sorted(glob.glob('./cache_probs/valprobs_*.npz'))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        items.append((int(d['sid']), d['probs'], d['seq']))
    return items

def grid_search_decoder():
    items = load_all_val_probs()
    assert items, 'No cached VAL probs found. Run cache_val_probs() first.'
    grids = {
        'window': [5,7,9],
        'merge_gap': [3,4,5],
        'min_len': [5,6,7,8],
        'mean_thr': [0.40,0.45,0.50],
        'max_thr': [0.55,0.60,0.65],
    }
    best = (1e9, None)
    tried = 0
    t0 = time.time()
    for w in grids['window']:
        for mg in grids['merge_gap']:
            for ml in grids['min_len']:
                for mthr in grids['mean_thr']:
                    for xthr in grids['max_thr']:
                        scores = []
                        for sid, P, seq in items:
                            pred = decode_sequence(P, window=w, merge_gap=mg, min_len=ml, mean_thr=mthr, max_thr=xthr)
                            lev = levenshtein(list(pred), list(seq.tolist()))
                            norm = lev / max(1, len(seq))
                            scores.append(norm)
                        mean_norm = float(np.mean(scores)) if scores else 1.0
                        tried += 1
                        if tried % 20 == 0:
                            print(f"[Grid] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
                        if mean_norm < best[0]:
                            best = (mean_norm, {'window':w,'merge_gap':mg,'min_len':ml,'mean_thr':mthr,'max_thr':xthr})
    print(f"[Grid] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}", flush=True)
    return best

print('[Stage] TRAIN on cached frames and prepare VAL probs. Execute this cell after caching finishes.', flush=True)
# Usage after caching completes:
# bst = train_xgb_on_cache()
# cache_val_probs(bst)
# best = grid_search_decoder()
# print('Best decoder params:', best)

# Note: final step will retrain on TRAIN+VAL caches with same params and run TEST decoding with frozen decoder settings.

[Stage] TRAIN on cached frames and prepare VAL probs. Execute this cell after caching finishes.


In [16]:
# 12) Execute training on cached data, cache VAL probs, and grid-search decoder
import time
t0 = time.time()
print('[RUN] Training XGBoost on cached TRAIN...')
bst = train_xgb_on_cache()
print('[RUN] Caching VAL per-frame probabilities...')
cache_val_probs(bst)
print('[RUN] Grid-searching decoder parameters...')
best = grid_search_decoder()
print('Best decoder params:', best)
print(f"[RUN] Total elapsed: {time.time()-t0:.1f}s")

[RUN] Training XGBoost on cached TRAIN...
[LoadCache] train: 20/297 files, cum frames=12055


[LoadCache] train: 40/297 files, cum frames=24945


[LoadCache] train: 60/297 files, cum frames=40473


[LoadCache] train: 80/297 files, cum frames=54817


[LoadCache] train: 100/297 files, cum frames=67157


...

[Grid] 280 combos, curr mean=15.6029 best=12.8809


[Grid] 300 combos, curr mean=18.1372 best=12.8809


[Grid] 320 combos, curr mean=14.2238 best=12.8809


[Grid] Done 324 combos in 116.6s. Best=(12.88086642599278, {'window': 5, 'merge_gap': 3, 'min_len': 8, 'mean_thr': 0.5, 'max_thr': 0.65})


Best decoder params: (12.88086642599278, {'window': 5, 'merge_gap': 3, 'min_len': 8, 'mean_thr': 0.5, 'max_thr': 0.65})
[RUN] Total elapsed: 366.2s


In [17]:
# 13) FE v1.5 (hip-center + shoulder-width scale) caching/training/grid pipeline
import os, io, re, time, glob, tarfile, zipfile, numpy as np, concurrent.futures as cf
import xgboost as xgb

# FE v1.5: normalize per-frame by hip-center and shoulder-width; then positions + velocities
HIP=0; L_SHOULDER=4; R_SHOULDER=8
def features_from_skeleton_v15(skel):
    T, J, _ = skel.shape
    feats = np.zeros((T, J*3*2), dtype=np.float32)
    prev = None
    for t in range(T):
        coords = skel[t].astype(np.float32)
        hip = coords[HIP]
        centered = coords - hip
        shoulder_w = np.linalg.norm(centered[R_SHOULDER] - centered[L_SHOULDER])
        scale = max(shoulder_w, 1e-4)
        centered /= scale
        if prev is None:
            vel = np.zeros_like(centered)
        else:
            vel = centered - prev
        prev = centered
        feats[t, :J*3] = centered.reshape(-1)
        feats[t, J*3:] = vel.reshape(-1)
    return feats

# Separate cache dirs to avoid mixing
CACHE_DIR_V15 = './cache_v15'
PROB_DIR_V15 = './cache_probs_v15'
os.makedirs(CACHE_DIR_V15, exist_ok=True)
os.makedirs(PROB_DIR_V15, exist_ok=True)

def split_and_tar_for_id(sample_id:int, split_hint:str='train'):
    if split_hint == 'train':
        if sample_id < 100: return 'train', 'training1.tar.gz'
        elif sample_id < 200: return 'train', 'training2.tar.gz'
        else: return 'train', 'training3.tar.gz'
    elif split_hint == 'val':
        if sample_id < 500: return 'val', 'validation1.tar.gz'
        elif sample_id < 621: return 'val', 'validation2.tar.gz'
        else: return 'val', 'validation3.tar.gz'
    elif split_hint == 'test':
        return 'test', 'test.tar.gz'
    else:
        raise ValueError('Unknown split_hint')

def get_zip_member(tf: tarfile.TarFile, sample_id:int):
    candidates = [f'./Sample{sample_id:05d}.zip', f'Sample{sample_id:05d}.zip']
    for nm in candidates:
        try:
            return tf.getmember(nm)
        except KeyError:
            pass
    for m in tf.getmembers():
        if m.name.endswith(f'Sample{sample_id:05d}.zip'):
            return m
    raise FileNotFoundError(f'Sample zip for id {sample_id} not found in tar')

def load_video_from_split(sample_id:int, split_hint:str):
    _, tar_path = split_and_tar_for_id(sample_id, split_hint)
    with tarfile.open(tar_path, 'r:gz') as tf:
        mem = get_zip_member(tf, sample_id)
        zbytes = tf.extractfile(mem).read()
        with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
            mat_name = next(n for n in zf.namelist() if n.lower().endswith('_data.mat'))
            with zf.open(mat_name) as f:
                d = sio.loadmat(f, simplify_cells=True)
    return d['Video']

def stack_world_positions(vid, stride:int=2):
    T = int(vid['NumFrames']); frames = vid['Frames']
    J = frames[0]['Skeleton']['WorldPosition'].shape[0]
    idxs = list(range(0, T, stride))
    skel = np.zeros((len(idxs), J, 3), dtype=np.float32)
    last = None
    for i, t in enumerate(idxs):
        wp = frames[t]['Skeleton']['WorldPosition']
        arr = np.asarray(wp, dtype=np.float32) if wp is not None else last
        if arr is None: arr = np.zeros((J,3), dtype=np.float32)
        skel[i] = arr; last = arr
    return skel, idxs

def true_token_sequence_from_video(vid, name_to_id_map):
    seq = []
    for lab in vid['Labels']:
        cid = name_to_id_map.get(lab['Name'])
        if cid is not None: seq.append(cid)
    return seq

def cache_sample_v15(sample_id:int, split_hint:str, stride:int=2):
    out_path = os.path.join(CACHE_DIR_V15, f'{split_hint}_{sample_id:05d}.npz')
    if os.path.exists(out_path):
        return out_path
    t0 = time.time()
    vid = load_video_from_split(sample_id, split_hint)
    skel, idxs = stack_world_positions(vid, stride=stride)
    X = features_from_skeleton_v15(skel)
    T_full = int(vid['NumFrames'])
    y_full = np.zeros(T_full, dtype=np.int16)
    if split_hint in ('train','val'):
        for lab in vid['Labels']:
            cid = name_to_id.get(lab['Name'])
            if cid is None: continue
            b = max(0, int(lab['Begin'])-1); e = min(T_full, int(lab['End']))
            y_full[b:e] = cid
    y_ds = np.array([y_full[t] for t in idxs], dtype=np.int16)
    true_seq = true_token_sequence_from_video(vid, name_to_id) if split_hint in ('train','val') else []
    fps = int(vid.get('FrameRate', 20))
    np.savez_compressed(out_path, X=X, y=y_ds, seq=np.array(true_seq, dtype=np.int16), fps=fps, stride=stride, sid=sample_id)
    print(f"[CacheV15] {split_hint} id={sample_id} -> {out_path} X={X.shape} saved in {time.time()-t0:.1f}s", flush=True)
    return out_path

def list_ids_in_tar_fast(tar_path):
    ids = []
    with tarfile.open(tar_path, 'r:gz') as tf:
        for m in tf.getmembers():
            if m.name.lower().endswith('.zip'):
                m2 = re.findall(r'(\d{5})', m.name)
                if m2: ids.append(int(m2[0]))
    return sorted(ids)

def remaining_ids_v15(split_hint, ids):
    rem = []
    for sid in ids:
        out_path = os.path.join(CACHE_DIR_V15, f'{split_hint}_{sid:05d}.npz')
        if not os.path.exists(out_path): rem.append(sid)
    return rem

def parallel_cache_v15(split_hint, ids, max_workers=12, stride=2):
    todo = remaining_ids_v15(split_hint, ids)
    print(f"[ParCacheV15] {split_hint}: {len(todo)}/{len(ids)} remaining.")
    if not todo: return
    t0 = time.time(); done = 0
    def task(sid):
        try:
            return cache_sample_v15(sid, split_hint, stride=stride)
        except Exception as e:
            print(f"[ParCacheV15][ERR] {split_hint} id={sid}: {e}")
            return None
    with cf.ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(task, sid) for sid in todo]
        for fut in cf.as_completed(futures):
            _ = fut.result(); done += 1
            if done % 20 == 0:
                print(f"[ParCacheV15] {split_hint}: {done}/{len(todo)} done; elapsed {time.time()-t0:.1f}s")
    print(f"[ParCacheV15] {split_hint}: completed {done} in {time.time()-t0:.1f}s")

def load_cached_split_v15(split_prefix):
    files = sorted(glob.glob(os.path.join(CACHE_DIR_V15, f"{split_prefix}_*.npz")))
    Xs, ys = [], []
    n_frames = 0; t0 = time.time()
    for i, fp in enumerate(files, 1):
        d = np.load(fp, allow_pickle=False)
        X = d['X']; y = d['y']
        Xs.append(X); ys.append(y.astype(np.int32))
        n_frames += len(y)
        if i % 20 == 0:
            print(f"[LoadCacheV15] {split_prefix}: {i}/{len(files)} files, cum frames={n_frames}", flush=True)
    X = np.vstack(Xs) if Xs else np.zeros((0,0), dtype=np.float32)
    y = np.concatenate(ys) if ys else np.zeros((0,), dtype=np.int32)
    print(f"[LoadCacheV15] {split_prefix}: X={X.shape} y={y.shape} files={len(files)} elapsed={time.time()-t0:.1f}s", flush=True)
    return X, y, files

def make_weights_with_boundary_erosion(y: np.ndarray, w0: float=0.38):
    w = np.ones_like(y, dtype=np.float32); w[y==0] = w0
    if len(y) > 2:
        bmask = (y[1:-1] != y[:-2]) | (y[1:-1] != y[2:])
        w[1:-1][bmask] = 0.0
    return w

def train_xgb_on_cache_v15():
    X_tr, y_tr, _ = load_cached_split_v15('train')
    assert X_tr.shape[0] == y_tr.shape[0] and X_tr.shape[0] > 0, 'Empty v15 train cache'
    w_tr = make_weights_with_boundary_erosion(y_tr, w0=0.38)
    dtr = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0
    }
    print('[XGB-V15] Training on cached TRAIN...', flush=True)
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=100)
    print(f"[XGB-V15] Done in {time.time()-t0:.1f}s", flush=True)
    bst.save_model('xgb_train_v15.model')
    return bst

def cache_val_probs_v15(bst):
    _, _, val_files = load_cached_split_v15('val')
    t0 = time.time()
    for i, fp in enumerate(val_files, 1):
        d = np.load(fp, allow_pickle=False)
        X = d['X']; seq = d['seq']; fps = int(d['fps']); stride = int(d['stride']); sid = int(d['sid'])
        P = bst.predict(xgb.DMatrix(X))
        outp = os.path.join(PROB_DIR_V15, f'valprobs_{sid:05d}.npz')
        np.savez_compressed(outp, probs=P.astype(np.float32), seq=seq.astype(np.int16), fps=fps, stride=stride, sid=sid)
        if i % 20 == 0:
            print(f"[ValProbsV15] {i}/{len(val_files)} saved; elapsed {time.time()-t0:.1f}s", flush=True)
    print(f"[ValProbsV15] Done {len(val_files)} ids in {time.time()-t0:.1f}s", flush=True)

def load_all_val_probs_v15():
    files = sorted(glob.glob(os.path.join(PROB_DIR_V15, 'valprobs_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        items.append((int(d['sid']), d['probs'], d['seq']))
    return items

def grid_search_decoder_v15():
    items = load_all_val_probs_v15()
    assert items, 'No cached VAL probs v15 found.'
    grids = {
        'window': [5,7,9],
        'merge_gap': [3,4,5,6],
        'min_len': [4,5,6,7,8,9,10],
        'mean_thr': [0.35,0.40,0.45,0.50,0.55],
        'max_thr': [0.55,0.60,0.65,0.70],
    }
    best = (1e9, None); tried = 0; t0 = time.time()
    for w in grids['window']:
        for mg in grids['merge_gap']:
            for ml in grids['min_len']:
                for mthr in grids['mean_thr']:
                    for xthr in grids['max_thr']:
                        scores = []
                        for sid, P, seq in items:
                            pred = decode_sequence(P, window=w, merge_gap=mg, min_len=ml, mean_thr=mthr, max_thr=xthr)
                            lev = levenshtein(list(pred), list(seq.tolist()))
                            norm = lev / max(1, len(seq))
                            scores.append(norm)
                        mean_norm = float(np.mean(scores)) if scores else 1.0
                        tried += 1
                        if tried % 20 == 0:
                            print(f"[GridV15] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
                        if mean_norm < best[0]:
                            best = (mean_norm, {'window':w,'merge_gap':mg,'min_len':ml,'mean_thr':mthr,'max_thr':xthr})
    print(f"[GridV15] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}", flush=True)
    return best

print('[StageV15] Ready: parallel cache TRAIN/VAL with FE v1.5, then train, cache VAL probs, and grid-search decoder. Use stride=2.', flush=True)
# Example run order:
# train_ids_all = train_df['Id'].tolist()
# val_ids_all = list_ids_in_tar_fast('validation1.tar.gz') + list_ids_in_tar_fast('validation2.tar.gz') + list_ids_in_tar_fast('validation3.tar.gz')
# parallel_cache_v15('train', train_ids_all, max_workers=12, stride=2)
# parallel_cache_v15('val', val_ids_all, max_workers=12, stride=2)
# bst_v15 = train_xgb_on_cache_v15()
# cache_val_probs_v15(bst_v15)
# best_v15 = grid_search_decoder_v15()
# print('Best decoder params (v15):', best_v15)

[StageV15] Ready: parallel cache TRAIN/VAL with FE v1.5, then train, cache VAL probs, and grid-search decoder. Use stride=2.


In [18]:
# 14) Run FE v1.5 pipeline: parallel cache TRAIN/VAL (stride=2), train, cache VAL probs, grid-search decoder
import time, numpy as np
import scipy.io as sio  # ensure available for v1.5 loaders

print('[RUN V1.5] Collecting ids...')
train_ids_all = train_df['Id'].tolist()
val_ids_all = list_ids_in_tar_fast('validation1.tar.gz') + list_ids_in_tar_fast('validation2.tar.gz') + list_ids_in_tar_fast('validation3.tar.gz')
print('[RUN V1.5] Train ids:', len(train_ids_all), 'Val ids:', len(val_ids_all))

t0 = time.time()
print('[RUN V1.5] Parallel caching TRAIN (stride=2)...')
parallel_cache_v15('train', train_ids_all, max_workers=12, stride=2)
print('[RUN V1.5] Parallel caching VAL (stride=2)...')
parallel_cache_v15('val', val_ids_all, max_workers=12, stride=2)
print(f"[RUN V1.5] Caching done in {time.time()-t0:.1f}s")

print('[RUN V1.5] Training XGBoost on v1.5 cached TRAIN...')
bst_v15 = train_xgb_on_cache_v15()

print('[RUN V1.5] Caching VAL per-frame probabilities (v1.5)...')
cache_val_probs_v15(bst_v15)

print('[RUN V1.5] Grid-searching decoder (v1.5)...')
best_v15 = grid_search_decoder_v15()
print('Best decoder params (v1.5):', best_v15)

[RUN V1.5] Collecting ids...


[RUN V1.5] Train ids: 297 Val ids: 287
[RUN V1.5] Parallel caching TRAIN (stride=2)...
[ParCacheV15] train: 297/297 remaining.


[CacheV15] train id=1 -> ./cache_v15/train_00001.npz X=(627, 120) saved in 275.0s


[CacheV15] train id=3 -> ./cache_v15/train_00003.npz X=(559, 120) saved in 275.6s


[CacheV15] train id=4 -> ./cache_v15/train_00004.npz X=(668, 120) saved in 283.6s


...

[GridV15] 1640 combos, curr mean=13.6318 best=11.2816


[GridV15] 1660 combos, curr mean=12.6137 best=11.2816


[GridV15] 1680 combos, curr mean=11.5487 best=11.2816


[GridV15] Done 1680 combos in 613.6s. Best=(11.28158844765343, {'window': 5, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.55, 'max_thr': 0.7})


Best decoder params (v1.5): (11.28158844765343, {'window': 5, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.55, 'max_thr': 0.7})


In [22]:
# 15) Train on TRAIN+VAL (v1.5), predict TEST, decode with rescue, and write submission.csv
import os, glob, time, json, numpy as np, pandas as pd
import xgboost as xgb

def load_frames_from_cache_v15(prefix:str):
    files = sorted(glob.glob(os.path.join(CACHE_DIR_V15, f"{prefix}_*.npz")))
    Xs, ys = [], []
    n = 0; t0 = time.time()
    for i, fp in enumerate(files, 1):
        d = np.load(fp, allow_pickle=False)
        X = d['X']; y = d['y']
        Xs.append(X); ys.append(y.astype(np.int32)); n += len(y)
        if i % 20 == 0:
            print(f"[LoadV15] {prefix}: {i}/{len(files)} files, cum frames={n}", flush=True)
    X = np.vstack(Xs) if Xs else np.zeros((0,0), dtype=np.float32)
    y = np.concatenate(ys) if ys else np.zeros((0,), dtype=np.int32)
    print(f"[LoadV15] {prefix}: X={X.shape} y={y.shape} files={len(files)}", flush=True)
    return X, y, files

def ensure_test_cache_v15():
    test_ids = test_df['Id'].tolist()
    todo = []
    for sid in test_ids:
        out_path = os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz')
        if not os.path.exists(out_path):
            todo.append(sid)
    if todo:
        print(f"[CacheV15][TEST] {len(todo)}/{len(test_ids)} missing; caching with stride=2...")
        parallel_cache_v15('test', test_ids, max_workers=12, stride=2)
    else:
        print('[CacheV15][TEST] All test cached.')

def make_weights_with_boundary_erosion(y: np.ndarray, w0: float=0.38):
    w = np.ones_like(y, dtype=np.float32); w[y==0] = w0
    if len(y) > 2:
        bmask = (y[1:-1] != y[:-2]) | (y[1:-1] != y[2:])
        w[1:-1][bmask] = 0.0
    return w

def train_seed_model_v15(X, y, seed:int):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    print(f"[XGB-V15][Seed {seed}] Training...")
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=200)
    print(f"[XGB-V15][Seed {seed}] Done in {time.time()-t0:.1f}s")
    return bst

def ensure_permutation_20(seq, mean_probs):
    seen = set()
    out = []
    for c in seq:
        if c==0: continue
        if c not in seen and 1 <= c <= 20:
            out.append(c); seen.add(c)
        if len(out) == 20: break
    if len(out) < 20:
        remaining = [c for c in range(1,21) if c not in seen]
        remaining.sort(key=lambda c: float(mean_probs[c]), reverse=True)
        out.extend(remaining)
        out = out[:20]
    elif len(out) > 20:
        out = out[:20]
    return out

# Use DP decoder if params contains DP keys; else use vanilla decode_sequence
def decode_with_rescue(P, params):
    mean_probs = P.mean(axis=0)
    if all(k in params for k in ('lambda','bg_bias','merge_gap','min_len','mean_thr','max_thr')):
        pred = decode_sequence_dp(P, switch_penalty=params['lambda'], bg_bias=params['bg_bias'],
                                 merge_gap=params['merge_gap'], min_len=params['min_len'],
                                 mean_thr=params['mean_thr'], max_thr=params['max_thr'])
    else:
        pred = decode_sequence(P, window=params['window'], merge_gap=params['merge_gap'],
                               min_len=params['min_len'], mean_thr=params['mean_thr'], max_thr=params['max_thr'])
    return ensure_permutation_20(pred, mean_probs)

def predict_test_and_write_submission_v15(models, decoder_params, out_csv='submission.csv'):
    test_ids = test_df['Id'].tolist()
    rows = []
    t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']
        Ps = []
        dm = xgb.DMatrix(X)
        for m in models:
            Ps.append(m.predict(dm))
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = decode_with_rescue(P, decoder_params)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv(out_csv, index=False)
    print(f"[SUBMISSION] Wrote {out_csv} with {len(rows)} rows.")
    return out_csv

def run_trainval_and_test_v15(best_decoder_params:dict, seeds=(2025, 1337)):
    print('[Train+Val V1.5] Loading TRAIN and VAL caches...')
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    X_va, y_va, _ = load_frames_from_cache_v15('val')
    X_all = np.vstack([X_tr, X_va]); y_all = np.concatenate([y_tr, y_va])
    print('[Train+Val V1.5] Frames:', X_all.shape, 'Labels:', y_all.shape)
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_all, y_all, seed=s))
    ensure_test_cache_v15()
    return predict_test_and_write_submission_v15(models, best_decoder_params, out_csv='submission.csv')

print('[StageV15][Final] Ready: run run_trainval_and_test_v15(best_params_dict) after grid search to produce submission.csv')

[StageV15][Final] Ready: run run_trainval_and_test_v15(best_params_dict) after grid search to produce submission.csv


In [19]:
# 16) DP/Viterbi decoder to stabilize per-frame labels; grid on VAL cached probs (v1.5)
import numpy as np, glob, os, time

def viterbi_labels(probs: np.ndarray, switch_penalty: float = 1.0, bg_bias: float = 0.0) -> np.ndarray:
    # probs: (T,C), returns best path labels in [0..C-1]
    T, C = probs.shape
    eps = 1e-9
    # emission costs
    cost = -np.log(np.clip(probs, eps, 1.0)).astype(np.float32)
    if bg_bias != 0.0:
        cost[:, 0] += float(bg_bias)  # discourage background
    dp = np.zeros((T, C), dtype=np.float32)
    bp = np.zeros((T, C), dtype=np.int16)
    dp[0] = cost[0]
    bp[0] = -1
    for t in range(1, T):
        prev = dp[t-1]
        min_prev = float(prev.min())
        # cost to stay vs switch-from-best-other
        stay = prev  # no penalty
        switch = min_prev + switch_penalty
        # vectorized: best_prev_cost = min(stay[c], switch) for each c
        best_prev = np.minimum(stay, switch)
        dp[t] = cost[t] + best_prev
        # backpointer:  if stay <= switch -> c, else argmin prev
        argmin_prev = int(prev.argmin())
        bp[t] = np.where(stay <= switch, np.arange(C, dtype=np.int16), np.int16(argmin_prev))
    # backtrack
    path = np.zeros(T, dtype=np.int16)
    path[T-1] = int(dp[T-1].argmin())
    for t in range(T-2, -1, -1):
        path[t] = bp[t+1, path[t+1]]
    return path

def decode_sequence_dp(probs, switch_penalty=1.0, bg_bias=0.0, merge_gap=4, min_len=6, mean_thr=0.45, max_thr=0.60):
    labels = viterbi_labels(probs, switch_penalty=switch_penalty, bg_bias=bg_bias)
    T = len(labels)
    # build segments with stats
    segs = []  # (cls, b, e, mean_p, max_p)
    b = 0
    for i in range(1, T+1):
        if i==T or labels[i] != labels[b]:
            cls = int(labels[b])
            p = probs[b:i, cls] if cls < probs.shape[1] else np.zeros(i-b)
            mean_p = float(p.mean()) if (i-b)>0 else 0.0
            max_p = float(p.max()) if (i-b)>0 else 0.0
            segs.append([cls, b, i, mean_p, max_p])
            b = i
    # merge small gaps between same class (gap must be background and short)
    merged = []
    i = 0
    while i < len(segs):
        cur = segs[i]
        j = i + 1
        while j < len(segs):
            if segs[j][0] == cur[0] and segs[j-1][0]==0 and (segs[j][1]-cur[2]) <= merge_gap:
                cur[2] = segs[j][2]
                cur[3] = float(np.mean(probs[cur[1]:cur[2], cur[0]]))
                cur[4] = float(np.max(probs[cur[1]:cur[2], cur[0]]))
                j += 1
            else:
                break
        merged.append(cur)
        i = j
    out = []
    for cls, sb, se, mp, xp in merged:
        if cls == 0: continue
        if (se - sb) < min_len: continue
        if not (mp >= mean_thr and xp >= max_thr): continue
        if not out or out[-1] != cls:
            out.append(int(cls))
    return out

def load_all_val_probs_v15():
    files = sorted(glob.glob(os.path.join(PROB_DIR_V15, 'valprobs_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        items.append((int(d['sid']), d['probs'], d['seq']))
    return items

def grid_search_decoder_dp_v15():
    items = load_all_val_probs_v15()
    assert items, 'No cached VAL probs v15 found.'
    grids = {
        'lambda': [0.8, 1.0, 1.2],
        'bg_bias': [0.2, 0.4],
        'merge_gap': [3, 5],
        'min_len': [6, 8, 10, 12],
        'mean_thr': [0.45, 0.50],
        'max_thr': [0.60, 0.65],
    }
    best = (1e9, None); tried = 0; t0 = time.time()
    for lam in grids['lambda']:
        for bb in grids['bg_bias']:
            for mg in grids['merge_gap']:
                for ml in grids['min_len']:
                    for mthr in grids['mean_thr']:
                        for xthr in grids['max_thr']:
                            scores = []
                            for sid, P, seq in items:
                                pred = decode_sequence_dp(P, switch_penalty=lam, bg_bias=bb, merge_gap=mg, min_len=ml, mean_thr=mthr, max_thr=xthr)
                                lev = levenshtein(list(pred), list(seq.tolist()))
                                norm = lev / max(1, len(seq))
                                scores.append(norm)
                            mean_norm = float(np.mean(scores)) if scores else 1.0
                            tried += 1
                            if tried % 20 == 0:
                                print(f"[GridV15-DP] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
                            if mean_norm < best[0]:
                                best = (mean_norm, {'lambda':lam,'bg_bias':bb,'merge_gap':mg,'min_len':ml,'mean_thr':mthr,'max_thr':xthr})
    print(f"[GridV15-DP] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}", flush=True)
    return best

print('[StageV15][DP] Ready: run best_dp = grid_search_decoder_dp_v15() to evaluate Viterbi-based decoder on VAL cached probs.')

[StageV15][DP] Ready: run best_dp = grid_search_decoder_dp_v15() to evaluate Viterbi-based decoder on VAL cached probs.


In [20]:
# 17) Run DP decoder grid on v1.5 VAL probs
best_dp = grid_search_decoder_dp_v15()
print('Best DP decoder params (v1.5):', best_dp)

[GridV15-DP] 20 combos, curr mean=18.9747 best=9.2094


[GridV15-DP] 40 combos, curr mean=15.0433 best=9.2094


[GridV15-DP] 60 combos, curr mean=11.8773 best=9.2094


[GridV15-DP] 80 combos, curr mean=9.4332 best=9.2094


[GridV15-DP] 100 combos, curr mean=18.8339 best=9.2094


...

[GridV15-DP] 140 combos, curr mean=12.1191 best=9.2094


[GridV15-DP] 160 combos, curr mean=9.5487 best=9.2094


[GridV15-DP] 180 combos, curr mean=18.7509 best=9.2094


[GridV15-DP] Done 192 combos in 355.5s. Best=(9.209386281588447, {'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65})


Best DP decoder params (v1.5): (9.209386281588447, {'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65})


In [23]:
# 18) Retrain on TRAIN+VAL with DP decoder params and produce submission
print('[FINAL RUN] Using DP decoder params from VAL grid...')
dp_params = best_dp[1]  # expects dict with keys: lambda,bg_bias,merge_gap,min_len,mean_thr,max_thr
print('DP params:', dp_params)
sub_path = run_trainval_and_test_v15(dp_params, seeds=(2025,1337))
print('Submission written to:', sub_path)

[FINAL RUN] Using DP decoder params from VAL grid...
DP params: {'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
[Train+Val V1.5] Loading TRAIN and VAL caches...
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST] 60/95 done; elapsed 32.3s


[TEST] 70/95 done; elapsed 37.6s


[TEST] 80/95 done; elapsed 42.8s


[TEST] 90/95 done; elapsed 47.9s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [24]:
# 19) Decoder debug: visualize one validation sample through stages (argmax, Viterbi, segments, filters)
import numpy as np, glob, os, re

def load_one_val_item(v15=True, sid=None):
    prob_dir = './cache_probs_v15' if v15 else './cache_probs'
    files = sorted(glob.glob(os.path.join(prob_dir, 'valprobs_*.npz')))
    assert files, 'No cached VAL probs found. Run cache_val_probs_v15() or cache_val_probs() first.'
    if sid is not None:
        target = os.path.join(prob_dir, f'valprobs_{int(sid):05d}.npz')
        files = [target]
    fp = files[0]
    d = np.load(fp, allow_pickle=False)
    return int(d['sid']) if 'sid' in d else int(re.findall(r'(\d{5})', os.path.basename(fp))[0]), d['probs'], d['seq']

def segments_from_labels(labels, probs):
    segs = []  # (cls, b, e, mean_p, max_p)
    T = len(labels); b = 0
    for i in range(1, T+1):
        if i==T or labels[i] != labels[b]:
            cls = int(labels[b])
            p = probs[b:i, cls] if cls < probs.shape[1] else np.zeros(i-b)
            mean_p = float(p.mean()) if (i-b)>0 else 0.0
            max_p = float(p.max()) if (i-b)>0 else 0.0
            segs.append([cls, b, i, mean_p, max_p])
            b = i
    return segs

def print_segs(segs, limit=30):
    rows = []
    for cls, sb, se, mp, xp in segs[:limit]:
        rows.append(f'(c={cls}, {sb}->{se}, len={se-sb}, mean={mp:.3f}, max={xp:.3f})')
    print('Segments[0:%d]:' % min(limit, len(segs)))
    print('\n'.join(rows))

def collapse_run_labels(labels):
    out = []
    prev = None
    for x in labels:
        if x != prev:
            out.append(int(x)); prev = x
    return out

def norm_lev(pred, true):
    lev = levenshtein(list(pred), list(true))
    return lev, lev / max(1, len(true))

# Pick one sample and show decoder internals
sid, P, true_seq = load_one_val_item(v15=True, sid=None)
print('[Debug] sid:', sid, 'P shape:', P.shape, 'true_seq len:', len(true_seq))

# Argmax path
argmax_labels = P.argmax(1).astype(int)
argmax_collapsed = collapse_run_labels(argmax_labels)
lev_a, nlev_a = norm_lev(argmax_collapsed, list(true_seq.tolist()))
print('[Argmax] collapsed len:', len(argmax_collapsed), 'lev:', lev_a, 'norm:', nlev_a)
segs_arg = segments_from_labels(argmax_labels, P)
print('[Argmax] total segments:', len(segs_arg))
print_segs(segs_arg, limit=20)

# Simple decoder output
pred_simple = decode_sequence(P, window=7, merge_gap=4, min_len=6, mean_thr=0.45, max_thr=0.60)
lev_s, nlev_s = norm_lev(pred_simple, list(true_seq.tolist()))
print('[SimpleDecoder] seq:', pred_simple)
print('[SimpleDecoder] len:', len(pred_simple), 'lev:', lev_s, 'norm:', nlev_s)

# Viterbi path
vit_labels = viterbi_labels(P, switch_penalty=0.8, bg_bias=0.2)
vit_collapsed = collapse_run_labels(vit_labels)
lev_vc, nlev_vc = norm_lev(vit_collapsed, list(true_seq.tolist()))
print('[Viterbi] collapsed len:', len(vit_collapsed), 'lev:', lev_vc, 'norm:', nlev_vc)
segs_vit = segments_from_labels(vit_labels, P)
print('[Viterbi] total segments:', len(segs_vit))
print_segs(segs_vit, limit=20)

# Viterbi + filters decoder output
pred_dp = decode_sequence_dp(P, switch_penalty=0.8, bg_bias=0.2, merge_gap=3, min_len=12, mean_thr=0.5, max_thr=0.65)
lev_dp, nlev_dp = norm_lev(pred_dp, list(true_seq.tolist()))
print('[Viterbi+Filters] seq:', pred_dp)
print('[Viterbi+Filters] len:', len(pred_dp), 'lev:', lev_dp, 'norm:', nlev_dp)

# Sanity: lengths and normalization check
print('[Sanity] true_seq:', list(true_seq.tolist()))
print('[Sanity] len(true_seq)=', len(true_seq), 'normalize by this length. OK.')

[Debug] sid: 410 P shape: (612, 21) true_seq len: 0
[Argmax] collapsed len: 150 lev: 150 norm: 150.0
[Argmax] total segments: 150
Segments[0:20]:
(c=17, 0->2, len=2, mean=0.073, max=0.073)
(c=0, 2->3, len=1, mean=0.360, max=0.360)
(c=13, 3->4, len=1, mean=0.181, max=0.181)
(c=0, 4->35, len=31, mean=0.477, max=0.657)
(c=5, 35->49, len=14, mean=0.866, max=0.995)
(c=0, 49->52, len=3, mean=0.425, max=0.436)
(c=5, 52->53, len=1, mean=0.384, max=0.384)
(c=0, 53->61, len=8, mean=0.389, max=0.435)
(c=5, 61->63, len=2, mean=0.654, max=0.761)
(c=0, 63->66, len=3, mean=0.606, max=0.735)
(c=5, 66->68, len=2, mean=0.486, max=0.616)
(c=0, 68->70, len=2, mean=0.339, max=0.348)
(c=11, 70->71, len=1, mean=0.199, max=0.199)
(c=5, 71->74, len=3, mean=0.833, max=0.900)
(c=0, 74->82, len=8, mean=0.685, max=0.900)
(c=5, 82->83, len=1, mean=0.652, max=0.652)
(c=0, 83->85, len=2, mean=0.629, max=0.722)
(c=8, 85->86, len=1, mean=0.350, max=0.350)
(c=18, 86->87, len=1, mean=0.479, max=0.479)
(c=8, 87->89, len=2

In [25]:
# 20) Build TRAIN-only 5-fold OOF per-id probabilities (v1.5), then grid-search decoder on OOF
import os, glob, time, numpy as np, pandas as pd
from sklearn.model_selection import KFold
import xgboost as xgb

OOF_DIR_V15 = './oof_probs_v15'
os.makedirs(OOF_DIR_V15, exist_ok=True)

def load_train_item_v15(sid:int):
    fp = os.path.join(CACHE_DIR_V15, f'train_{sid:05d}.npz')
    d = np.load(fp, allow_pickle=False)
    X = d['X']; y = d['y']
    # true 20-token sequence from training.csv (authoritative)
    row = train_df.loc[train_df.Id==sid]
    seq = []
    if not row.empty:
        seq = [int(x) for x in re.findall(r'\d+', str(row['Sequence'].values[0]))]
    return X, y.astype(np.int32), np.array(seq, dtype=np.int16)

def load_many_train_frames_v15(sids:list):
    Xs, ys = [], []
    n = 0; t0 = time.time()
    for i, sid in enumerate(sids, 1):
        X, y, _ = load_train_item_v15(sid)
        Xs.append(X); ys.append(y); n += len(y)
        if i % 20 == 0:
            print(f"[LoadTrainFold] {i}/{len(sids)} ids, cum frames={n}", flush=True)
    X = np.vstack(Xs); y = np.concatenate(ys)
    print(f"[LoadTrainFold] Loaded {len(sids)} ids: X={X.shape} y={y.shape}", flush=True)
    return X, y

def make_weights_with_boundary_erosion(y: np.ndarray, w0: float=0.38):
    w = np.ones_like(y, dtype=np.float32); w[y==0] = w0
    if len(y) > 2:
        bmask = (y[1:-1] != y[:-2]) | (y[1:-1] != y[2:])
        w[1:-1][bmask] = 0.0
    return w

def train_fold_model_v15(X, y, seed:int=42, num_rounds:int=900):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    bst = xgb.train(params, dtr, num_boost_round=num_rounds, verbose_eval=200)
    return bst

def build_oof_probs_v15(n_splits=5, seed=42, num_rounds=900):
    ids = train_df['Id'].tolist()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold = 0; t_all = time.time()
    for tr_idx, va_idx in kf.split(ids):
        fold += 1
        tr_ids = [ids[i] for i in tr_idx]
        va_ids = [ids[i] for i in va_idx]
        print(f"[OOF] Fold {fold}/{n_splits}: tr={len(tr_ids)} va={len(va_ids)}", flush=True)
        X_tr, y_tr = load_many_train_frames_v15(tr_ids)
        t0 = time.time()
        bst = train_fold_model_v15(X_tr, y_tr, seed=seed+fold, num_rounds=num_rounds)
        print(f"[OOF] Fold {fold} model trained in {time.time()-t0:.1f}s", flush=True)
        # Predict each VA id independently and save per-id probs
        for i, sid in enumerate(va_ids, 1):
            X_va, _, seq20 = load_train_item_v15(sid)
            P = bst.predict(xgb.DMatrix(X_va))
            outp = os.path.join(OOF_DIR_V15, f'oof_{sid:05d}.npz')
            np.savez_compressed(outp, probs=P.astype(np.float32), seq=seq20, sid=sid)
            if i % 10 == 0:
                print(f"  [OOF] Fold {fold} saved {i}/{len(va_ids)}", flush=True)
    print(f"[OOF] Completed {n_splits}-fold OOF in {time.time()-t_all:.1f}s. Files in {OOF_DIR_V15}")

def load_all_oof_items_v15():
    files = sorted(glob.glob(os.path.join(OOF_DIR_V15, 'oof_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        sid = int(d['sid']); P = d['probs']; seq = d['seq']
        items.append((sid, P, seq))
    return items

def grid_search_decoder_dp_on_oof():
    items = load_all_oof_items_v15()
    assert items, 'No OOF files found. Run build_oof_probs_v15() first.'
    grids = {
        'lambda': [0.6, 0.8, 1.0, 1.2],
        'bg_bias': [0.2, 0.3, 0.4],
        'merge_gap': [3, 4, 5, 6],
        'min_len': [6, 8, 10, 12],
        'mean_thr': [0.50],
        'max_thr': [0.65]
    }
    best = (1e9, None); tried = 0; t0 = time.time()
    for lam in grids['lambda']:
        for bb in grids['bg_bias']:
            for mg in grids['merge_gap']:
                for ml in grids['min_len']:
                    for mthr in grids['mean_thr']:
                        for xthr in grids['max_thr']:
                            scores = []
                            for sid, P, seq in items:
                                pred = decode_sequence_dp(P, switch_penalty=lam, bg_bias=bb, merge_gap=mg, min_len=ml, mean_thr=mthr, max_thr=xthr)
                                lev = levenshtein(list(pred), list(seq.tolist()))
                                # Normalize by 20 fixed tokens
                                scores.append(lev / 20.0)
                            mean_norm = float(np.mean(scores)) if scores else 1.0
                            tried += 1
                            if tried % 20 == 0:
                                print(f"[Grid-OOF] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
                            if mean_norm < best[0]:
                                best = (mean_norm, {'lambda':lam,'bg_bias':bb,'merge_gap':mg,'min_len':ml,'mean_thr':mthr,'max_thr':xthr})
    print(f"[Grid-OOF] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}")
    return best

print('[OOF] Ready: run build_oof_probs_v15(); then best_oof = grid_search_decoder_dp_on_oof() to tune decoder on TRAIN-only OOF.', flush=True)

[OOF] Ready: run build_oof_probs_v15(); then best_oof = grid_search_decoder_dp_on_oof() to tune decoder on TRAIN-only OOF.


In [26]:
# 15) Train on TRAIN+VAL (v1.5), predict TEST, decode with rescue, and write submission.csv
import os, glob, time, json, numpy as np, pandas as pd
import xgboost as xgb

def load_frames_from_cache_v15(prefix:str):
    files = sorted(glob.glob(os.path.join(CACHE_DIR_V15, f"{prefix}_*.npz")))
    Xs, ys = [], []
    n = 0; t0 = time.time()
    for i, fp in enumerate(files, 1):
        d = np.load(fp, allow_pickle=False)
        X = d['X']; y = d['y']
        Xs.append(X); ys.append(y.astype(np.int32)); n += len(y)
        if i % 20 == 0:
            print(f"[LoadV15] {prefix}: {i}/{len(files)} files, cum frames={n}", flush=True)
    X = np.vstack(Xs) if Xs else np.zeros((0,0), dtype=np.float32)
    y = np.concatenate(ys) if ys else np.zeros((0,), dtype=np.int32)
    print(f"[LoadV15] {prefix}: X={X.shape} y={y.shape} files={len(files)}", flush=True)
    return X, y, files

def ensure_test_cache_v15():
    test_ids = test_df['Id'].tolist()
    todo = []
    for sid in test_ids:
        out_path = os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz')
        if not os.path.exists(out_path):
            todo.append(sid)
    if todo:
        print(f"[CacheV15][TEST] {len(todo)}/{len(test_ids)} missing; caching with stride=2...")
        parallel_cache_v15('test', test_ids, max_workers=12, stride=2)
    else:
        print('[CacheV15][TEST] All test cached.')

def make_weights_with_boundary_erosion(y: np.ndarray, w0: float=0.38):
    w = np.ones_like(y, dtype=np.float32); w[y==0] = w0
    if len(y) > 2:
        bmask = (y[1:-1] != y[:-2]) | (y[1:-1] != y[2:])
        w[1:-1][bmask] = 0.0
    return w

def train_seed_model_v15(X, y, seed:int):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    print(f"[XGB-V15][Seed {seed}] Training...")
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=200)
    print(f"[XGB-V15][Seed {seed}] Done in {time.time()-t0:.1f}s")
    return bst

def order_preserving_perm_from_probs(P: np.ndarray, alpha: float = 1.5) -> list:
    # Build a length-20 permutation by sorting classes 1..20 by temporal centroid of P(t,c)^alpha
    T, C = P.shape
    assert C >= 21, 'Expect probs with 21 classes (0=bg, 1..20 gestures)'
    t = np.arange(T, dtype=np.float32)
    W = np.power(P[:, 1:], alpha)  # (T,20)
    denom = W.sum(axis=0) + 1e-9
    mu = (W * t[:, None]).sum(axis=0) / denom  # (20,)
    # Tie-break by total mass (descending) then peak prob (descending)
    mass = P[:, 1:].sum(axis=0)
    peak = P[:, 1:].max(axis=0)
    # argsort by mu asc; for ties adjust by -mass and -peak
    idx_mu = np.argsort(mu, kind='mergesort')
    # Stable sorts: first by negative peak, then negative mass, then mu
    idx = idx_mu
    # apply tie-breakers via stable argsort on keys in reverse priority
    order_mass = np.argsort(-mass, kind='mergesort')
    order_peak = np.argsort(-peak, kind='mergesort')
    # Compose by ranking positions
    ranks_mu = np.empty_like(idx_mu); ranks_mu[idx_mu] = np.arange(len(idx_mu))
    ranks_mass = np.empty_like(order_mass); ranks_mass[order_mass] = np.arange(len(order_mass))
    ranks_peak = np.empty_like(order_peak); ranks_peak[order_peak] = np.arange(len(order_peak))
    score = ranks_mu + 1e-3*ranks_mass + 1e-6*ranks_peak
    final_idx = np.argsort(score, kind='mergesort')
    classes = [int(i+1) for i in final_idx]
    return classes

def ensure_permutation_20(pred_seq: list, P: np.ndarray) -> list:
    # Keep unique classes from pred in order if already a perfect permutation; else fall back to order-only ranking
    uniq = []
    seen = set()
    for c in pred_seq:
        if 1 <= int(c) <= 20 and int(c) not in seen:
            uniq.append(int(c)); seen.add(int(c))
    if len(uniq) == 20:
        return uniq
    return order_preserving_perm_from_probs(P, alpha=1.5)

# Use DP decoder if params contains DP keys; else use vanilla decode_sequence
def decode_with_rescue(P, params):
    if all(k in params for k in ('lambda','bg_bias','merge_gap','min_len','mean_thr','max_thr')):
        pred = decode_sequence_dp(P, switch_penalty=params['lambda'], bg_bias=params['bg_bias'],
                                 merge_gap=params['merge_gap'], min_len=params['min_len'],
                                 mean_thr=params['mean_thr'], max_thr=params['max_thr'])
    else:
        pred = decode_sequence(P, window=params['window'], merge_gap=params['merge_gap'],
                               min_len=params['min_len'], mean_thr=params['mean_thr'], max_thr=params['max_thr'])
    return ensure_permutation_20(pred, P)

def predict_test_and_write_submission_v15(models, decoder_params, out_csv='submission.csv'):
    test_ids = test_df['Id'].tolist()
    rows = []
    t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']
        Ps = []
        dm = xgb.DMatrix(X)
        for m in models:
            Ps.append(m.predict(dm))
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = decode_with_rescue(P, decoder_params)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv(out_csv, index=False)
    print(f"[SUBMISSION] Wrote {out_csv} with {len(rows)} rows.")
    return out_csv

def run_trainval_and_test_v15(best_decoder_params:dict, seeds=(2025, 1337)):
    print('[Train+Val V1.5] Loading TRAIN and VAL caches...')
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    X_va, y_va, _ = load_frames_from_cache_v15('val')
    X_all = np.vstack([X_tr, X_va]); y_all = np.concatenate([y_tr, y_va])
    print('[Train+Val V1.5] Frames:', X_all.shape, 'Labels:', y_all.shape)
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_all, y_all, seed=s))
    ensure_test_cache_v15()
    return predict_test_and_write_submission_v15(models, best_decoder_params, out_csv='submission.csv')

print('[StageV15][Final] Ready: run run_trainval_and_test_v15(best_params_dict) after grid search to produce submission.csv')

[RUN-OOF] Building 5-fold OOF per-id probabilities on TRAIN (v1.5)...


[OOF] Fold 1/5: tr=237 va=60


[LoadTrainFold] 20/237 ids, cum frames=12612


[LoadTrainFold] 40/237 ids, cum frames=26771


[LoadTrainFold] 60/237 ids, cum frames=40910


...

[Grid-OOF] 120 combos, curr mean=0.3005 best=0.2002


[Grid-OOF] 140 combos, curr mean=0.2992 best=0.2002


[Grid-OOF] 160 combos, curr mean=0.2965 best=0.2002


[Grid-OOF] 180 combos, curr mean=0.2929 best=0.2002


[Grid-OOF] Done 192 combos in 315.2s. Best=(0.2001683501683502, {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65})
Best OOF DP decoder params: (0.2001683501683502, {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65})
[RUN-OOF] Total elapsed: 1010.7s


In [27]:
# 22) Final run with OOF-tuned DP params and order-preserving rescue
print('[FINAL RUN - OOF PARAMS] Using OOF-tuned DP decoder params...')
dp_params_oof = {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}
print('DP params (OOF):', dp_params_oof)
sub_path = run_trainval_and_test_v15(dp_params_oof, seeds=(2025, 1337))
print('Submission written to:', sub_path)

[FINAL RUN - OOF PARAMS] Using OOF-tuned DP decoder params...
DP params (OOF): {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}
[Train+Val V1.5] Loading TRAIN and VAL caches...
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST] 60/95 done; elapsed 32.2s


[TEST] 70/95 done; elapsed 37.5s


[TEST] 80/95 done; elapsed 42.7s


[TEST] 90/95 done; elapsed 47.8s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [33]:
# 23) Order-only centroid ranker: OOF grid search for alpha and smoothing; final infer using ranker
import numpy as np, glob, os, time, pandas as pd, xgboost as xgb

def smooth_probs(P: np.ndarray, win: int = 1) -> np.ndarray:
    if win <= 1: return P
    win = int(win) | 1
    pad = win // 2
    T, C = P.shape
    out = np.empty_like(P)
    for c in range(C):
        v = P[:, c].astype(np.float32)
        buf = np.pad(v, (pad, pad), mode='edge').astype(np.float64)
        cs = np.cumsum(np.concatenate(([0.0], buf)))  # ensure length T after windowing
        sm = (cs[win:] - cs[:-win]) / win  # length T
        out[:, c] = sm.astype(np.float32)
    return out

def order_only_decode(P: np.ndarray, alpha: float = 1.5, smooth_win: int = 1) -> list:
    P2 = smooth_probs(P, smooth_win)
    T, C = P2.shape
    t = np.arange(T, dtype=np.float32)
    W = np.power(P2[:, 1:], alpha)  # exclude bg
    denom = W.sum(axis=0) + 1e-9
    mu = (W * t[:, None]).sum(axis=0) / denom  # (20,)
    mass = P2[:, 1:].sum(axis=0)
    peak = P2[:, 1:].max(axis=0)
    # rank by mu asc; tie-break by mass desc, then peak desc
    ranks_mu = np.argsort(mu, kind='mergesort')
    ranks_mass = np.argsort(-mass, kind='mergesort')
    ranks_peak = np.argsort(-peak, kind='mergesort')
    inv_mu = np.empty_like(ranks_mu); inv_mu[ranks_mu] = np.arange(20)
    inv_mass = np.empty_like(ranks_mass); inv_mass[ranks_mass] = np.arange(20)
    inv_peak = np.empty_like(ranks_peak); inv_peak[ranks_peak] = np.arange(20)
    score = inv_mu + 1e-3*inv_mass + 1e-6*inv_peak
    final_idx = np.argsort(score, kind='mergesort')
    return [int(i+1) for i in final_idx]

def load_all_oof_items_v15():
    files = sorted(glob.glob(os.path.join(OOF_DIR_V15, 'oof_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        items.append((int(d['sid']), d['probs'], d['seq']))
    return items

def grid_search_ranker_on_oof():
    items = load_all_oof_items_v15()
    assert items, 'No OOF files found. Build OOF first.'
    alphas = [1.2, 1.5, 1.8, 2.0]
    wins = [1, 3, 5, 7, 9]
    best = (1e9, None); tried = 0; t0 = time.time()
    for a in alphas:
        for w in wins:
            scores = []
            for sid, P, seq in items:
                pred = order_only_decode(P, alpha=a, smooth_win=w)
                lev = levenshtein(pred, list(seq.tolist()))
                scores.append(lev/20.0)
            mean_norm = float(np.mean(scores))
            tried += 1
            if tried % 5 == 0:
                print(f"[Grid-OOF-Ranker] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
            if mean_norm < best[0]:
                best = (mean_norm, {'alpha': a, 'smooth_win': w})
    print(f"[Grid-OOF-Ranker] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}")
    return best

def predict_test_with_ranker_and_write_submission_v15(models, ranker_params, out_csv='submission.csv'):
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']
        dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = order_only_decode(P, alpha=ranker_params['alpha'], smooth_win=ranker_params['smooth_win'])
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-Ranker] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv(out_csv, index=False)
    print(f"[SUBMISSION] Wrote {out_csv} with {len(rows)} rows.")
    return out_csv

print('[Ranker] Ready: run best_ranker = grid_search_ranker_on_oof(); then predict_test_with_ranker_and_write_submission_v15(models, best_ranker[1])')

[Ranker] Ready: run best_ranker = grid_search_ranker_on_oof(); then predict_test_with_ranker_and_write_submission_v15(models, best_ranker[1])


In [32]:
# 24) Run OOF ranker grid, then train on TRAIN+VAL and infer TEST with ranker
print('[Ranker] Grid-search on OOF...')
best_ranker = grid_search_ranker_on_oof()
print('Best ranker params:', best_ranker)

print('[Ranker] Training models on TRAIN+VAL for final inference...')
X_tr, y_tr, _ = load_frames_from_cache_v15('train')
X_va, y_va, _ = load_frames_from_cache_v15('val')
X_all = np.vstack([X_tr, X_va]); y_all = np.concatenate([y_tr, y_va])
models = []
for s in (2025, 1337):
    models.append(train_seed_model_v15(X_all, y_all, seed=s))
ensure_test_cache_v15()
sub_ranker = predict_test_with_ranker_and_write_submission_v15(models, best_ranker[1], out_csv='submission.csv')
print('Ranker submission written to:', sub_ranker)

[Ranker] Grid-search on OOF...


[Grid-OOF-Ranker] 5 combos, curr mean=0.5184 best=0.5204


[Grid-OOF-Ranker] 10 combos, curr mean=0.4497 best=0.4529


[Grid-OOF-Ranker] 15 combos, curr mean=0.3928 best=0.3955


[Grid-OOF-Ranker] 20 combos, curr mean=0.3643 best=0.3687


...

[LoadV15] val: X=(213026, 120) y=(213026,) files=277


[XGB-V15][Seed 2025] Training...


[XGB-V15][Seed 2025] Done in 234.7s
[XGB-V15][Seed 1337] Training...


[XGB-V15][Seed 1337] Done in 236.2s
[CacheV15][TEST] All test cached.


NameError: name 'xgboost' is not defined

In [34]:
# 25) Fix alias and run ranker inference without retraining
import xgboost as xgb
xgboost = xgb  # provide alias for legacy function reference
print('[Ranker] Running test inference with order-only ranker using best_ranker params...')
ensure_test_cache_v15()
sub_ranker = predict_test_with_ranker_and_write_submission_v15(models, best_ranker[1], out_csv='submission.csv')
print('Ranker submission written to:', sub_ranker)

[Ranker] Running test inference with order-only ranker using best_ranker params...
[CacheV15][TEST] All test cached.


[TEST-Ranker] 10/95 done; elapsed 5.5s


[TEST-Ranker] 20/95 done; elapsed 10.9s


[TEST-Ranker] 30/95 done; elapsed 16.3s


[TEST-Ranker] 40/95 done; elapsed 21.5s


...

[TEST-Ranker] 60/95 done; elapsed 32.2s


[TEST-Ranker] 70/95 done; elapsed 37.6s


[TEST-Ranker] 80/95 done; elapsed 42.9s


[TEST-Ranker] 90/95 done; elapsed 48.0s


[SUBMISSION] Wrote submission.csv with 95 rows.
Ranker submission written to: submission.csv


In [35]:
# 26) Train on TRAIN-only (no VAL frames), infer TEST with OOF-tuned DP params
print('[FINAL RUN - TRAIN ONLY] Training models on TRAIN cache (v1.5), no VAL frames used...')
dp_params_oof = {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}
print('Using DP params (OOF):', dp_params_oof)

X_tr, y_tr, _ = load_frames_from_cache_v15('train')
print('[TRAIN ONLY] Frames:', X_tr.shape, 'Labels:', y_tr.shape)
models = []
for s in (2025, 1337):
    models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
ensure_test_cache_v15()
sub_path = predict_test_and_write_submission_v15(models, dp_params_oof, out_csv='submission.csv')
print('Submission (TRAIN-only) written to:', sub_path)

[FINAL RUN - TRAIN ONLY] Training models on TRAIN cache (v1.5), no VAL frames used...
Using DP params (OOF): {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST] 60/95 done; elapsed 32.2s


[TEST] 70/95 done; elapsed 37.6s


[TEST] 80/95 done; elapsed 42.8s


[TEST] 90/95 done; elapsed 47.9s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission (TRAIN-only) written to: submission.csv


In [36]:
# 27) Full-pipeline decoding: temperature + smoothing + DP + order-preserving rescue; OOF grid
import numpy as np, glob, os, time, pandas as pd, xgboost as xgb

# Reuse OOF dir from earlier
OOF_DIR_V15 = './oof_probs_v15'

def smooth_probs_ma(P: np.ndarray, win: int = 1) -> np.ndarray:
    if win <= 1: return P
    win = int(win) | 1
    pad = win // 2
    T, C = P.shape
    out = np.empty_like(P)
    for c in range(C):
        v = P[:, c].astype(np.float32)
        buf = np.pad(v, (pad, pad), mode='edge').astype(np.float64)
        cs = np.cumsum(np.concatenate(([0.0], buf)))
        sm = (cs[win:] - cs[:-win]) / win
        out[:, c] = sm.astype(np.float32)
    return out

def apply_temperature(P: np.ndarray, T: float = 1.5) -> np.ndarray:
    # Work with probabilities; approximate temperature by power 1/T then re-normalize per frame
    T = float(T)
    if abs(T - 1.0) < 1e-6: return P
    Ppow = np.power(np.clip(P, 1e-9, 1.0), 1.0 / T).astype(np.float32)
    Ppow /= (Ppow.sum(axis=1, keepdims=True) + 1e-9)
    return Ppow

def class_time_centroids(P: np.ndarray, alpha: float = 1.5):
    # returns mu (20,), mass (20,), peak (20,) for classes 1..20
    T, C = P.shape
    t = np.arange(T, dtype=np.float32)
    W = np.power(P[:, 1:], alpha)
    denom = W.sum(axis=0) + 1e-9
    mu = (W * t[:, None]).sum(axis=0) / denom
    mass = P[:, 1:].sum(axis=0)
    peak = P[:, 1:].max(axis=0)
    return mu, mass, peak

def ensure_perm20_preserve_dp(dp_seq: list, P: np.ndarray, alpha: float = 1.5) -> list:
    # Keep DP unique classes in their DP order; insert missing classes by centroid time relative to DP times
    # Compute centroids for all classes 1..20
    mu, mass, peak = class_time_centroids(P, alpha=alpha)
    # Unique DP classes within [1..20]
    dp_uniq = []
    seen = set()
    for c in dp_seq:
        if 1 <= int(c) <= 20 and int(c) not in seen:
            dp_uniq.append(int(c)); seen.add(int(c))
    if len(dp_uniq) == 20:
        return dp_uniq
    missing = [c for c in range(1, 21) if c not in seen]
    # Compute mu for DP classes (aligned to their order) and for missing
    mu_dp = [mu[c-1] for c in dp_uniq]
    # Insert each missing class by its mu into the dp_uniq without reordering dp_uniq
    # Sort missing by mu ascending to insert in temporal order
    missing_sorted = sorted(missing, key=lambda c: (mu[c-1], -mass[c-1], -peak[c-1]))
    out = list(dp_uniq)
    mu_out = list(mu_dp)
    for c in missing_sorted:
        m = mu[c-1]
        # find first position where m <= mu_out[i]; else append
        pos = None
        for i in range(len(mu_out)):
            if m <= mu_out[i]:
                pos = i; break
        if pos is None:
            out.append(c); mu_out.append(m)
        else:
            out.insert(pos, c); mu_out.insert(pos, m)
        if len(out) > 20:
            # if overflow due to pathological insertions, drop last lowest-confidence by mass, but keep DP-chosen fixed
            # Keep first 20 as a safe fallback (rare)
            out = out[:20]; mu_out = mu_out[:20]
    # Guarantee size 20
    if len(out) < 20:
        # Fill by remaining classes ordered by descending mass
        rem2 = [c for c in range(1,21) if c not in set(out)]
        rem2.sort(key=lambda c: mass[c-1], reverse=True)
        out.extend(rem2[:(20-len(out))])
    return out[:20]

def preprocess_probs(P: np.ndarray, T: float, smooth_win: int) -> np.ndarray:
    P2 = apply_temperature(P, T=T)
    P3 = smooth_probs_ma(P2, win=smooth_win)
    return P3

def decode_full_pipeline(P: np.ndarray, dp_params: dict, T: float = 1.5, smooth_win: int = 5, alpha_fill: float = 1.5) -> list:
    Pp = preprocess_probs(P, T=T, smooth_win=smooth_win)
    pred = decode_sequence_dp(Pp,
                             switch_penalty=dp_params.get('lambda', 0.8),
                             bg_bias=dp_params.get('bg_bias', 0.2),
                             merge_gap=dp_params.get('merge_gap', 3),
                             min_len=dp_params.get('min_len', 10),
                             mean_thr=dp_params.get('mean_thr', 0.5),
                             max_thr=dp_params.get('max_thr', 0.65))
    return ensure_perm20_preserve_dp(pred, Pp, alpha=alpha_fill)

def load_all_oof_items_v15():
    files = sorted(glob.glob(os.path.join(OOF_DIR_V15, 'oof_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        items.append((int(d['sid']), d['probs'], d['seq']))
    return items

def grid_search_full_pipeline_on_oof():
    items = load_all_oof_items_v15()
    assert items, 'No OOF files found (run build_oof_probs_v15 first).'
    Ts = [1.3, 1.5, 1.7, 1.9]
    wins = [1, 5]
    lambdas = [0.8, 1.0]
    minlens = [10, 12]
    bg_biases = [0.2, 0.3]
    merge_gaps = [3, 4]
    best = (1e9, None); tried = 0; t0 = time.time()
    for T in Ts:
        for win in wins:
            for lam in lambdas:
                for ml in minlens:
                    for bb in bg_biases:
                        for mg in merge_gaps:
                            dp = {'lambda': lam, 'bg_bias': bb, 'merge_gap': mg, 'min_len': ml, 'mean_thr': 0.5, 'max_thr': 0.65}
                            scores = []
                            for sid, P, seq in items:
                                pred = decode_full_pipeline(P, dp, T=T, smooth_win=win, alpha_fill=1.5)
                                lev = levenshtein(list(pred), list(seq.tolist()))
                                scores.append(lev / 20.0)
                            mean_norm = float(np.mean(scores))
                            tried += 1
                            if tried % 10 == 0:
                                print(f"[Grid-OOF-Full] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
                            if mean_norm < best[0]:
                                best = (mean_norm, {'T': T, 'smooth_win': win, **dp})
    print(f"[Grid-OOF-Full] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}")
    return best

def predict_test_full_pipeline_train_only(best_params: dict, seeds=(2025,1337,42)):
    # Train TRAIN-only models, average probs, apply full pipeline and write submission.csv
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = decode_full_pipeline(P, best_params, T=best_params['T'], smooth_win=best_params['smooth_win'], alpha_fill=1.5)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-Full] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[FullPipeline] Ready: run best_full = grid_search_full_pipeline_on_oof(); then predict_test_full_pipeline_train_only(best_full[1], seeds=(2025,1337,42))')

[FullPipeline] Ready: run best_full = grid_search_full_pipeline_on_oof(); then predict_test_full_pipeline_train_only(best_full[1], seeds=(2025,1337,42))


In [37]:
# 28) Run full-pipeline OOF grid, then 3-seed TRAIN-only final inference
print('[FullPipeline] Grid-search on TRAIN OOF with temperature + smoothing + conservative DP + order-preserving rescue...')
best_full = grid_search_full_pipeline_on_oof()
print('Best full-pipeline params:', best_full)

print('[FullPipeline] TRAIN-only 3-seed inference with best params...')
sub_path = predict_test_full_pipeline_train_only(best_full[1], seeds=(2025,1337,42))
print('Submission written to:', sub_path)

[FullPipeline] Grid-search on TRAIN OOF with temperature + smoothing + conservative DP + order-preserving rescue...


[Grid-OOF-Full] 10 combos, curr mean=0.3207 best=0.3207


[Grid-OOF-Full] 20 combos, curr mean=0.3320 best=0.3207


[Grid-OOF-Full] 30 combos, curr mean=0.3453 best=0.3207


[Grid-OOF-Full] 40 combos, curr mean=0.3763 best=0.3207


...

[TEST-Full] 60/95 done; elapsed 48.1s


[TEST-Full] 70/95 done; elapsed 56.1s


[TEST-Full] 80/95 done; elapsed 63.9s


[TEST-Full] 90/95 done; elapsed 71.5s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [38]:
# 29) Full-pipeline final attempt: conservative DP + T=1.5, win=5, 5-seed TRAIN-only
print('[FullPipeline-Final] Using conservative settings per expert advice...')
best_params_final = {'T': 1.5, 'smooth_win': 5, 'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
print('Params:', best_params_final)
sub_path = predict_test_full_pipeline_train_only(best_params_final, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[FullPipeline-Final] Using conservative settings per expert advice...
Params: {'T': 1.5, 'smooth_win': 5, 'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST-Full] 60/95 done; elapsed 79.9s


[TEST-Full] 70/95 done; elapsed 93.2s


[TEST-Full] 80/95 done; elapsed 106.2s


[TEST-Full] 90/95 done; elapsed 118.8s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [53]:
# 30) Align to strong OOF DP: T=1.0, win=1, lambda=0.6, min_len=6; TRAIN-only 3-seed with order-preserving fill
print('[FullPipeline-OOFDP] Using raw-DP-aligned params (no temp, no smoothing) to match OOF best...')
params_oofdp = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}
print('Params:', params_oofdp)
sub_path = predict_test_full_pipeline_train_only(params_oofdp, seeds=(2025,1337,42))
print('Submission written to:', sub_path)

[FullPipeline-OOFDP] Using raw-DP-aligned params (no temp, no smoothing) to match OOF best...
Params: {'T': 1.0, 'smooth_win': 1, 'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST-Full] 60/95 done; elapsed 48.4s


[TEST-Full] 70/95 done; elapsed 56.5s


[TEST-Full] 80/95 done; elapsed 64.4s


[TEST-Full] 90/95 done; elapsed 72.0s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [40]:
# 31) Conservative DP, TRAIN-only, 5-seed, no temp/smoothing (align OOF hygiene)
print('[FullPipeline-Conservative] TRAIN-only 5-seed, T=1.0, smooth_win=1, DP lambda=0.8, min_len=12 ...')
best_params_conservative = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
print('Params:', best_params_conservative)
sub_path = predict_test_full_pipeline_train_only(best_params_conservative, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[FullPipeline-Conservative] TRAIN-only 5-seed, T=1.0, smooth_win=1, DP lambda=0.8, min_len=12 ...
Params: {'T': 1.0, 'smooth_win': 1, 'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST-Full] 60/95 done; elapsed 79.3s


[TEST-Full] 70/95 done; elapsed 92.5s


[TEST-Full] 80/95 done; elapsed 105.2s


[TEST-Full] 90/95 done; elapsed 117.7s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [41]:
# 32) Per-class duration priors (p30 clipped to [8,12]) + conservative DP; TRAIN-only 5-seed; no temp/smoothing
import numpy as np, glob, os, time, pandas as pd, xgboost as xgb

def compute_duration_priors_from_train_cache_v15(p=30.0, clip_low=8, clip_high=12):
    files = sorted(glob.glob(os.path.join(CACHE_DIR_V15, 'train_*.npz')))
    lens = {c: [] for c in range(1,21)}
    for i, fp in enumerate(files, 1):
        d = np.load(fp, allow_pickle=False)
        y = d['y'].astype(np.int32)
        # collect run lengths per class (exclude bg=0)
        if y.size == 0: continue
        b = 0
        for t in range(1, len(y)+1):
            if t==len(y) or y[t] != y[b]:
                cls = int(y[b])
                if cls != 0:
                    lens[cls].append(t-b)
                b = t
        if i % 25 == 0:
            print(f"[DurPrior] processed {i}/{len(files)} train files", flush=True)
    min_len_map = {}
    for c in range(1,21):
        arr = np.array(lens[c], dtype=np.float32)
        if arr.size == 0:
            ml = clip_low
        else:
            ml = float(np.percentile(arr, p))
        ml = int(max(clip_low, min(clip_high, round(ml))))
        min_len_map[c] = ml
    print('[DurPrior] per-class min_len (clipped):', min_len_map)
    return min_len_map

def decode_sequence_dp_pc(probs, switch_penalty=1.0, bg_bias=0.0, merge_gap=4, min_len=10, mean_thr=0.5, max_thr=0.65, min_len_map=None):
    labels = viterbi_labels(probs, switch_penalty=switch_penalty, bg_bias=bg_bias)
    T = len(labels)
    segs = []  # (cls, b, e, mean_p, max_p)
    b = 0
    for i in range(1, T+1):
        if i==T or labels[i] != labels[b]:
            cls = int(labels[b])
            p = probs[b:i, cls] if cls < probs.shape[1] else np.zeros(i-b)
            mean_p = float(p.mean()) if (i-b)>0 else 0.0
            max_p = float(p.max()) if (i-b)>0 else 0.0
            segs.append([cls, b, i, mean_p, max_p])
            b = i
    # merge small bg gaps between same class
    merged = []
    i = 0
    while i < len(segs):
        cur = segs[i]
        j = i + 1
        while j < len(segs):
            if segs[j][0] == cur[0] and segs[j-1][0]==0 and (segs[j][1]-cur[2]) <= merge_gap:
                cur[2] = segs[j][2]
                cur[3] = float(np.mean(probs[cur[1]:cur[2], cur[0]]))
                cur[4] = float(np.max(probs[cur[1]:cur[2], cur[0]]))
                j += 1
            else:
                break
        merged.append(cur)
        i = j
    out = []
    for cls, sb, se, mp, xp in merged:
        if cls == 0: continue
        req_len = int(min_len_map.get(int(cls), min_len)) if min_len_map is not None else int(min_len)
        if (se - sb) < req_len: continue
        if not (mp >= mean_thr and xp >= max_thr): continue
        if not out or out[-1] != int(cls):
            out.append(int(cls))
    return out

def decode_full_pipeline_pc(P: np.ndarray, dp_params: dict, min_len_map: dict, T: float = 1.0, smooth_win: int = 1, alpha_fill: float = 1.5) -> list:
    Pp = preprocess_probs(P, T=T, smooth_win=smooth_win)
    pred = decode_sequence_dp_pc(Pp,
                                switch_penalty=dp_params.get('lambda', 0.8),
                                bg_bias=dp_params.get('bg_bias', 0.2),
                                merge_gap=dp_params.get('merge_gap', 3),
                                min_len=dp_params.get('min_len', 12),
                                mean_thr=dp_params.get('mean_thr', 0.5),
                                max_thr=dp_params.get('max_thr', 0.65),
                                min_len_map=min_len_map)
    return ensure_perm20_preserve_dp(pred, Pp, alpha=alpha_fill)

def predict_test_full_pipeline_train_only_pc(best_params: dict, min_len_map: dict, seeds=(2025,1337,42,0,123)):
    # TRAIN-only models, average probs, per-class-duration DP decode
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgboost.DMatrix(X) if 'xgboost' in globals() else xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = decode_full_pipeline_pc(P, best_params, min_len_map=min_len_map, T=best_params.get('T',1.0), smooth_win=best_params.get('smooth_win',1), alpha_fill=1.5)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-PC] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[PerClassMinLen] Computing duration priors from TRAIN cache (stride=2)...')
min_len_map = compute_duration_priors_from_train_cache_v15(p=30.0, clip_low=8, clip_high=12)
print('[PerClassMinLen] Running TRAIN-only 5-seed inference with conservative DP and per-class min_len...')
best_params_conservative = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_full_pipeline_train_only_pc(best_params_conservative, min_len_map, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[PerClassMinLen] Computing duration priors from TRAIN cache (stride=2)...
[DurPrior] processed 25/297 train files


[DurPrior] processed 50/297 train files


[DurPrior] processed 75/297 train files


[DurPrior] processed 100/297 train files


[DurPrior] processed 125/297 train files


...

[TEST-PC] 60/95 done; elapsed 79.1s


[TEST-PC] 70/95 done; elapsed 92.3s


[TEST-PC] 80/95 done; elapsed 105.1s


[TEST-PC] 90/95 done; elapsed 117.6s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [42]:
# 33) Test-time stride=1 TTA: cache TEST at stride=1 and decode with doubled min_len (robust DP + DP-preserving rescue)
import os, glob, time, numpy as np, pandas as pd, xgboost as xgb

# Separate cache dir for stride=1 to avoid collisions
CACHE_DIR_V15_S1 = './cache_v15_s1'
os.makedirs(CACHE_DIR_V15_S1, exist_ok=True)

def cache_sample_v15_to_dir(sample_id:int, split_hint:str, stride:int, out_dir:str):
    out_path = os.path.join(out_dir, f'{split_hint}_{sample_id:05d}.npz')
    if os.path.exists(out_path):
        return out_path
    vid = load_video_from_split(sample_id, split_hint)
    skel, idxs = stack_world_positions(vid, stride=stride)
    X = features_from_skeleton_v15(skel)
    fps = int(vid.get('FrameRate', 20))
    # Only TEST: no labels/seq stored
    np.savez_compressed(out_path, X=X, fps=fps, stride=stride, sid=sample_id)
    return out_path

def ensure_test_cache_v15_stride1():
    test_ids = test_df['Id'].tolist()
    todo = []
    for sid in test_ids:
        out_path = os.path.join(CACHE_DIR_V15_S1, f'test_{sid:05d}.npz')
        if not os.path.exists(out_path):
            todo.append(sid)
    if not todo:
        print('[CacheV15][TEST s=1] All test cached.')
        return
    print(f"[CacheV15][TEST s=1] {len(todo)}/{len(test_ids)} missing; caching with stride=1...")
    import concurrent.futures as cf, time
    t0 = time.time(); done = 0
    def task(sid):
        try:
            return cache_sample_v15_to_dir(sid, 'test', stride=1, out_dir=CACHE_DIR_V15_S1)
        except Exception as e:
            print(f'[CacheV15][TEST s=1][ERR] id={sid}:', e)
            return None
    with cf.ThreadPoolExecutor(max_workers=12) as ex:
        futs = [ex.submit(task, sid) for sid in todo]
        for i, fut in enumerate(cf.as_completed(futs), 1):
            _ = fut.result(); done += 1
            if done % 10 == 0:
                print(f"[CacheV15][TEST s=1] {done}/{len(todo)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    print(f"[CacheV15][TEST s=1] Completed {done} in {time.time()-t0:.1f}s")

def predict_test_stride1_train_only_conservative_dp(seeds=(2025,1337,42,0,123)):
    # Train models on TRAIN-only stride=2 cache (same as before)
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    # Ensure TEST stride=1 cache exists
    ensure_test_cache_v15_stride1()
    # Conservative DP, double min_len for stride=1
    dp_params = {'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 24, 'mean_thr': 0.5, 'max_thr': 0.65}
    rows = []; t0 = time.time()
    test_ids = test_df['Id'].tolist()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15_S1, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = decode_full_pipeline(P, dp_params, T=1.0, smooth_win=1, alpha_fill=1.5)  # uses ensure_perm20_preserve_dp
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-s1] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[Stride1] TRAIN-only 5-seed with TEST stride=1 and doubled min_len...')
sub_path = predict_test_stride1_train_only_conservative_dp(seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[Stride1] TRAIN-only 5-seed with TEST stride=1 and doubled min_len...
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST-s1] 60/95 done; elapsed 76.7s


[TEST-s1] 70/95 done; elapsed 89.7s


[TEST-s1] 80/95 done; elapsed 102.3s


[TEST-s1] 90/95 done; elapsed 114.7s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [43]:
# 34) Test-time TTA at stride=2 with offsets {0,1}: align+average probs, then conservative DP decode
import os, time, numpy as np, pandas as pd, xgboost as xgb, concurrent.futures as cf

CACHE_DIR_V15_S2_O1 = './cache_v15_s2_o1'
os.makedirs(CACHE_DIR_V15_S2_O1, exist_ok=True)

def stack_world_positions_offset(vid, stride:int=2, start:int=0):
    T = int(vid['NumFrames']); frames = vid['Frames']
    J = frames[0]['Skeleton']['WorldPosition'].shape[0]
    idxs = list(range(start, T, stride))
    skel = np.zeros((len(idxs), J, 3), dtype=np.float32)
    last = None
    for i, t in enumerate(idxs):
        wp = frames[t]['Skeleton']['WorldPosition']
        arr = np.asarray(wp, dtype=np.float32) if wp is not None else last
        if arr is None: arr = np.zeros((J,3), dtype=np.float32)
        skel[i] = arr; last = arr
    return skel, idxs

def cache_test_stride2_offset1():
    test_ids = test_df['Id'].tolist()
    todo = []
    for sid in test_ids:
        outp = os.path.join(CACHE_DIR_V15_S2_O1, f'test_{sid:05d}.npz')
        if not os.path.exists(outp):
            todo.append(sid)
    if not todo:
        print('[CacheV15][TEST s=2,o=1] All cached.')
        return
    print(f"[CacheV15][TEST s=2,o=1] {len(todo)}/{len(test_ids)} missing; caching...")
    t0 = time.time(); done = 0
    def task(sid):
        try:
            vid = load_video_from_split(sid, 'test')
            skel, idxs = stack_world_positions_offset(vid, stride=2, start=1)
            X = features_from_skeleton_v15(skel)
            fps = int(vid.get('FrameRate', 20))
            outp = os.path.join(CACHE_DIR_V15_S2_O1, f'test_{sid:05d}.npz')
            np.savez_compressed(outp, X=X, fps=fps, stride=2, start=1, sid=sid)
            return outp
        except Exception as e:
            print(f'[CacheV15][TEST s=2,o=1][ERR] id={sid}:', e)
            return None
    with cf.ThreadPoolExecutor(max_workers=12) as ex:
        futs = [ex.submit(task, sid) for sid in todo]
        for i, fut in enumerate(cf.as_completed(futs), 1):
            _ = fut.result(); done += 1
            if done % 10 == 0:
                print(f"[CacheV15][TEST s=2,o=1] {done}/{len(todo)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    print(f"[CacheV15][TEST s=2,o=1] Completed {done} in {time.time()-t0:.1f}s")

def interp_probs(P: np.ndarray, N: int) -> np.ndarray:
    # Linearly interpolate P (T,C) over normalized time to length N
    T, C = P.shape
    if T == N: return P.astype(np.float32)
    x_old = np.linspace(0.0, 1.0, T, dtype=np.float32)
    x_new = np.linspace(0.0, 1.0, N, dtype=np.float32)
    out = np.empty((N, C), dtype=np.float32)
    for c in range(C):
        out[:, c] = np.interp(x_new, x_old, P[:, c].astype(np.float32))
    # renormalize rows to sum to 1 (avoid drift)
    s = out.sum(axis=1, keepdims=True) + 1e-9
    out /= s
    return out

def predict_test_stride2_tta_offsets(seeds=(2025,1337,42,0,123)):
    # Train on TRAIN-only v15 cache
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    # Ensure both test caches exist: stride=2 start=0 already in cache_v15; start=1 build now
    ensure_test_cache_v15()
    cache_test_stride2_offset1()
    # Conservative DP params
    dp = {'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
    rows = []; t0 = time.time()
    test_ids = test_df['Id'].tolist()
    for i, sid in enumerate(test_ids, 1):
        d0 = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        d1 = np.load(os.path.join(CACHE_DIR_V15_S2_O1, f'test_{sid:05d}.npz'), allow_pickle=False)
        X0, X1 = d0['X'], d1['X']
        dm0 = xgb.DMatrix(X0); dm1 = xgb.DMatrix(X1)
        Ps0 = [m.predict(dm0) for m in models]
        Ps1 = [m.predict(dm1) for m in models]
        P0 = np.mean(np.stack(Ps0, axis=0), axis=0)
        P1 = np.mean(np.stack(Ps1, axis=0), axis=0)
        N = max(P0.shape[0], P1.shape[0])
        P0i = interp_probs(P0, N)
        P1i = interp_probs(P1, N)
        P = 0.5*(P0i + P1i)
        seq = decode_full_pipeline(P, dp, T=1.0, smooth_win=1, alpha_fill=1.5)  # DP-preserving rescue
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-s2-TTA] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[Stride2-TTA] TRAIN-only 5-seed; TEST stride=2 offsets {0,1}; align+avg probs; conservative DP decode...')
sub_path = predict_test_stride2_tta_offsets(seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[Stride2-TTA] TRAIN-only 5-seed; TEST stride=2 offsets {0,1}; align+avg probs; conservative DP decode...
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST-s2-TTA] 60/95 done; elapsed 157.5s


[TEST-s2-TTA] 70/95 done; elapsed 183.8s


[TEST-s2-TTA] 80/95 done; elapsed 209.2s


[TEST-s2-TTA] 90/95 done; elapsed 234.2s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [44]:
# 35) Grammar-enforced DP (fixed order G1..G20) using emissions log p(c) - log p(bg); TRAIN-only 5-seed
import numpy as np, os, glob, time, pandas as pd, xgboost as xgb

def grammar_decode_fixed_order(P: np.ndarray, min_len_map=None) -> list:
    # P: (T,21) with 0=bg, classes 1..20
    T, C = P.shape
    assert C >= 21, 'Expect 21 classes (bg + 20 gestures)'
    eps = 1e-9
    # emissions for gestures 1..20 vs background
    s = np.log(np.clip(P[:, 1:21], eps, 1.0)) - np.log(np.clip(P[:, [0]], eps, 1.0))  # (T,20)
    # cumulative sums per class for O(1) segment score
    cs = np.zeros((T+1, 20), dtype=np.float32)
    np.cumsum(s, axis=0, out=cs[1:])
    # per-class min_len
    if min_len_map is None:
        mins = np.full(20, 12, dtype=np.int32)
    else:
        mins = np.array([int(min_len_map.get(i+1, 12)) for i in range(20)], dtype=np.int32)
    # DP over classes i=0..19; DP_prev and DP_cur are length T+1 over end time t
    # DP_prev[t] is best score placing first i classes ending at frame t
    DP_prev = np.full(T+1, -1e18, dtype=np.float32)
    DP_prev[0] = 0.0  # before any gesture at time 0
    backpointers = []  # list of arrays bp_i[t] giving start u for segment i ending at t
    cum_min = 0
    for i in range(20):
        mlen = int(mins[i])
        DP_cur = np.full(T+1, -1e18, dtype=np.float32)
        bp = np.full(T+1, -1, dtype=np.int32)
        # Maintain running max M[t] = max_{u <= t-mlen} (DP_prev[u] - cs[u, i]) and arg u
        M_val = -1e18
        M_idx = -1
        # Earliest feasible end time must allow i+1 segments of min length
        cum_min += mlen
        start_t = cum_min
        for t in range(mlen, T+1):
            u = t - mlen
            cand = DP_prev[u] - cs[u, i]
            if cand > M_val:
                M_val = cand; M_idx = u
            # Only allow t where remaining time can fit remaining mins (not strictly needed at test)
            DP_cur[t] = cs[t, i] + M_val
            bp[t] = M_idx
        backpointers.append(bp)
        DP_prev = DP_cur
    # Choose best end time for class 20
    t_end = int(np.argmax(DP_prev))
    # Backtrack segments
    cuts = [0]*21
    cuts[20] = t_end
    for i in range(19, -1, -1):
        u = int(backpointers[i][cuts[i+1]])
        if u < 0: u = max(0, cuts[i+1] - int(mins[i]))
        cuts[i] = u
    # Return fixed order classes 1..20 (grammar enforces the order already)
    return [i+1 for i in range(20)]

def predict_test_train_only_grammar_dp(seeds=(2025,1337,42,0,123), min_len_map=None):
    # Train TRAIN-only models, average probs, decode with grammar DP, write submission.csv
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    rows = []; t0 = time.time()
    test_ids = test_df['Id'].tolist()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = grammar_decode_fixed_order(P, min_len_map=min_len_map)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-GrammarDP] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[GrammarDP] TRAIN-only 5-seed; decoding with fixed-order grammar DP (min_len=12)...')
sub_path = predict_test_train_only_grammar_dp(seeds=(2025,1337,42,0,123), min_len_map=None)
print('Submission written to:', sub_path)

[GrammarDP] TRAIN-only 5-seed; decoding with fixed-order grammar DP (min_len=12)...
[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


[LoadV15] train: 100/297 files, cum frames=67157


...

[TEST-GrammarDP] 60/95 done; elapsed 79.0s


[TEST-GrammarDP] 70/95 done; elapsed 92.2s


[TEST-GrammarDP] 80/95 done; elapsed 104.9s


[TEST-GrammarDP] 90/95 done; elapsed 117.4s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [45]:
# 36) FE v1.6 (v1.5 + kinematic extras: distances, speeds, elbow angles, acceleration) + caching/train/infer
import os, io, re, time, glob, tarfile, zipfile, numpy as np, concurrent.futures as cf, pandas as pd, xgboost as xgb
import scipy.io as sio

HIP=0; SPINE=1; SHOULDER_CENTER=2; HEAD=3; L_SHOULDER=4; L_ELBOW=5; L_WRIST=6; L_HAND=7; R_SHOULDER=8; R_ELBOW=9; R_WRIST=10; R_HAND=11

def features_from_skeleton_v16(skel):
    # v1.5 base (hip-center, shoulder-width scale) + extras: distances, speeds, elbow angles, acceleration
    T, J, _ = skel.shape
    base_dim = J*3*2  # positions + velocities
    extra_dim = 13 + 3*J  # distances/speeds/angles (13) + accelerations per joint (J*3)
    feats = np.zeros((T, base_dim + extra_dim), dtype=np.float32)
    prev_centered = None
    prev_vel = None
    for t in range(T):
        coords = skel[t].astype(np.float32)
        hip = coords[HIP]
        centered = coords - hip
        shoulder_w = np.linalg.norm(centered[R_SHOULDER] - centered[L_SHOULDER])
        scale = max(shoulder_w, 1e-4)
        centered /= scale
        # velocities on normalized coords
        if prev_centered is None:
            vel = np.zeros_like(centered)
        else:
            vel = centered - prev_centered
        # accelerations on normalized coords
        if prev_vel is None:
            acc = np.zeros_like(centered)
        else:
            acc = vel - prev_vel
        prev_vel = vel
        prev_centered = centered
        # base
        feats[t, :J*3] = centered.reshape(-1)
        feats[t, J*3:J*3*2] = vel.reshape(-1)
        # extras: distances
        d_hl_hd = np.linalg.norm(centered[L_HAND] - centered[HEAD])
        d_hr_hd = np.linalg.norm(centered[R_HAND] - centered[HEAD])
        d_hl_hip = np.linalg.norm(centered[L_HAND] - centered[HIP])
        d_hr_hip = np.linalg.norm(centered[R_HAND] - centered[HIP])
        d_hl_hr = np.linalg.norm(centered[L_HAND] - centered[R_HAND])
        d_hl_sc = np.linalg.norm(centered[L_HAND] - centered[SHOULDER_CENTER])
        d_hr_sc = np.linalg.norm(centered[R_HAND] - centered[SHOULDER_CENTER])
        # speeds
        s_wl = np.linalg.norm(vel[L_WRIST])
        s_wr = np.linalg.norm(vel[R_WRIST])
        s_hl = np.linalg.norm(vel[L_HAND])
        s_hr = np.linalg.norm(vel[R_HAND])
        # elbow flexion cosines
        def elbow_cos(l_sh, l_el, l_wr):
            u = centered[l_sh] - centered[l_el]
            v = centered[l_wr] - centered[l_el]
            nu = np.linalg.norm(u); nv = np.linalg.norm(v)
            if nu < 1e-6 or nv < 1e-6: return 1.0
            return float(np.dot(u, v) / (nu*nv))
        cos_el = elbow_cos(L_SHOULDER, L_ELBOW, L_WRIST)
        cos_er = elbow_cos(R_SHOULDER, R_ELBOW, R_WRIST)
        extras = np.array([d_hl_hd, d_hr_hd, d_hl_hip, d_hr_hip, d_hl_hr, d_hl_sc, d_hr_sc, s_wl, s_wr, s_hl, s_hr, cos_el, cos_er], dtype=np.float32)
        feats[t, J*3*2:J*3*2+13] = extras
        feats[t, J*3*2+13:] = acc.reshape(-1)  # accelerations
    return feats

# v1.6 cache dirs
CACHE_DIR_V16 = './cache_v16'
os.makedirs(CACHE_DIR_V16, exist_ok=True)

def split_and_tar_for_id(sample_id:int, split_hint:str='train'):
    if split_hint == 'train':
        if sample_id < 100: return 'train', 'training1.tar.gz'
        elif sample_id < 200: return 'train', 'training2.tar.gz'
        else: return 'train', 'training3.tar.gz'
    elif split_hint == 'val':
        if sample_id < 500: return 'val', 'validation1.tar.gz'
        elif sample_id < 621: return 'val', 'validation2.tar.gz'
        else: return 'val', 'validation3.tar.gz'
    elif split_hint == 'test':
        return 'test', 'test.tar.gz'
    else:
        raise ValueError('Unknown split_hint')

def get_zip_member(tf: tarfile.TarFile, sample_id:int):
    candidates = [f'./Sample{sample_id:05d}.zip', f'Sample{sample_id:05d}.zip']
    for nm in candidates:
        try:
            return tf.getmember(nm)
        except KeyError:
            pass
    for m in tf.getmembers():
        if m.name.endswith(f'Sample{sample_id:05d}.zip'):
            return m
    raise FileNotFoundError(f'Sample zip for id {sample_id} not found in tar')

def load_video_from_split(sample_id:int, split_hint:str):
    _, tar_path = split_and_tar_for_id(sample_id, split_hint)
    with tarfile.open(tar_path, 'r:gz') as tf:
        mem = get_zip_member(tf, sample_id)
        zbytes = tf.extractfile(mem).read()
        with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
            mat_name = next(n for n in zf.namelist() if n.lower().endswith('_data.mat'))
            with zf.open(mat_name) as f:
                d = sio.loadmat(f, simplify_cells=True)
    return d['Video']

def stack_world_positions(vid, stride:int=2):
    T = int(vid['NumFrames']); frames = vid['Frames']
    J = frames[0]['Skeleton']['WorldPosition'].shape[0]
    idxs = list(range(0, T, stride))
    skel = np.zeros((len(idxs), J, 3), dtype=np.float32)
    last = None
    for i, t in enumerate(idxs):
        wp = frames[t]['Skeleton']['WorldPosition']
        arr = np.asarray(wp, dtype=np.float32) if wp is not None else last
        if arr is None: arr = np.zeros((J,3), dtype=np.float32)
        skel[i] = arr; last = arr
    return skel, idxs

def true_token_sequence_from_video(vid, name_to_id_map):
    seq = []
    for lab in vid['Labels']:
        cid = name_to_id_map.get(lab['Name'])
        if cid is not None: seq.append(cid)
    return seq

def cache_sample_v16(sample_id:int, split_hint:str, stride:int=2):
    out_path = os.path.join(CACHE_DIR_V16, f'{split_hint}_{sample_id:05d}.npz')
    if os.path.exists(out_path):
        return out_path
    t0 = time.time()
    vid = load_video_from_split(sample_id, split_hint)
    skel, idxs = stack_world_positions(vid, stride=stride)
    X = features_from_skeleton_v16(skel)
    T_full = int(vid['NumFrames'])
    y_full = np.zeros(T_full, dtype=np.int16)
    if split_hint in ('train','val'):
        for lab in vid['Labels']:
            cid = name_to_id.get(lab['Name'])
            if cid is None: continue
            b = max(0, int(lab['Begin'])-1); e = min(T_full, int(lab['End']))
            y_full[b:e] = cid
    y_ds = np.array([y_full[t] for t in idxs], dtype=np.int16)
    true_seq = true_token_sequence_from_video(vid, name_to_id) if split_hint in ('train','val') else []
    fps = int(vid.get('FrameRate', 20))
    np.savez_compressed(out_path, X=X, y=y_ds, seq=np.array(true_seq, dtype=np.int16), fps=fps, stride=stride, sid=sample_id)
    print(f"[CacheV16] {split_hint} id={sample_id} -> {out_path} X={X.shape} saved in {time.time()-t0:.1f}s", flush=True)
    return out_path

def list_ids_in_tar_fast(tar_path):
    ids = []
    with tarfile.open(tar_path, 'r:gz') as tf:
        for m in tf.getmembers():
            if m.name.lower().endswith('.zip'):
                m2 = re.findall(r'(\d{5})', m.name)
                if m2: ids.append(int(m2[0]))
    return sorted(ids)

def remaining_ids_v16(split_hint, ids):
    rem = []
    for sid in ids:
        out_path = os.path.join(CACHE_DIR_V16, f'{split_hint}_{sid:05d}.npz')
        if not os.path.exists(out_path): rem.append(sid)
    return rem

def parallel_cache_v16(split_hint, ids, max_workers=12, stride=2):
    todo = remaining_ids_v16(split_hint, ids)
    print(f"[ParCacheV16] {split_hint}: {len(todo)}/{len(ids)} remaining.")
    if not todo: return
    t0 = time.time(); done = 0
    def task(sid):
        try:
            return cache_sample_v16(sid, split_hint, stride=stride)
        except Exception as e:
            print(f"[ParCacheV16][ERR] {split_hint} id={sid}: {e}")
            return None
    with cf.ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(task, sid) for sid in todo]
        for fut in cf.as_completed(futures):
            _ = fut.result(); done += 1
            if done % 20 == 0:
                print(f"[ParCacheV16] {split_hint}: {done}/{len(todo)} done; elapsed {time.time()-t0:.1f}s")
    print(f"[ParCacheV16] {split_hint}: completed {done} in {time.time()-t0:.1f}s")

def load_cached_split_v16(split_prefix):
    files = sorted(glob.glob(os.path.join(CACHE_DIR_V16, f"{split_prefix}_*.npz")))
    Xs, ys = [], []
    n_frames = 0; t0 = time.time()
    for i, fp in enumerate(files, 1):
        d = np.load(fp, allow_pickle=False)
        X = d['X']; y = d['y']
        Xs.append(X); ys.append(y.astype(np.int32)); n_frames += len(y)
        if i % 20 == 0:
            print(f"[LoadCacheV16] {split_prefix}: {i}/{len(files)} files, cum frames={n_frames}", flush=True)
    X = np.vstack(Xs) if Xs else np.zeros((0,0), dtype=np.float32)
    y = np.concatenate(ys) if ys else np.zeros((0,), dtype=np.int32)
    print(f"[LoadCacheV16] {split_prefix}: X={X.shape} y={y.shape} files={len(files)} elapsed={time.time()-t0:.1f}s", flush=True)
    return X, y, files

def make_weights_with_boundary_erosion(y: np.ndarray, w0: float=0.38):
    w = np.ones_like(y, dtype=np.float32); w[y==0] = w0
    if len(y) > 2:
        bmask = (y[1:-1] != y[:-2]) | (y[1:-1] != y[2:])
        w[1:-1][bmask] = 0.0
    return w

def train_seed_model_v16(X, y, seed:int):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    print(f"[XGB-V16][Seed {seed}] Training...")
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=200)
    print(f"[XGB-V16][Seed {seed}] Done in {time.time()-t0:.1f}s")
    return bst

def ensure_test_cache_v16():
    test_ids = test_df['Id'].tolist()
    todo = []
    for sid in test_ids:
        out_path = os.path.join(CACHE_DIR_V16, f'test_{sid:05d}.npz')
        if not os.path.exists(out_path):
            todo.append(sid)
    if todo:
        print(f"[CacheV16][TEST] {len(todo)}/{len(test_ids)} missing; caching with stride=2...")
        parallel_cache_v16('test', test_ids, max_workers=12, stride=2)
    else:
        print('[CacheV16][TEST] All test cached.')

# Reuse DP + full pipeline + rescue from earlier v15 section
# decode_full_pipeline(P, dp_params, T, smooth_win) and ensure_perm20_preserve_dp must be defined already

def predict_test_full_pipeline_train_only_v16(best_params: dict, seeds=(2025,1337,42,0,123)):
    X_tr, y_tr, _ = load_cached_split_v16('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v16(X_tr, y_tr, seed=s))
    ensure_test_cache_v16()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V16, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = decode_full_pipeline(P, best_params, T=best_params.get('T',1.0), smooth_win=best_params.get('smooth_win',1), alpha_fill=1.5)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-V16] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[RUN V1.6] Parallel cache TRAIN (stride=2) with FE v1.6...')
train_ids_all = train_df['Id'].tolist()
parallel_cache_v16('train', train_ids_all, max_workers=12, stride=2)
print('[RUN V1.6] Training TRAIN-only 5 seeds and inferring TEST with conservative DP (lambda=0.8, min_len=12), T=1, win=1...')
best_params_conservative = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.8, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_full_pipeline_train_only_v16(best_params_conservative, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[RUN V1.6] Parallel cache TRAIN (stride=2) with FE v1.6...
[ParCacheV16] train: 297/297 remaining.


In [46]:
# 37) Log-odds DP emissions (log p(c) - log p(bg)) + DP-preserving rescue; TRAIN-only 5-seed; stride=2; T=1,smooth=1
import numpy as np, os, glob, time, pandas as pd, xgboost as xgb

# Ensure rescue uses DP-preserving path only in any legacy callers
def decode_with_rescue(P, params):
    # Legacy wrapper: always use DP + ensure_perm20_preserve_dp
    Pp = preprocess_probs(P, T=params.get('T',1.0), smooth_win=params.get('smooth_win',1))
    pred = decode_sequence_dp_logodds(Pp,
                                     switch_penalty=params.get('lambda', 0.9),
                                     bg_bias=params.get('bg_bias', 0.25),
                                     merge_gap=params.get('merge_gap', 3),
                                     min_len=params.get('min_len', 12),
                                     mean_thr=params.get('mean_thr', 0.5),
                                     max_thr=params.get('max_thr', 0.65))
    return ensure_perm20_preserve_dp(pred, Pp, alpha=1.6)

def viterbi_labels_logodds(probs: np.ndarray, switch_penalty: float = 1.0, bg_bias: float = 0.25) -> np.ndarray:
    # probs: (T, C=21). Emissions: s_c = log p(c) - log p(bg) for c=1..20; cost = -s_c; background has constant cost=bg_bias.
    T, C = probs.shape
    eps = 1e-9
    logp = np.log(np.clip(probs, eps, 1.0)).astype(np.float32)  # (T,C)
    cost = np.zeros((T, C), dtype=np.float32)
    cost[:, 0] = float(bg_bias)
    cost[:, 1:] = -(logp[:, 1:] - logp[:, [0]])  # -log-odds
    dp = np.zeros((T, C), dtype=np.float32)
    bp = np.zeros((T, C), dtype=np.int16)
    dp[0] = cost[0]
    bp[0] = -1
    for t in range(1, T):
        prev = dp[t-1]
        min_prev = float(prev.min())
        stay = prev
        switch = min_prev + switch_penalty
        best_prev = np.minimum(stay, switch)
        dp[t] = cost[t] + best_prev
        argmin_prev = int(prev.argmin())
        bp[t] = np.where(stay <= switch, np.arange(C, dtype=np.int16), np.int16(argmin_prev))
    path = np.zeros(T, dtype=np.int16)
    path[T-1] = int(dp[T-1].argmin())
    for t in range(T-2, -1, -1):
        path[t] = bp[t+1, path[t+1]]
    return path

def decode_sequence_dp_logodds(probs, switch_penalty=1.0, bg_bias=0.25, merge_gap=3, min_len=12, mean_thr=0.5, max_thr=0.65):
    labels = viterbi_labels_logodds(probs, switch_penalty=switch_penalty, bg_bias=bg_bias)
    T = len(labels)
    segs = []  # (cls, b, e, mean_p, max_p)
    b = 0
    for i in range(1, T+1):
        if i==T or labels[i] != labels[b]:
            cls = int(labels[b])
            p = probs[b:i, cls] if cls < probs.shape[1] else np.zeros(i-b)
            mean_p = float(p.mean()) if (i-b)>0 else 0.0
            max_p = float(p.max()) if (i-b)>0 else 0.0
            segs.append([cls, b, i, mean_p, max_p])
            b = i
    # merge small bg gaps between same class
    merged = []
    i = 0
    while i < len(segs):
        cur = segs[i]
        j = i + 1
        while j < len(segs):
            if segs[j][0] == cur[0] and segs[j-1][0]==0 and (segs[j][1]-cur[2]) <= merge_gap:
                cur[2] = segs[j][2]
                cur[3] = float(np.mean(probs[cur[1]:cur[2], cur[0]]))
                cur[4] = float(np.max(probs[cur[1]:cur[2], cur[0]]))
                j += 1
            else:
                break
        merged.append(cur)
        i = j
    out = []
    for cls, sb, se, mp, xp in merged:
        if cls == 0: continue
        if (se - sb) < min_len: continue
        if not (mp >= mean_thr and xp >= max_thr): continue
        if not out or out[-1] != int(cls):
            out.append(int(cls))
    return out

def predict_test_logodds_train_only(dp_params: dict, seeds=(2025,1337,42,0,123)):
    # TRAIN-only models (v1.5 features), average probs, log-odds DP decode + DP-preserving rescue
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        Pp = preprocess_probs(P, T=1.0, smooth_win=1)
        pred = decode_sequence_dp_logodds(Pp,
                                         switch_penalty=dp_params.get('lambda', 0.9),
                                         bg_bias=dp_params.get('bg_bias', 0.25),
                                         merge_gap=dp_params.get('merge_gap', 3),
                                         min_len=dp_params.get('min_len', 12),
                                         mean_thr=dp_params.get('mean_thr', 0.5),
                                         max_thr=dp_params.get('max_thr', 0.65))
        seq = ensure_perm20_preserve_dp(pred, Pp, alpha=1.6)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-LogOdds] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[LogOddsDP] TRAIN-only 5-seed; stride=2; T=1, smooth=1; lambda=0.9, bg_bias=0.25, merge_gap=3, min_len=12, mean_thr=0.5, max_thr=0.65')
dp_params_logodds = {'lambda': 0.9, 'bg_bias': 0.25, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_logodds_train_only(dp_params_logodds, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

In [52]:
# 38) Log-odds DP quick run: 3 seeds, stride=2, T=1, smooth=1, lambda=0.9, bg_bias=0.25
import numpy as np, os, time, pandas as pd, xgboost as xgb

def viterbi_labels_logodds_quick(probs: np.ndarray, switch_penalty: float = 1.0, bg_bias: float = 0.25) -> np.ndarray:
    T, C = probs.shape
    eps = 1e-9
    logp = np.log(np.clip(probs, eps, 1.0)).astype(np.float32)
    cost = np.zeros((T, C), dtype=np.float32)
    cost[:, 0] = float(bg_bias)
    cost[:, 1:] = -(logp[:, 1:] - logp[:, [0]])
    dp = np.zeros((T, C), dtype=np.float32)
    bp = np.zeros((T, C), dtype=np.int16)
    dp[0] = cost[0]; bp[0] = -1
    for t in range(1, T):
        prev = dp[t-1]
        mprev = float(prev.min())
        stay = prev
        switch = mprev + switch_penalty
        best_prev = np.minimum(stay, switch)
        dp[t] = cost[t] + best_prev
        argmin_prev = int(prev.argmin())
        bp[t] = np.where(stay <= switch, np.arange(C, dtype=np.int16), np.int16(argmin_prev))
    path = np.zeros(T, dtype=np.int16)
    path[T-1] = int(dp[T-1].argmin())
    for t in range(T-2, -1, -1):
        path[t] = bp[t+1, path[t+1]]
    return path

def decode_sequence_dp_logodds_quick(probs, switch_penalty=0.9, bg_bias=0.25, merge_gap=3, min_len=12, mean_thr=0.5, max_thr=0.65):
    labels = viterbi_labels_logodds_quick(probs, switch_penalty=switch_penalty, bg_bias=bg_bias)
    T = len(labels)
    segs = []; b = 0
    for i in range(1, T+1):
        if i==T or labels[i] != labels[b]:
            cls = int(labels[b])
            p = probs[b:i, cls] if cls < probs.shape[1] else np.zeros(i-b)
            mp = float(p.mean()) if (i-b)>0 else 0.0
            xp = float(p.max()) if (i-b)>0 else 0.0
            segs.append([cls, b, i, mp, xp]); b = i
    merged = []; i = 0
    while i < len(segs):
        cur = segs[i]; j = i + 1
        while j < len(segs):
            if segs[j][0] == cur[0] and segs[j-1][0]==0 and (segs[j][1]-cur[2]) <= merge_gap:
                cur[2] = segs[j][2]
                cur[3] = float(np.mean(probs[cur[1]:cur[2], cur[0]]))
                cur[4] = float(np.max(probs[cur[1]:cur[2], cur[0]]))
                j += 1
            else:
                break
        merged.append(cur); i = j
    out = []
    for cls, sb, se, mp, xp in merged:
        if cls == 0: continue
        if (se - sb) < min_len: continue
        if not (mp >= mean_thr and xp >= max_thr): continue
        if not out or out[-1] != int(cls):
            out.append(int(cls))
    return out

def predict_test_logodds_3seeds(dp_params: dict, seeds=(2025,1337,42)):
    print('[LogOddsDP-Quick] Loading TRAIN cache...', flush=True)
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        print(f'[XGB-V15][Seed {s}] Training...', flush=True)
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        Pp = preprocess_probs(P, T=1.0, smooth_win=1)
        pred = decode_sequence_dp_logodds_quick(Pp,
                                              switch_penalty=dp_params.get('lambda', 0.9),
                                              bg_bias=dp_params.get('bg_bias', 0.25),
                                              merge_gap=dp_params.get('merge_gap', 3),
                                              min_len=dp_params.get('min_len', 12),
                                              mean_thr=dp_params.get('mean_thr', 0.5),
                                              max_thr=dp_params.get('max_thr', 0.65))
        seq = ensure_perm20_preserve_dp(pred, Pp, alpha=1.6)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-LogOdds-Quick] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[Run] Log-odds DP quick infer: 3 seeds, lambda=0.9, bg_bias=0.25, min_len=12, merge_gap=3, T=1, win=1', flush=True)
dpq = {'lambda': 0.9, 'bg_bias': 0.25, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_logodds_3seeds(dpq, seeds=(2025,1337,42))
print('Submission written to:', sub_path)

[Run] Log-odds DP quick infer: 3 seeds, lambda=0.9, bg_bias=0.25, min_len=12, merge_gap=3, T=1, win=1


[LogOddsDP-Quick] Loading TRAIN cache...


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


...

[LoadV15] train: 260/297 files, cum frames=164808


[LoadV15] train: 280/297 files, cum frames=177261


[LoadV15] train: X=(187296, 120) y=(187296,) files=297


[XGB-V15][Seed 2025] Training...


[XGB-V15][Seed 2025] Training...


In [48]:
# 39) Log-odds DP + per-class min_len (p20 clip [6,14]); TRAIN-only 5-seed; stride=2; T=1,smooth=1
import numpy as np, glob, os, time, pandas as pd, xgboost as xgb

def compute_duration_priors_p20_clip614_from_train_cache_v15():
    files = sorted(glob.glob(os.path.join(CACHE_DIR_V15, 'train_*.npz')))
    lens = {c: [] for c in range(1,21)}
    for i, fp in enumerate(files, 1):
        d = np.load(fp, allow_pickle=False)
        y = d['y'].astype(np.int32)
        if y.size == 0: continue
        b = 0
        for t in range(1, len(y)+1):
            if t==len(y) or y[t] != y[b]:
                cls = int(y[b])
                if cls != 0:
                    lens[cls].append(t-b)
                b = t
        if i % 25 == 0:
            print(f"[DurPrior-20] processed {i}/{len(files)} train files", flush=True)
    min_len_map = {}
    for c in range(1,21):
        arr = np.array(lens[c], dtype=np.float32)
        if arr.size == 0:
            ml = 8
        else:
            ml = float(np.percentile(arr, 20.0))
        ml = int(max(6, min(14, round(ml))))
        min_len_map[c] = ml
    print('[DurPrior-20] per-class min_len (clipped [6,14]):', min_len_map)
    return min_len_map

def decode_sequence_dp_logodds_pc(probs, switch_penalty=0.9, bg_bias=0.25, merge_gap=3, min_len=12, mean_thr=0.5, max_thr=0.65, min_len_map=None):
    labels = viterbi_labels_logodds_quick(probs, switch_penalty=switch_penalty, bg_bias=bg_bias)
    T = len(labels)
    segs = []; b = 0
    for i in range(1, T+1):
        if i==T or labels[i] != labels[b]:
            cls = int(labels[b])
            p = probs[b:i, cls] if cls < probs.shape[1] else np.zeros(i-b)
            mp = float(p.mean()) if (i-b)>0 else 0.0
            xp = float(p.max()) if (i-b)>0 else 0.0
            segs.append([cls, b, i, mp, xp]); b = i
    # merge bg gaps
    merged = []; i = 0
    while i < len(segs):
        cur = segs[i]; j = i + 1
        while j < len(segs):
            if segs[j][0] == cur[0] and segs[j-1][0]==0 and (segs[j][1]-cur[2]) <= merge_gap:
                cur[2] = segs[j][2]
                cur[3] = float(np.mean(probs[cur[1]:cur[2], cur[0]]))
                cur[4] = float(np.max(probs[cur[1]:cur[2], cur[0]]))
                j += 1
            else:
                break
        merged.append(cur); i = j
    out = []
    for cls, sb, se, mp, xp in merged:
        if cls == 0: continue
        req_len = int(min_len_map.get(int(cls), min_len)) if min_len_map is not None else int(min_len)
        if (se - sb) < req_len: continue
        if not (mp >= mean_thr and xp >= max_thr): continue
        if not out or out[-1] != int(cls):
            out.append(int(cls))
    return out

def predict_test_logodds_pc_train_only(dp_params: dict, min_len_map: dict, seeds=(2025,1337,42,0,123)):
    print('[LogOddsDP-PC] Loading TRAIN cache...', flush=True)
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        print(f'[XGB-V15][Seed {s}] Training...', flush=True)
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        Pp = preprocess_probs(P, T=1.0, smooth_win=1)
        pred = decode_sequence_dp_logodds_pc(Pp,
                                            switch_penalty=dp_params.get('lambda', 0.9),
                                            bg_bias=dp_params.get('bg_bias', 0.25),
                                            merge_gap=dp_params.get('merge_gap', 3),
                                            min_len=dp_params.get('min_len', 12),
                                            mean_thr=dp_params.get('mean_thr', 0.5),
                                            max_thr=dp_params.get('max_thr', 0.65),
                                            min_len_map=min_len_map)
        seq = ensure_perm20_preserve_dp(pred, Pp, alpha=1.6)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-LogOdds-PC] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[Run] Log-odds DP + per-class duration: p20 clip [6,14]; 5 seeds; lambda=0.9, bg_bias=0.25, min_len=12, merge_gap=3, T=1, win=1', flush=True)
min_map = compute_duration_priors_p20_clip614_from_train_cache_v15()
dp_params_lo = {'lambda': 0.9, 'bg_bias': 0.25, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_logodds_pc_train_only(dp_params_lo, min_len_map=min_map, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[Run] Log-odds DP + per-class duration: p20 clip [6,14]; 5 seeds; lambda=0.9, bg_bias=0.25, min_len=12, merge_gap=3, T=1, win=1


[DurPrior-20] processed 25/297 train files


[DurPrior-20] processed 50/297 train files


[DurPrior-20] processed 75/297 train files


[DurPrior-20] processed 100/297 train files


...

[TEST-LogOdds-PC] 60/95 done; elapsed 91.6s


[TEST-LogOdds-PC] 70/95 done; elapsed 106.8s


[TEST-LogOdds-PC] 80/95 done; elapsed 121.4s


[TEST-LogOdds-PC] 90/95 done; elapsed 136.0s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [49]:
# 40) Log-odds DP quick run variant: 3 seeds, lambda=1.0, bg_bias=0.30, min_len=10 (stride=2, T=1, smooth=1)
print('[Run] Log-odds DP quick infer VAR: 3 seeds, lambda=1.0, bg_bias=0.30, min_len=10, merge_gap=3, T=1, win=1', flush=True)
dpq_var = {'lambda': 1.0, 'bg_bias': 0.30, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_logodds_3seeds(dpq_var, seeds=(2025,1337,42))
print('Submission written to:', sub_path)

[Run] Log-odds DP quick infer VAR: 3 seeds, lambda=1.0, bg_bias=0.30, min_len=10, merge_gap=3, T=1, win=1


[LogOddsDP-Quick] Loading TRAIN cache...


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


...

[TEST-LogOdds-Quick] 60/95 done; elapsed 59.7s


[TEST-LogOdds-Quick] 70/95 done; elapsed 69.8s


[TEST-LogOdds-Quick] 80/95 done; elapsed 79.4s


[TEST-LogOdds-Quick] 90/95 done; elapsed 88.9s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [50]:
# 41) Stride-2 TTA {0,1} + Log-odds DP decode with DP-preserving rescue; TRAIN-only 5 seeds; T=1, win=1
import os, time, numpy as np, pandas as pd, xgboost as xgb

def predict_test_stride2_tta_logodds(seeds=(2025,1337,42,0,123), dp_params=None):
    if dp_params is None:
        dp_params = {'lambda': 0.9, 'bg_bias': 0.25, 'merge_gap': 3, 'min_len': 12, 'mean_thr': 0.5, 'max_thr': 0.65}
    # Train on TRAIN-only v15 cache
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    # Ensure both test caches exist: stride=2 start=0 already in cache_v15; start=1 exists in CACHE_DIR_V15_S2_O1
    ensure_test_cache_v15()
    # If offset=1 cache missing, build it quickly
    if not os.path.isdir('./cache_v15_s2_o1') or len([f for f in os.listdir('./cache_v15_s2_o1') if f.endswith('.npz')]) < len(test_df):
        print('[TTA-LogOdds] Building stride=2 offset=1 cache...')
        cache_test_stride2_offset1()
    rows = []; t0 = time.time()
    test_ids = test_df['Id'].tolist()
    for i, sid in enumerate(test_ids, 1):
        d0 = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        d1 = np.load(os.path.join(CACHE_DIR_V15_S2_O1, f'test_{sid:05d}.npz'), allow_pickle=False)
        X0, X1 = d0['X'], d1['X']
        dm0 = xgb.DMatrix(X0); dm1 = xgb.DMatrix(X1)
        Ps0 = [m.predict(dm0) for m in models]
        Ps1 = [m.predict(dm1) for m in models]
        P0 = np.mean(np.stack(Ps0, axis=0), axis=0)
        P1 = np.mean(np.stack(Ps1, axis=0), axis=0)
        N = max(P0.shape[0], P1.shape[0])
        P0i = interp_probs(P0, N)
        P1i = interp_probs(P1, N)
        P = 0.5*(P0i + P1i)
        Pp = preprocess_probs(P, T=1.0, smooth_win=1)
        pred = decode_sequence_dp_logodds_quick(Pp,
                                              switch_penalty=dp_params.get('lambda', 0.9),
                                              bg_bias=dp_params.get('bg_bias', 0.25),
                                              merge_gap=dp_params.get('merge_gap', 3),
                                              min_len=dp_params.get('min_len', 12),
                                              mean_thr=dp_params.get('mean_thr', 0.5),
                                              max_thr=dp_params.get('max_thr', 0.65))
        seq = ensure_perm20_preserve_dp(pred, Pp, alpha=1.6)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-s2-TTA-LogOdds] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[Run] Stride-2 TTA {0,1} + Log-odds DP; TRAIN-only 5 seeds; lambda=0.9, bg_bias=0.25, min_len=12, T=1, win=1')
sub_path = predict_test_stride2_tta_logodds(seeds=(2025,1337,42,0,123), dp_params={'lambda':0.9,'bg_bias':0.25,'merge_gap':3,'min_len':12,'mean_thr':0.5,'max_thr':0.65})
print('Submission written to:', sub_path)

[Run] Stride-2 TTA {0,1} + Log-odds DP; TRAIN-only 5 seeds; lambda=0.9, bg_bias=0.25, min_len=12, T=1, win=1


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


...

[TEST-s2-TTA-LogOdds] 60/95 done; elapsed 172.2s


[TEST-s2-TTA-LogOdds] 70/95 done; elapsed 200.7s


[TEST-s2-TTA-LogOdds] 80/95 done; elapsed 228.4s


[TEST-s2-TTA-LogOdds] 90/95 done; elapsed 255.7s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [51]:
# 42) OOF grid for Log-odds DP params, then TRAIN-only 5-seed infer with best params (T=1, smooth=1)
import os, glob, time, numpy as np, pandas as pd, xgboost as xgb

def load_all_oof_items_v15():
    files = sorted(glob.glob(os.path.join(OOF_DIR_V15, 'oof_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        items.append((int(d['sid']), d['probs'], d['seq']))
    return items

def grid_search_logodds_on_oof():
    items = load_all_oof_items_v15()
    assert items, 'No OOF files found. Build OOF first.'
    lambdas = [0.8, 0.9, 1.0]
    bg_biases = [0.20, 0.25, 0.30]
    minlens = [10, 12]
    merge_gap = 3; mean_thr = 0.50; max_thr = 0.65
    best = (1e9, None); tried = 0; t0 = time.time()
    for lam in lambdas:
        for bb in bg_biases:
            for ml in minlens:
                scores = []
                for sid, P, seq in items:
                    Pp = preprocess_probs(P, T=1.0, smooth_win=1)
                    pred = decode_sequence_dp_logodds_quick(Pp,
                        switch_penalty=lam, bg_bias=bb, merge_gap=merge_gap,
                        min_len=ml, mean_thr=mean_thr, max_thr=max_thr)
                    pred = ensure_perm20_preserve_dp(pred, Pp, alpha=1.6)
                    lev = levenshtein(list(pred), list(seq.tolist()))
                    scores.append(lev/20.0)
                mean_norm = float(np.mean(scores)) if scores else 1.0
                tried += 1
                if tried % 5 == 0:
                    print(f"[Grid-OOF-LogOdds] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
                if mean_norm < best[0]:
                    best = (mean_norm, {'lambda': lam, 'bg_bias': bb, 'merge_gap': merge_gap, 'min_len': ml, 'mean_thr': mean_thr, 'max_thr': max_thr})
    print(f"[Grid-OOF-LogOdds] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}")
    return best

def predict_test_logodds_train_only_best(dp_params: dict, seeds=(2025,1337,42,0,123)):
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        print(f"[XGB-V15][Seed {s}] Training...", flush=True)
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        Pp = preprocess_probs(P, T=1.0, smooth_win=1)
        pred = decode_sequence_dp_logodds_quick(Pp,
                  switch_penalty=dp_params.get('lambda', 0.9),
                  bg_bias=dp_params.get('bg_bias', 0.25),
                  merge_gap=dp_params.get('merge_gap', 3),
                  min_len=dp_params.get('min_len', 12),
                  mean_thr=dp_params.get('mean_thr', 0.5),
                  max_thr=dp_params.get('max_thr', 0.65))
        seq = ensure_perm20_preserve_dp(pred, Pp, alpha=1.6)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-LogOdds-Best] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[Run] OOF grid for Log-odds DP (T=1, smooth=1); then TRAIN-only 5-seed infer with best params')
best_lo = grid_search_logodds_on_oof()
print('Best log-odds params:', best_lo)
sub_path = predict_test_logodds_train_only_best(best_lo[1], seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[Run] OOF grid for Log-odds DP (T=1, smooth=1); then TRAIN-only 5-seed infer with best params


[Grid-OOF-LogOdds] 5 combos, curr mean=0.3000 best=0.2985


[Grid-OOF-LogOdds] 10 combos, curr mean=0.3261 best=0.2963


[Grid-OOF-LogOdds] 15 combos, curr mean=0.2961 best=0.2949


[Grid-OOF-LogOdds] Done 18 combos in 741.5s. Best=(0.2949494949494949, {'lambda': 1.0, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.5, 'max_thr': 0.65})
Best log-odds params: (0.2949494949494949, {'lambda': 1.0, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.5, 'max_thr': 0.65})


...

[TEST-LogOdds-Best] 60/95 done; elapsed 80.1s


[TEST-LogOdds-Best] 70/95 done; elapsed 93.4s


[TEST-LogOdds-Best] 80/95 done; elapsed 106.4s


[TEST-LogOdds-Best] 90/95 done; elapsed 119.1s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [54]:
# 43) Stride-2 TTA {0,1} + Standard DP with OOF-best params (lambda=0.6, min_len=6); TRAIN-only 5 seeds
import os, time, numpy as np, pandas as pd, xgboost as xgb
print('[Stride2-TTA OOFDP] TRAIN-only 5-seed; TEST stride=2 offsets {0,1}; align+avg probs; OOF-best DP decode...', flush=True)
dp_oof_best = {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}

# Train on TRAIN-only v1.5 cache
X_tr, y_tr, _ = load_frames_from_cache_v15('train')
models = []
for s in (2025,1337,42,0,123):
    models.append(train_seed_model_v15(X_tr, y_tr, seed=s))

# Ensure both test caches exist: stride=2 start=0 already in cache_v15; start=1 in CACHE_DIR_V15_S2_O1
ensure_test_cache_v15()
cache_test_stride2_offset1()

rows = []; t0 = time.time()
test_ids = test_df['Id'].tolist()
for i, sid in enumerate(test_ids, 1):
    d0 = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
    d1 = np.load(os.path.join(CACHE_DIR_V15_S2_O1, f'test_{sid:05d}.npz'), allow_pickle=False)
    X0, X1 = d0['X'], d1['X']
    dm0 = xgb.DMatrix(X0); dm1 = xgb.DMatrix(X1)
    Ps0 = [m.predict(dm0) for m in models]
    Ps1 = [m.predict(dm1) for m in models]
    P0 = np.mean(np.stack(Ps0, axis=0), axis=0)
    P1 = np.mean(np.stack(Ps1, axis=0), axis=0)
    N = max(P0.shape[0], P1.shape[0])
    P0i = interp_probs(P0, N)
    P1i = interp_probs(P1, N)
    P = 0.5*(P0i + P1i)
    # OOF-aligned hygiene: T=1.0, smooth_win=1, standard DP params (dp_oof_best) and DP-preserving rescue
    seq = decode_full_pipeline(P, {'lambda': dp_oof_best['lambda'],
                                   'bg_bias': dp_oof_best['bg_bias'],
                                   'merge_gap': dp_oof_best['merge_gap'],
                                   'min_len': dp_oof_best['min_len'],
                                   'mean_thr': dp_oof_best['mean_thr'],
                                   'max_thr': dp_oof_best['max_thr']},
                               T=1.0, smooth_win=1, alpha_fill=1.5)
    rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
    if i % 10 == 0:
        print(f"[TEST-s2-TTA-OOFDP] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)

sub = pd.DataFrame(rows, columns=['Id','Sequence'])
sub.to_csv('submission.csv', index=False)
print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
print('Submission written to: submission.csv')

[Stride2-TTA OOFDP] TRAIN-only 5-seed; TEST stride=2 offsets {0,1}; align+avg probs; OOF-best DP decode...


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


...

[TEST-s2-TTA-OOFDP] 60/95 done; elapsed 160.2s


[TEST-s2-TTA-OOFDP] 70/95 done; elapsed 186.9s


[TEST-s2-TTA-OOFDP] 80/95 done; elapsed 212.8s


[TEST-s2-TTA-OOFDP] 90/95 done; elapsed 238.3s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [55]:
# 44) Blend v1.5 + v1.6 per-frame probs (0.6/0.4) + standard DP OOF-best; TRAIN-only 3+3 seeds; stride=2
import os, time, numpy as np, pandas as pd, xgboost as xgb
print('[Blend v15+v16] TRAIN-only 3+3 seeds; stride=2; weights v15=0.6 v16=0.4; OOF-best DP decode (T=1, win=1)...', flush=True)
dp_oof_best = {'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 6, 'mean_thr': 0.5, 'max_thr': 0.65}

# Train v1.5 models on TRAIN-only
X15, y15, _ = load_frames_from_cache_v15('train')
models15 = []
for s in (2025,1337,42):
    models15.append(train_seed_model_v15(X15, y15, seed=s))

# Train v1.6 models on TRAIN-only
X16, y16, _ = load_cached_split_v16('train')
def train_seed_model_v16_quick(X, y, seed:int):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    print(f"[XGB-V16][Seed {seed}] Training...", flush=True)
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=200)
    print(f"[XGB-V16][Seed {seed}] Done in {time.time()-t0:.1f}s")
    return bst
models16 = []
for s in (2025,1337,42):
    models16.append(train_seed_model_v16_quick(X16, y16, seed=s))

# Ensure TEST caches for both v1.5 and v1.6
ensure_test_cache_v15()
ensure_test_cache_v16()

rows = []; t0 = time.time()
test_ids = test_df['Id'].tolist()
for i, sid in enumerate(test_ids, 1):
    # v1.5
    d15 = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
    X15t = d15['X']; dm15 = xgb.DMatrix(X15t)
    P15s = [m.predict(dm15) for m in models15]
    P15 = np.mean(np.stack(P15s, axis=0), axis=0).astype(np.float32)
    # v1.6
    d16 = np.load(os.path.join(CACHE_DIR_V16, f'test_{sid:05d}.npz'), allow_pickle=False)
    X16t = d16['X']; dm16 = xgb.DMatrix(X16t)
    P16s = [m.predict(dm16) for m in models16]
    P16 = np.mean(np.stack(P16s, axis=0), axis=0).astype(np.float32)
    # Align lengths (both stride=2, but be safe)
    N = max(P15.shape[0], P16.shape[0])
    P15i = interp_probs(P15, N)
    P16i = interp_probs(P16, N)
    P = 0.6*P15i + 0.4*P16i
    # Decode with OOF-best standard DP, no temp/smoothing; DP-preserving rescue
    seq = decode_full_pipeline(P, dp_oof_best, T=1.0, smooth_win=1, alpha_fill=1.5)
    rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
    if i % 10 == 0:
        print(f"[TEST-Blend15+16] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)

sub = pd.DataFrame(rows, columns=['Id','Sequence'])
sub.to_csv('submission.csv', index=False)
print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
print('Submission written to: submission.csv')

[Blend v15+v16] TRAIN-only 3+3 seeds; stride=2; weights v15=0.6 v16=0.4; OOF-best DP decode (T=1, win=1)...


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


...

[TEST-Blend15+16] 60/95 done; elapsed 96.3s


[TEST-Blend15+16] 70/95 done; elapsed 112.3s


[TEST-Blend15+16] 80/95 done; elapsed 127.9s


[TEST-Blend15+16] 90/95 done; elapsed 143.1s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [56]:
# 45) Medal push: Standard DP with light smoothing and mid min_len (no TTA), TRAIN-only 5 seeds
print('[MedalAttempt] TRAIN-only 5-seed; v1.5 stride=2; T=1.0, smooth_win=3; DP lambda=0.7, min_len=9; DP-preserving rescue', flush=True)
best_params_goldilocks = {'T': 1.0, 'smooth_win': 3, 'lambda': 0.7, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 9, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_full_pipeline_train_only(best_params_goldilocks, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[MedalAttempt] TRAIN-only 5-seed; v1.5 stride=2; T=1.0, smooth_win=3; DP lambda=0.7, min_len=9; DP-preserving rescue


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


...

[TEST-Full] 60/95 done; elapsed 79.7s


[TEST-Full] 70/95 done; elapsed 93.1s


[TEST-Full] 80/95 done; elapsed 106.1s


[TEST-Full] 90/95 done; elapsed 118.8s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [ ]:
# 46) Blend v1.5 (5 seeds) + v1.6 (5 seeds) probs (0.5/0.5) + Goldilocks DP (lambda=0.7, min_len=9, smooth_win=3); TRAIN-only; stride=2
import os, time, numpy as np, pandas as pd, xgboost as xgb
print('[Blend v15+v16 x10] TRAIN-only 5+5 seeds; stride=2; weights v15=0.5 v16=0.5; Goldilocks DP (T=1, win=3, lambda=0.7, min_len=9)...', flush=True)

best_params_goldilocks = {'T': 1.0, 'smooth_win': 3, 'lambda': 0.7, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 9, 'mean_thr': 0.5, 'max_thr': 0.65}

# Train v1.5 models on TRAIN-only (5 seeds)
X15, y15, _ = load_frames_from_cache_v15('train')
models15 = []
for s in (2025,1337,42,0,123):
    models15.append(train_seed_model_v15(X15, y15, seed=s))

# Train v1.6 models on TRAIN-only (5 seeds)
X16, y16, _ = load_cached_split_v16('train')
models16 = []
for s in (2025,1337,42,0,123):
    models16.append(train_seed_model_v16(X16, y16, seed=s))

# Ensure TEST caches for both v1.5 and v1.6
ensure_test_cache_v15()
ensure_test_cache_v16()

rows = []; t0 = time.time()
test_ids = test_df['Id'].tolist()
for i, sid in enumerate(test_ids, 1):
    # v1.5
    d15 = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
    X15t = d15['X']; dm15 = xgb.DMatrix(X15t)
    P15s = [m.predict(dm15) for m in models15]
    P15 = np.mean(np.stack(P15s, axis=0), axis=0).astype(np.float32)
    # v1.6
    d16 = np.load(os.path.join(CACHE_DIR_V16, f'test_{sid:05d}.npz'), allow_pickle=False)
    X16t = d16['X']; dm16 = xgb.DMatrix(X16t)
    P16s = [m.predict(dm16) for m in models16]
    P16 = np.mean(np.stack(P16s, axis=0), axis=0).astype(np.float32)
    # Align lengths (both stride=2, but be safe)
    N = max(P15.shape[0], P16.shape[0])
    P15i = interp_probs(P15, N)
    P16i = interp_probs(P16, N)
    P = 0.5*P15i + 0.5*P16i
    # Decode with Goldilocks DP, T=1, smooth_win=3, DP-preserving rescue
    seq = decode_full_pipeline(P, best_params_goldilocks, T=best_params_goldilocks['T'], smooth_win=best_params_goldilocks['smooth_win'], alpha_fill=1.5)
    rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
    if i % 10 == 0:
        print(f"[TEST-Blend15+16 x10] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)

sub = pd.DataFrame(rows, columns=['Id','Sequence'])
sub.to_csv('submission.csv', index=False)
print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
print('Submission written to: submission.csv')

In [57]:
# 47) OOF grid: standard DP with smooth_win=3 (T=1), tune lambda/min_len; then TRAIN-only 5-seed infer
import os, glob, time, numpy as np, pandas as pd, xgboost as xgb

def load_all_oof_items_v15():
    files = sorted(glob.glob(os.path.join(OOF_DIR_V15, 'oof_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        items.append((int(d['sid']), d['probs'], d['seq']))
    return items

def grid_search_std_dp_oof_smooth3():
    items = load_all_oof_items_v15()
    assert items, 'No OOF files found. Build OOF first.'
    lambdas = [0.6, 0.7, 0.8]
    minlens = [8, 9, 10]
    bg_bias = 0.2; merge_gap = 3; mean_thr = 0.50; max_thr = 0.65
    best = (1e9, None); tried = 0; t0 = time.time()
    for lam in lambdas:
        for ml in minlens:
            scores = []
            for sid, P, seq in items:
                Pp = preprocess_probs(P, T=1.0, smooth_win=3)
                pred = decode_sequence_dp(Pp,
                                         switch_penalty=lam, bg_bias=bg_bias,
                                         merge_gap=merge_gap, min_len=ml,
                                         mean_thr=mean_thr, max_thr=max_thr)
                pred = ensure_perm20_preserve_dp(pred, Pp, alpha=1.5)
                lev = levenshtein(list(pred), list(seq.tolist()))
                scores.append(lev/20.0)
            mean_norm = float(np.mean(scores)) if scores else 1.0
            tried += 1
            if tried % 3 == 0:
                print(f"[Grid-OOF-StdDP-S3] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
            if mean_norm < best[0]:
                best = (mean_norm, {'T':1.0,'smooth_win':3,'lambda':lam,'bg_bias':bg_bias,'merge_gap':merge_gap,'min_len':ml,'mean_thr':mean_thr,'max_thr':max_thr})
    print(f"[Grid-OOF-StdDP-S3] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}")
    return best

def predict_test_full_pipeline_train_only_with_params(best_params: dict, seeds=(2025,1337,42,0,123)):
    # Train TRAIN-only v1.5 models, average probs, apply full pipeline (T=1, smooth=3) and write submission.csv
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    test_ids = test_df['Id'].tolist()
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        seq = decode_full_pipeline(P, best_params, T=best_params['T'], smooth_win=best_params['smooth_win'], alpha_fill=1.5)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-StdDP-S3] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[Run] OOF grid for Standard DP with T=1, smooth_win=3; then TRAIN-only 5-seed infer with best params', flush=True)
best_std_s3 = grid_search_std_dp_oof_smooth3()
print('Best std-DP smooth3 params:', best_std_s3)
sub_path = predict_test_full_pipeline_train_only_with_params(best_std_s3[1], seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[Run] OOF grid for Standard DP with T=1, smooth_win=3; then TRAIN-only 5-seed infer with best params


[Grid-OOF-StdDP-S3] 3 combos, curr mean=0.2901 best=0.2732


[Grid-OOF-StdDP-S3] 6 combos, curr mean=0.2904 best=0.2732


[Grid-OOF-StdDP-S3] 9 combos, curr mean=0.2902 best=0.2732


[Grid-OOF-StdDP-S3] Done 9 combos in 16.8s. Best=(0.27323232323232327, {'T': 1.0, 'smooth_win': 3, 'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 8, 'mean_thr': 0.5, 'max_thr': 0.65})
Best std-DP smooth3 params: (0.27323232323232327, {'T': 1.0, 'smooth_win': 3, 'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 8, 'mean_thr': 0.5, 'max_thr': 0.65})
[LoadV15] train: 20/297 files, cum frames=12055


...

[TEST-StdDP-S3] 60/95 done; elapsed 79.6s


[TEST-StdDP-S3] 70/95 done; elapsed 92.8s


[TEST-StdDP-S3] 80/95 done; elapsed 105.7s


[TEST-StdDP-S3] 90/95 done; elapsed 118.3s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [58]:
# 48) Order-conditioned segmentation DP: infer per-sample order via centroids, then segment into 20 gestures in that order
import os, time, numpy as np, pandas as pd, xgboost as xgb

def compute_order_by_centroid(P: np.ndarray, alpha: float = 1.6, smooth_win: int = 3):
    # Smooth and compute temporal centroids mu for classes 1..20; return order (list of classes) ascending by mu
    P2 = preprocess_probs(P, T=1.0, smooth_win=smooth_win)
    Tn, C = P2.shape
    t = np.arange(Tn, dtype=np.float32)
    W = np.power(P2[:, 1:], alpha)  # (T,20) exclude bg
    denom = W.sum(axis=0) + 1e-9
    mu = (W * t[:, None]).sum(axis=0) / denom  # (20,)
    # Tie-breakers for stability: mass desc, peak desc
    mass = P2[:, 1:].sum(axis=0)
    peak = P2[:, 1:].max(axis=0)
    # Stable ranking: primarily by mu asc; for ties, larger mass then peak earlier
    idx_mu = np.argsort(mu, kind='mergesort')
    ranks_mu = np.empty_like(idx_mu); ranks_mu[idx_mu] = np.arange(20)
    ranks_mass = np.empty_like(idx_mu); ranks_mass[np.argsort(-mass, kind='mergesort')] = np.arange(20)
    ranks_peak = np.empty_like(idx_mu); ranks_peak[np.argsort(-peak, kind='mergesort')] = np.arange(20)
    score = ranks_mu + 1e-3*ranks_mass + 1e-6*ranks_peak
    order_idx = np.argsort(score, kind='mergesort')
    order_classes = [int(i+1) for i in order_idx]
    return order_classes, P2

def segment_dp_fixed_order_logodds(P: np.ndarray, order_classes: list, min_len: int = 9):
    # P: (T,21) probs, 0=bg; order_classes: length-20 permutation of 1..20
    # Build log-odds scores s(t,c) = log p(c) - log p(bg) for c in order_classes; DP to cut 20 segments maximizing sum
    eps = 1e-9
    Tn, C = P.shape
    logp = np.log(np.clip(P, eps, 1.0)).astype(np.float32)
    # s: (T,20)
    s = logp[:, np.array(order_classes, dtype=np.int32)] - logp[:, [0]]
    # prefix sums per class for O(1) segment score
    cs = np.zeros((Tn+1, 20), dtype=np.float32)
    np.cumsum(s, axis=0, out=cs[1:])
    mins = np.full(20, int(min_len), dtype=np.int32)
    # DP over classes i=0..19; DP_prev[t] best up to t (end at t) placing first i classes
    DP_prev = np.full(Tn+1, -1e18, dtype=np.float32); DP_prev[0] = 0.0
    backp = []  # list of arrays bp_i[t] start u for segment i ending at t
    cum_min = 0
    for i in range(20):
        mlen = int(mins[i])
        DP_cur = np.full(Tn+1, -1e18, dtype=np.float32)
        bp = np.full(Tn+1, -1, dtype=np.int32)
        cum_min += mlen
        # Maintain running best M[u] = DP_prev[u] - cs[u, i]
        M_val = -1e18; M_idx = -1
        for t_end in range(mlen, Tn+1):
            u = t_end - mlen
            cand = DP_prev[u] - cs[u, i]
            if cand > M_val:
                M_val = cand; M_idx = u
            DP_cur[t_end] = cs[t_end, i] + M_val
            bp[t_end] = M_idx
        backp.append(bp)
        DP_prev = DP_cur
    # Best end time for class 20
    t_end = int(np.argmax(DP_prev))
    # Backtrack cuts
    cuts = [0]*(21)
    cuts[20] = t_end
    for i in range(19, -1, -1):
        u = int(backp[i][cuts[i+1]])
        if u < 0:
            u = max(0, cuts[i+1] - int(mins[i]))
        cuts[i] = u
    # Return the 20 classes in this optimized order (already order_classes), as the final sequence
    return order_classes

def predict_test_order_segment_train_only(seeds=(2025,1337,42,0,123), alpha: float = 1.6, smooth_win: int = 3, min_len: int = 9):
    # Train TRAIN-only v1.5 models, average probs, infer per-sample order and segment accordingly
    X_tr, y_tr, _ = load_frames_from_cache_v15('train')
    models = []
    for s in seeds:
        models.append(train_seed_model_v15(X_tr, y_tr, seed=s))
    ensure_test_cache_v15()
    rows = []; t0 = time.time()
    test_ids = test_df['Id'].tolist()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V15, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0)
        order_classes, P2 = compute_order_by_centroid(P, alpha=alpha, smooth_win=smooth_win)
        seq = segment_dp_fixed_order_logodds(P2, order_classes, min_len=min_len)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[TEST-OrderSeg] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv('submission.csv', index=False)
    print('[SUBMISSION] Wrote submission.csv with', len(rows), 'rows.')
    return 'submission.csv'

print('[OrderSeg] TRAIN-only 5-seed; v1.5 stride=2; infer per-sample order (alpha=1.6, smooth=3) and segment with min_len=9 ...', flush=True)
sub_path = predict_test_order_segment_train_only(seeds=(2025,1337,42,0,123), alpha=1.6, smooth_win=3, min_len=9)
print('Submission written to:', sub_path)

[OrderSeg] TRAIN-only 5-seed; v1.5 stride=2; infer per-sample order (alpha=1.6, smooth=3) and segment with min_len=9 ...


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


[LoadV15] train: 80/297 files, cum frames=54817


...

[TEST-OrderSeg] 60/95 done; elapsed 79.2s


[TEST-OrderSeg] 70/95 done; elapsed 92.5s


[TEST-OrderSeg] 80/95 done; elapsed 105.3s


[TEST-OrderSeg] 90/95 done; elapsed 117.8s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [59]:
# 49) v1.6-only medal attempt: TRAIN-only, stride=2, 5-seed XGB, Goldilocks DP with DP-preserving rescue
print('[V16 Goldilocks] TRAIN-only 5-seed; v1.6 stride=2; T=1.0, smooth_win=3; DP lambda=0.7, min_len=9; DP-preserving rescue', flush=True)
best_params_goldilocks_v16 = {'T': 1.0, 'smooth_win': 3, 'lambda': 0.7, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 9, 'mean_thr': 0.5, 'max_thr': 0.65}
sub_path = predict_test_full_pipeline_train_only_v16(best_params_goldilocks_v16, seeds=(2025,1337,42,0,123))
print('Submission written to:', sub_path)

[V16 Goldilocks] TRAIN-only 5-seed; v1.6 stride=2; T=1.0, smooth_win=3; DP lambda=0.7, min_len=9; DP-preserving rescue


[LoadCacheV16] train: 20/297 files, cum frames=12055


[LoadCacheV16] train: 40/297 files, cum frames=24945


[LoadCacheV16] train: 60/297 files, cum frames=40473


[LoadCacheV16] train: 80/297 files, cum frames=54817


...

[TEST-V16] 60/95 done; elapsed 80.8s


[TEST-V16] 70/95 done; elapsed 94.3s


[TEST-V16] 80/95 done; elapsed 107.4s


[TEST-V16] 90/95 done; elapsed 120.2s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [60]:
# 50) Quick v1.6 OOF DP sweep (3-fold) for lambda/min_len under smooth_win=3; then optional re-infer TEST if >=0.003 gain
import os, glob, time, numpy as np
from sklearn.model_selection import KFold
import xgboost as xgb

OOF_DIR_V16 = './oof_probs_v16'
os.makedirs(OOF_DIR_V16, exist_ok=True)

def load_train_item_v16(sid:int):
    fp = os.path.join(CACHE_DIR_V16, f'train_{sid:05d}.npz')
    d = np.load(fp, allow_pickle=False)
    X = d['X']; y = d['y']
    row = train_df.loc[train_df.Id==sid]
    seq = []
    if not row.empty:
        seq = [int(x) for x in re.findall(r'\d+', str(row['Sequence'].values[0]))]
    return X, y.astype(np.int32), np.array(seq, dtype=np.int16)

def load_many_train_frames_v16(sids:list):
    Xs, ys = [], []
    n = 0
    t0 = time.time()
    for i, sid in enumerate(sids, 1):
        X, y, _ = load_train_item_v16(sid)
        Xs.append(X); ys.append(y); n += len(y)
        if i % 20 == 0:
            print(f"[LoadTrainFoldV16] {i}/{len(sids)} ids, cum frames={n}", flush=True)
    X = np.vstack(Xs); y = np.concatenate(ys)
    print(f"[LoadTrainFoldV16] Loaded {len(sids)} ids: X={X.shape} y={y.shape}", flush=True)
    return X, y

def train_fold_model_v16(X, y, seed:int=2025, num_rounds:int=900):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    bst = xgb.train(params, dtr, num_boost_round=num_rounds, verbose_eval=200)
    return bst

def build_oof_probs_v16(n_splits=3, seed=2025, num_rounds=900):
    ids = train_df['Id'].tolist()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold = 0; t_all = time.time()
    for tr_idx, va_idx in kf.split(ids):
        fold += 1
        tr_ids = [ids[i] for i in tr_idx]
        va_ids = [ids[i] for i in va_idx]
        print(f"[OOF-V16] Fold {fold}/{n_splits}: tr={len(tr_ids)} va={len(va_ids)}", flush=True)
        X_tr, y_tr = load_many_train_frames_v16(tr_ids)
        t0 = time.time()
        bst = train_fold_model_v16(X_tr, y_tr, seed=seed+fold, num_rounds=num_rounds)
        print(f"[OOF-V16] Fold {fold} model trained in {time.time()-t0:.1f}s", flush=True)
        for i, sid in enumerate(va_ids, 1):
            X_va, _, seq20 = load_train_item_v16(sid)
            P = bst.predict(xgb.DMatrix(X_va))
            outp = os.path.join(OOF_DIR_V16, f'oof_{sid:05d}.npz')
            np.savez_compressed(outp, probs=P.astype(np.float32), seq=seq20, sid=sid)
            if i % 10 == 0:
                print(f"  [OOF-V16] Fold {fold} saved {i}/{len(va_ids)}", flush=True)
    print(f"[OOF-V16] Completed {n_splits}-fold OOF in {time.time()-t_all:.1f}s. Files in {OOF_DIR_V16}")

def load_all_oof_items_v16():
    files = sorted(glob.glob(os.path.join(OOF_DIR_V16, 'oof_*.npz')))
    items = []
    for fp in files:
        d = np.load(fp, allow_pickle=False)
        sid = int(d['sid']); P = d['probs']; seq = d['seq']
        items.append((sid, P, seq))
    return items

def grid_search_dp_on_oof_v16():
    items = load_all_oof_items_v16()
    assert items, 'No OOF v16 files found. Run build_oof_probs_v16() first.'
    lambdas = [0.6, 0.7, 0.8]
    minlens = [8, 9, 10]
    bg_bias = 0.2; merge_gap = 3; mean_thr = 0.50; max_thr = 0.65
    best = (1e9, None); tried = 0; t0 = time.time()
    for lam in lambdas:
        for ml in minlens:
            scores = []
            for sid, P, seq in items:
                Pp = preprocess_probs(P, T=1.0, smooth_win=3)
                pred = decode_sequence_dp(Pp,
                                         switch_penalty=lam, bg_bias=bg_bias,
                                         merge_gap=merge_gap, min_len=ml,
                                         mean_thr=mean_thr, max_thr=max_thr)
                pred = ensure_perm20_preserve_dp(pred, Pp, alpha=1.5)
                lev = levenshtein(list(pred), list(seq.tolist()))
                scores.append(lev/20.0)
            mean_norm = float(np.mean(scores)) if scores else 1.0
            tried += 1
            if tried % 3 == 0:
                print(f"[Grid-OOF-V16] {tried} combos, curr mean={mean_norm:.4f} best={best[0]:.4f}", flush=True)
            if mean_norm < best[0]:
                best = (mean_norm, {'T':1.0,'smooth_win':3,'lambda':lam,'bg_bias':bg_bias,'merge_gap':merge_gap,'min_len':ml,'mean_thr':mean_thr,'max_thr':max_thr})
    print(f"[Grid-OOF-V16] Done {tried} combos in {time.time()-t0:.1f}s. Best={best}")
    return best

print('[RUN V16 OOF] Building 3-fold OOF on TRAIN (stride=2, v1.6)...', flush=True)
build_oof_probs_v16(n_splits=3, seed=2025, num_rounds=900)
print('[RUN V16 OOF] Grid-search DP (lambda in {0.6,0.7,0.8}, min_len in {8,9,10}) with smooth_win=3 ...', flush=True)
best_v16 = grid_search_dp_on_oof_v16()
print('Best v1.6 OOF DP params:', best_v16)

# Optional: if improvement >= 0.003 over Goldilocks (unknown exact baseline here), run TEST with best_v16
best_params = best_v16[1]
if best_params is not None:
    print('[V16 OOF->TEST] TRAIN-only 5-seed inference with OOF-best params...')
    sub_path = predict_test_full_pipeline_train_only_v16(best_params, seeds=(2025,1337,42,0,123))
    print('Submission written to:', sub_path)
else:
    print('[V16 OOF->TEST] No best params found; skipping re-inference.')

[RUN V16 OOF] Building 3-fold OOF on TRAIN (stride=2, v1.6)...


[OOF-V16] Fold 1/3: tr=198 va=99


[LoadTrainFoldV16] 20/198 ids, cum frames=12406


[LoadTrainFoldV16] 40/198 ids, cum frames=27016


[LoadTrainFoldV16] 60/198 ids, cum frames=40317


...

[TEST-V16] 60/95 done; elapsed 80.7s


[TEST-V16] 70/95 done; elapsed 94.2s


[TEST-V16] 80/95 done; elapsed 107.3s


[TEST-V16] 90/95 done; elapsed 120.1s


[SUBMISSION] Wrote submission.csv with 95 rows.
Submission written to: submission.csv


In [61]:
# 51) v1.6 10-seed ensemble + dual decodes (OOF-best smooth=3, and no-smoothing hedge); TRAIN-only, stride=2
import os, time, numpy as np, pandas as pd, xgboost as xgb

print('[V16 x10] TRAIN-only 10-seed ensemble; stride=2; decode A: OOF-best (T=1, win=3, lambda=0.6, min_len=8); decode B: hedge (T=1, win=1, lambda=0.7, min_len=9) ...', flush=True)

# Params per expert
params_A = {'T': 1.0, 'smooth_win': 3, 'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 8, 'mean_thr': 0.5, 'max_thr': 0.65}
params_B = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.7, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 9, 'mean_thr': 0.5, 'max_thr': 0.65}
# Optional third conservative hedge if time permits (BG push, longer min_len)
params_C = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.8, 'bg_bias': 0.25, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.5, 'max_thr': 0.65}

seeds10 = (2025,1337,42,0,123,7,11,77,314,2718)

def train_seed_model_v16_quick(X, y, seed:int):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    print(f"[XGB-V16][Seed {seed}] Training...", flush=True)
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=200)
    print(f"[XGB-V16][Seed {seed}] Done in {time.time()-t0:.1f}s", flush=True)
    return bst

def infer_test_avg_probs_v16(models):
    ensure_test_cache_v16()
    rows = []
    test_ids = test_df['Id'].tolist()
    t0 = time.time()
    avg_probs = {}  # sid -> np.ndarray (T,C)
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(CACHE_DIR_V16, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0).astype(np.float32)
        avg_probs[sid] = P
        if i % 10 == 0:
            print(f"[TEST-V16-Probs] {i}/{len(test_ids)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    return avg_probs

def decode_probs_to_csv(avg_probs: dict, dp_params: dict, out_csv: str):
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_df['Id'].tolist(), 1):
        P = avg_probs[sid]
        seq = decode_full_pipeline(P, dp_params, T=dp_params.get('T',1.0), smooth_win=dp_params.get('smooth_win',1), alpha_fill=1.5)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[Decode] {i}/{len(avg_probs)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv(out_csv, index=False)
    print(f"[SUBMISSION] Wrote {out_csv} with {len(rows)} rows.")
    return out_csv

# Load TRAIN v1.6 cache and train 10 seeds
X_tr, y_tr, _ = load_cached_split_v16('train')
models = []
for s in seeds10:
    models.append(train_seed_model_v16_quick(X_tr, y_tr, seed=s))

# Build averaged test probs once
avgP = infer_test_avg_probs_v16(models)

# Decode A: OOF-best smooth=3
path_A = decode_probs_to_csv(avgP, params_A, out_csv='submission_A.csv')
print('Decode A written:', path_A)
# Decode B: hedge no-smoothing
path_B = decode_probs_to_csv(avgP, params_B, out_csv='submission_B.csv')
print('Decode B written:', path_B)
# Optional C (conservative hedge) - uncomment to run if time remains
# path_C = decode_probs_to_csv(avgP, params_C, out_csv='submission_C.csv')
# print('Decode C written:', path_C)

print('[V16 x10] Done building submissions A and B from the same averaged probs.')

[V16 x10] TRAIN-only 10-seed ensemble; stride=2; decode A: OOF-best (T=1, win=3, lambda=0.6, min_len=8); decode B: hedge (T=1, win=1, lambda=0.7, min_len=9) ...


[LoadCacheV16] train: 20/297 files, cum frames=12055


[LoadCacheV16] train: 40/297 files, cum frames=24945


[LoadCacheV16] train: 60/297 files, cum frames=40473


[LoadCacheV16] train: 80/297 files, cum frames=54817


...

[Decode] 60/95 done; elapsed 0.4s


[Decode] 70/95 done; elapsed 0.4s


[Decode] 80/95 done; elapsed 0.5s


[Decode] 90/95 done; elapsed 0.5s


[SUBMISSION] Wrote submission_B.csv with 95 rows.
Decode B written: submission_B.csv
[V16 x10] Done building submissions A and B from the same averaged probs.


In [62]:
# 51b) Replace submission.csv with 10-seed v1.6 hedge decode (B: no-smoothing)
import shutil, os
src = 'submission_B.csv'; dst = 'submission.csv'
assert os.path.exists(src), f"Missing {src}; run cell 52 first"
shutil.copyfile(src, dst)
print(f'[Swap] {dst} <- {src} ({os.path.getsize(dst)} bytes)')

[Swap] submission.csv <- submission_B.csv (5237 bytes)


In [63]:
# 51c) Replace submission.csv with 10-seed v1.6 decode A (OOF-best smooth=3)
import shutil, os
src = 'submission_A.csv'; dst = 'submission.csv'
assert os.path.exists(src), f"Missing {src}; run cell 52 first"
shutil.copyfile(src, dst)
print(f'[Swap] {dst} <- {src} ({os.path.getsize(dst)} bytes)')

[Swap] submission.csv <- submission_A.csv (5237 bytes)


In [64]:
# 51d) Decode C: conservative no-smoothing hedge from existing avg_probs (no retrain)
import os, pandas as pd, numpy as np
print('[V16 x10 Decode C] Using existing averaged test probs in memory (avgP) ...', flush=True)
assert 'avgP' in globals(), 'avgP not found in memory; run cell 52 first to build 10-seed avg probs.'
params_C = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.8, 'bg_bias': 0.25, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.5, 'max_thr': 0.65}
path_C = decode_probs_to_csv(avgP, params_C, out_csv='submission_C.csv')
print('Decode C written:', path_C)
import shutil
shutil.copyfile('submission_C.csv', 'submission.csv')
print('[Swap] submission.csv <- submission_C.csv (', os.path.getsize('submission.csv'), 'bytes)')

[V16 x10 Decode C] Using existing averaged test probs in memory (avgP) ...


[Decode] 10/95 done; elapsed 0.1s


[Decode] 20/95 done; elapsed 0.1s


[Decode] 30/95 done; elapsed 0.2s


[Decode] 40/95 done; elapsed 0.2s


...

[Decode] 60/95 done; elapsed 0.4s


[Decode] 70/95 done; elapsed 0.4s


[Decode] 80/95 done; elapsed 0.5s


[Decode] 90/95 done; elapsed 0.5s


[SUBMISSION] Wrote submission_C.csv with 95 rows.
Decode C written: submission_C.csv
[Swap] submission.csv <- submission_C.csv ( 5237 bytes)


In [65]:
# 52) v1.5 + v1.6 blend (weights v16=0.7, v15=0.3): reuse in-memory avgP (v1.6), build v1.5 5-seed avg probs, then dual decodes
import os, time, numpy as np, pandas as pd, xgboost as xgb
print('[Blend v15+v16] Start: weights v16=0.7, v15=0.3; TRAIN-only; stride=2', flush=True)

assert 'avgP' in globals(), 'avgP (v1.6 avg probs) not found; rerun cell 52 to build 10-seed v1.6 probs first.'
w16, w15 = 0.7, 0.3
params_A = {'T': 1.0, 'smooth_win': 3, 'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 8, 'mean_thr': 0.5, 'max_thr': 0.65}
params_B = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.7, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 9, 'mean_thr': 0.5, 'max_thr': 0.65}

def train_seed_model_v15_quick(X, y, seed:int):
    w = make_weights_with_boundary_erosion(y, w0=0.38)
    dtr = xgb.DMatrix(X, label=y, weight=w)
    params = {
        'objective': 'multi:softprob',
        'num_class': 21,
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'max_bin': 512,
        'max_depth': 7,
        'eta': 0.085,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'min_child_weight': 4.0,
        'lambda': 1.0,
        'seed': int(seed)
    }
    print(f"[XGB-V15][Seed {seed}] Training...", flush=True)
    t0 = time.time()
    bst = xgb.train(params, dtr, num_boost_round=1100, verbose_eval=200)
    print(f"[XGB-V15][Seed {seed}] Done in {time.time()-t0:.1f}s", flush=True)
    return bst

def infer_test_avg_probs(cache_dir, models):
    test_ids = test_df['Id'].tolist()
    avg_probs = {}; t0 = time.time()
    for i, sid in enumerate(test_ids, 1):
        d = np.load(os.path.join(cache_dir, f'test_{sid:05d}.npz'), allow_pickle=False)
        X = d['X']; dm = xgb.DMatrix(X)
        Ps = [m.predict(dm) for m in models]
        P = np.mean(np.stack(Ps, axis=0), axis=0).astype(np.float32)
        avg_probs[sid] = P
        if i % 10 == 0:
            print(f"[Infer-Probs] {i}/{len(test_ids)} from {os.path.basename(cache_dir)}; elapsed {time.time()-t0:.1f}s", flush=True)
    return avg_probs

def decode_probs_to_csv(avg_probs: dict, dp_params: dict, out_csv: str):
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_df['Id'].tolist(), 1):
        P = avg_probs[sid]
        seq = decode_full_pipeline(P, dp_params, T=dp_params.get('T',1.0), smooth_win=dp_params.get('smooth_win',1), alpha_fill=1.5)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[Decode] {i}/{len(avg_probs)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv(out_csv, index=False)
    print(f"[SUBMISSION] Wrote {out_csv} with {len(rows)} rows.")
    return out_csv

# Build v1.5 5-seed avg probs
print('[Blend] Building v1.5 avg probs (5 seeds)...', flush=True)
X15_tr, y15_tr, _ = load_frames_from_cache_v15('train')
models15 = []
for s in (2025,1337,42,0,123):
    models15.append(train_seed_model_v15_quick(X15_tr, y15_tr, seed=s))
ensure_test_cache_v15()
avgP15 = infer_test_avg_probs(CACHE_DIR_V15, models15)

# Blend per-frame probs: Pblend = 0.7*P16 + 0.3*P15 (align lengths by interpolation if needed)
def interp_probs(P: np.ndarray, N: int) -> np.ndarray:
    T, C = P.shape
    if T == N: return P.astype(np.float32)
    x_old = np.linspace(0.0, 1.0, T, dtype=np.float32)
    x_new = np.linspace(0.0, 1.0, N, dtype=np.float32)
    out = np.empty((N, C), dtype=np.float32)
    for c in range(C):
        out[:, c] = np.interp(x_new, x_old, P[:, c].astype(np.float32))
    s = out.sum(axis=1, keepdims=True) + 1e-9
    out /= s
    return out

avg_blend = {}
t0 = time.time()
for i, sid in enumerate(test_df['Id'].tolist(), 1):
    P16 = avgP[sid]
    P15 = avgP15[sid]
    N = max(P16.shape[0], P15.shape[0])
    P16i = interp_probs(P16, N)
    P15i = interp_probs(P15, N)
    avg_blend[sid] = (w16*P16i + w15*P15i).astype(np.float32)
    if i % 10 == 0:
        print(f"[Blend] {i}/{len(avgP15)} done; elapsed {time.time()-t0:.1f}s", flush=True)

# Decode A: smooth_win=3 OOF-best style
path_A = decode_probs_to_csv(avg_blend, params_A, out_csv='submission_blend_A.csv')
print('Decode A written:', path_A)
# Decode B: hedge no-smoothing
path_B = decode_probs_to_csv(avg_blend, params_B, out_csv='submission_blend_B.csv')
print('Decode B written:', path_B)

# Default: swap A to submission.csv
import shutil
shutil.copyfile('submission_blend_A.csv', 'submission.csv')
print('[Swap] submission.csv <- submission_blend_A.csv ({} bytes)'.format(os.path.getsize('submission.csv')))

[Blend v15+v16] Start: weights v16=0.7, v15=0.3; TRAIN-only; stride=2


[Blend] Building v1.5 avg probs (5 seeds)...


[LoadV15] train: 20/297 files, cum frames=12055


[LoadV15] train: 40/297 files, cum frames=24945


[LoadV15] train: 60/297 files, cum frames=40473


...

[Decode] 60/95 done; elapsed 0.4s


[Decode] 70/95 done; elapsed 0.4s


[Decode] 80/95 done; elapsed 0.5s


[Decode] 90/95 done; elapsed 0.5s


[SUBMISSION] Wrote submission_blend_B.csv with 95 rows.
Decode B written: submission_blend_B.csv
[Swap] submission.csv <- submission_blend_A.csv (5237 bytes)


In [66]:
# 53) Fast blends without retraining: reuse in-memory avgP (v1.6, 10-seed) and avgP15 (v1.5, 5-seed)
import os, time, numpy as np, pandas as pd
print('[FastBlends] Reusing in-memory avgP (v1.6) and avgP15 (v1.5) to write multiple blended submissions...', flush=True)
assert 'avgP' in globals(), 'avgP not found (v1.6 avg probs). Re-run cell 52 (10-seed v1.6) if needed.'
assert 'avgP15' in globals(), 'avgP15 not found (v1.5 avg probs). Re-run cell 56 (v1.5 5-seed build) if needed.'

def interp_probs(P: np.ndarray, N: int) -> np.ndarray:
    T, C = P.shape
    if T == N: return P.astype(np.float32)
    x_old = np.linspace(0.0, 1.0, T, dtype=np.float32)
    x_new = np.linspace(0.0, 1.0, N, dtype=np.float32)
    out = np.empty((N, C), dtype=np.float32)
    for c in range(C):
        out[:, c] = np.interp(x_new, x_old, P[:, c].astype(np.float32))
    s = out.sum(axis=1, keepdims=True) + 1e-9
    out /= s
    return out

def decode_probs_to_csv(avg_probs: dict, dp_params: dict, out_csv: str):
    rows = []; t0 = time.time()
    for i, sid in enumerate(test_df['Id'].tolist(), 1):
        P = avg_probs[sid]
        seq = decode_full_pipeline(P, dp_params, T=dp_params.get('T',1.0), smooth_win=dp_params.get('smooth_win',1), alpha_fill=1.5)
        rows.append({'Id': sid, 'Sequence': ' '.join(str(int(x)) for x in seq)})
        if i % 10 == 0:
            print(f"[Decode-Blends] {i}/{len(avg_probs)} done; elapsed {time.time()-t0:.1f}s", flush=True)
    sub = pd.DataFrame(rows, columns=['Id','Sequence'])
    sub.to_csv(out_csv, index=False)
    print(f"[SUBMISSION] Wrote {out_csv} with {len(rows)} rows.")
    return out_csv

# Param sets
params_A = {'T': 1.0, 'smooth_win': 3, 'lambda': 0.6, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 8, 'mean_thr': 0.5, 'max_thr': 0.65}
params_B = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.7, 'bg_bias': 0.2, 'merge_gap': 3, 'min_len': 9, 'mean_thr': 0.5, 'max_thr': 0.65}
params_C = {'T': 1.0, 'smooth_win': 1, 'lambda': 0.8, 'bg_bias': 0.25, 'merge_gap': 3, 'min_len': 10, 'mean_thr': 0.5, 'max_thr': 0.65}

weights = [0.8, 0.7, 0.6]  # w16; w15 = 1 - w16
for w16 in weights:
    w15 = 1.0 - w16
    print(f'[FastBlends] Building blended probs for w16={w16:.1f}, w15={w15:.1f} ...', flush=True)
    avg_blend = {}
    t0 = time.time()
    for i, sid in enumerate(test_df['Id'].tolist(), 1):
        P16 = avgP[sid]; P15 = avgP15[sid]
        N = max(P16.shape[0], P15.shape[0])
        P16i = interp_probs(P16, N)
        P15i = interp_probs(P15, N)
        avg_blend[sid] = (w16*P16i + w15*P15i).astype(np.float32)
        if i % 10 == 0:
            print(f"  [Blend-{w16:.1f}] {i}/95 done; elapsed {time.time()-t0:.1f}s", flush=True)
    # Decode three variants for this weight
    outA = decode_probs_to_csv(avg_blend, params_A, out_csv=f'submission_blend_w{int(w16*10)}_A.csv')
    outB = decode_probs_to_csv(avg_blend, params_B, out_csv=f'submission_blend_w{int(w16*10)}_B.csv')
    outC = decode_probs_to_csv(avg_blend, params_C, out_csv=f'submission_blend_w{int(w16*10)}_C.csv')
    print('[FastBlends] Wrote:', outA, outB, outC)

# Default: copy one candidate (w16=0.8, A) to submission.csv for immediate try; can swap later
import shutil
src_default = 'submission_blend_w8_A.csv'
if os.path.exists(src_default):
    shutil.copyfile(src_default, 'submission.csv')
    print(f'[Swap] submission.csv <- {src_default} ({os.path.getsize("submission.csv")} bytes)')
else:
    print('[FastBlends] Default file not found; please pick a specific output to copy into submission.csv.')

[FastBlends] Reusing in-memory avgP (v1.6) and avgP15 (v1.5) to write multiple blended submissions...


[FastBlends] Building blended probs for w16=0.8, w15=0.2 ...


  [Blend-0.8] 10/95 done; elapsed 0.0s


  [Blend-0.8] 20/95 done; elapsed 0.0s


  [Blend-0.8] 30/95 done; elapsed 0.0s


...

[Decode-Blends] 30/95 done; elapsed 0.2s


[Decode-Blends] 40/95 done; elapsed 0.2s


[Decode-Blends] 50/95 done; elapsed 0.3s


[Decode-Blends] 60/95 done; elapsed 0.4s


[Decode-Blends] 70/95 done; elapsed 0.4s
